## Загрузка библиотек

In [1]:
#basic
import pandas as pd
from typing import Dict
import numpy as np
import re

#error
import warnings
warnings.filterwarnings('ignore')

## Загрузка данных

#### Напишем функцию, с помощью которой будем загружать данные

> Так как у нас таблицы из разных источников, то и форматирование у них разное, в том числе начало строк с информацией по блогерам можем быть на 10 строчки, а название столбцов у разных источников отличаться. Чтобы привести к единому формату, мы в фунции укажет ключи, по которому обрежем таблицу снизу и сверху

In [2]:
def read_csv_from_dict(data_dict: Dict[str, str], column_number: int, header_keyword: str, footer_keyword: str) -> pd.DataFrame:
    
    
    """
    Функция принимает как параметр словарь data_dict, в котором ключ - название DataFrame, а объект - ссылка на источник,
    и в дальнейшем мы обрезаем DataFrame с помощью параметров column_number, header_keyword, footer_keyword.
    
    ПАРАМЕТРЫ:
    - data_dict (DataFrame) - словарь, в котором ключ - это будущее название датафрейма, объект - 
    ссылка на страницу таблицы, из которой берём данные.
    - column_number (int) - номер столбца, по которому определяем границы DataFrame.
    - header_keyword (str) - ключ, по которому мы обрезаем таблице сверху.
    - footer_keyword - ключ, по которому мы обрезаем таблице снизу.
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df: DataFrame, название и источник которого указан в словаре data_dict и который обрезан по нашим параметрам header_keyword и footer_keyword
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    didenok_df = read_csv_from_dict(didenok_data_dict, 0, didenok_header_keyword, didenok_footer_keyword)
    """
    
    for name, url in data_dict.items():
        url = url.replace('/edit#gid=', '/export?format=csv&gid=')
        df = pd.read_csv(url)

# В pandas метод .iloc используется для доступа к элементам в DataFrame или Series по их целочисленным позициям.
# Индексация начинается с нуля, поэтому .iloc[n] обращается к элементу с позицией n-1.        
    
        header_index = df.loc[df.iloc[:, column_number].str.strip().isin(header_keyword)].index[0]
        
#Используем globals() для создания новой переменной с именем, которое хранится в переменной name,
# и присваиваем этой переменной значение, полученное в результате операции df.set_axis(df.iloc[header_index], axis='columns').

        globals()[name] = df.set_axis(df.iloc[header_index], axis='columns')

        if footer_keyword:
            footer_index = df.loc[df.iloc[:, column_number].str.strip().isin(footer_keyword)].index[0] - 2
            globals()[name] = globals()[name].iloc[header_index + 1:footer_index]
        else:
            globals()[name] = globals()[name].iloc[header_index + 1:]
            
    return df

In [4]:
print(read_csv_from_dict.__doc__)


    Функция принимает как параметр словарь data_dict, в котором ключ - название DataFrame, а объект - ссылка на источник,
    и в дальнейшем мы обрезаем DataFrame с помощью параметров column_number, header_keyword, footer_keyword.
    
    ПАРАМЕТРЫ:
    - data_dict (DataFrame) - словарь, в котором ключ - это будущее название датафрейма, объект - 
    ссылка на страницу таблицы, из которой берём данные.
    - column_number (int) - номер столбца, по которому определяем границы DataFrame.
    - header_keyword (str) - ключ, по которому мы обрезаем таблице сверху.
    - footer_keyword - ключ, по которому мы обрезаем таблице снизу.
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df: DataFrame, название и источник которого указан в словаре data_dict и который обрезан по нашим параметрам header_keyword и footer_keyword
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    didenok_df = read_csv_from_dict(didenok_data_dict, 0, didenok_header_keyword, didenok_footer_keyword)
    


> Стоит отметить, что так как исходные данные у нас лежат на Google-таблицах, то помимо разного формата, мы ещё столнулись с тем, что часть ссылок сделаны гиперссылками в виде текста, которые Pandas и специальные сторонние библиотеки переводят текст. Поэтому чтобы не разводить лишний код, мы сделали определённые преобразования в Google-таблицах, а именно:
>* Через функцию IMPORTRANGE копируем данные из разных Google-таблиц в единую.
>* Написали функцию для App Script на специальном языке google script (gs), которая преобразовывает гиперссылки в ссылки и выводит их в отдельный столбец (пример функции ниже):
>
><em>function GetURL(input) {  
  var myFormula = SpreadsheetApp.getActiveRange().getFormula();  
  var myAddress = myFormula.replace('=GetURL(','').replace(')','');   
  var myRange = SpreadsheetApp.getActiveSheet().getRange(myAddress);  
  return myRange.getRichTextValue().getLinkUrl();  
 };</em>  
> 
>**За счёт этого мы**:
>* Имеем единую Google-таблицу, в которой видим все предварительные данные, в том числе если будут какие-то изменения в первоначальных данным, то мы это увидим, что позволит нам избежать ошибки.
>* Решили проблему с гиперссылками.

### Блогеры от Didenok Team

In [5]:
didenok_data_dict = {'didenok_instagram':'HREF',  
                     'didenok_youtube':'HREF',
                     'didenok_telegram':'HREF',
                     'didenok_vk_zen_show':'HREF',
                     'didenok_vk_group':'HREF',
#                      'didenok_vk_ls':'HREF',
                     'didenok_twitch':'HREF'
                    }

In [6]:
didenok_header_keyword = ['Блогер', 'Канал']
didenok_footer_keyword = ['Примечание']

In [7]:
didenok_df = read_csv_from_dict(didenok_data_dict,
                                0,
                                didenok_header_keyword,
                                didenok_footer_keyword)

> Проверим результат

In [8]:
didenok_instagram.head()

7,Блогер,Ссылка,Категория/\nТематика блогера,Вид сотрудничества**,Количество подписчиков,Средний охват фотопоста*,Средний охват 1 сторис*,Среднее кол-во просмотров Reels*,Среднее кол-во лайков*,Среднее кол-во комментариев*,"ERR, %",Стоимость за фотопост (руб.) до НДС,Стоимость за 1 сторис до 15 сек (руб.) до НДС,Стоимость за серию из 3х сториc до 45 сек (руб.) до НДС,Стоимость за Reels (руб.) до НДС,CPU (фото),CPU (сторис),CPV \n(Reels),Статистика,Статистика
8,Катя Клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,П,10 100 000,250 000,150 000,200 000,50 000,100,"20,04%",р.375 000,р.335 000,р.530 000,р.820 000,"р.1,5","р.2,2","р.4,1",Статистика,https://drive.google.com/drive/folders/1J26W2R...
9,Влад Бумага А4,https://www.instagram.com/a4omg/,челленджи,П,7 100 000,900 000,400 000,1 000 000,60 000,700,"6,74%",р.875 000,р.440 000,р.625 000,р.1 875 000,"р.1,0","р.1,1","р.1,9",-,NaN
10,Катя Адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,Э,6 300 000,700 000,260 000,600 000,30 000,100,"4,30%",р.445 000,р.305 000,р.625 000,р.685 000,"р.0,6","р.1,2","р.1,1",Статистика,https://drive.google.com/drive/u/1/folders/11S...
11,Амина Тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,Э,3 900 000,600 000,300 000,650 000,110 000,150,"18,36%",р.695 000,р.265 000,р.625 000,р.835 000,"р.1,2","р.0,9","р.1,3",Статистика,https://drive.google.com/drive/u/1/folders/1M3...
12,VikiShow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,Э,2 500 000,70 000,30 000,80 000,15 000,200,"21,71%",р.120 000,р.60 000,р.135 000,р.155 000,"р.1,7","р.2,0","р.1,9",Статистика,https://drive.google.com/drive/folders/1haDuDs...


### Блогеры от VK

In [9]:
vk_data_dict = {'vk_bloggers':\
                'HREF'
               }

In [10]:
vk_header_keyword = ['Сообщество']
vk_footer_keyword = []

In [11]:
vk_df = read_csv_from_dict(vk_data_dict,
                           0,
                           vk_header_keyword,
                           vk_footer_keyword
                          )

> Проверим результат

In [12]:
vk_bloggers.head()

,Сообщество,Шоу,KPI (стартов просмотра на 1 эпизод на 30-е сутки после релиза),Стоимость интеграции 1-й слот до НДС,Стоимость интеграции 2-й слот до НДС,Продакт плейсмент (30 сек) до НДС,Стоимость рекламной заставки (пре-ролл до 20 сек) до НДС,Стоимость рекламной заставки (мид-ролл до 20 сек) до НДС,Статистика,Контакты менеджера по продажам,Ссылка,Статистика ссылка
1,Азамат Мусагалиев,Большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,Статистика,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,Это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,NaN,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,Я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,NaN,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,Шоу памяти Азамата Мусагалиева NEW,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,NaN,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,Пикабу,Чебатков читает Пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


### Блогеры от BlogoSphere

In [13]:
blogosphere_data_dict = {'blogosphere_youtube':'HREF',
                        'blogosphere_tg_vk_zen':'HREF'
                        }

In [14]:
blogosphere_header_keyword = ['Название канала', 'Имя блогера']
blogosphere_footer_keyword = []

In [15]:
blogosphere_df = read_csv_from_dict(blogosphere_data_dict,
                                    1,
                                    blogosphere_header_keyword,
                                    blogosphere_footer_keyword)

> Проверим результат

In [16]:
blogosphere_youtube.head()

5,NaN,Название канала,Шоу,Подписчики,Среднее кол-во просмотров на видео,Статистика аудитории,Стандартные форматы,NaN,NaN,NaN,...,NaN,NaN,NaN,Пример рекламы 1,Пример рекламы 2,Пример рекламы 3,Среднее CPV,NaN,Ссылка,Статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,...,NaN,Формат 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,...,Цена* от,Описание,Цена* от,NaN,NaN,NaN,Стандартный Формат 1,Стандартный Формат 2,NaN,NaN
8,Э,NaN,"Осторожно, Собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,...,1 310 000 ₽,Product Placement,1 470 000 ₽,Eduson,Экоплант,Травелата,"1,31₽","0,96₽",NaN,NaN
9,NaN,Ксения Собчак,Осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,...,NaN,NaN,NaN,Astons,CGPods,SkillFactory,"1,38₽","0,93₽",https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,NaN,Красота требует КЭШ,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,...,NaN,NaN,NaN,Золотое яблоко,4FORMS,FOREO,"1,20₽","1,08₽",NaN,NaN


### Блогеры от Insight People

In [17]:
insight_data_dict = {'insight_instagram':'HREF',
                     'insight_threads':'HREF',
                     'insight_telegram':'HREF',
                     'insight_vk_ls':'HREF',
                     'insight_vk_group':'HREF',
                     'insight_youtube':'HREF',
                     'insight_zen':'HREF',
                     'insight_likee':'HREF',
                     'insight_yappy':'HREF',
                     'insight_rutube':'HREF',
#                      'insight_ok':'HREF'
                    }

In [18]:
insight_header_keyword = ['Город']
insight_footer_keyword = []

In [19]:
insight_df = read_csv_from_dict(insight_data_dict,
                                0,
                                insight_header_keyword,
                                insight_footer_keyword)

> Проверим результат

In [20]:
insight_rutube.head()

3,Город,ФИО,Ссылка,Количество подписчиков,Средние просмотры за месяц,Стоимость преролла,Стоимость интеграции (до 1 минуты),"CPV, преролл","CPV, интеграция"
4,Тверь,Аркатов Станислав Александрович,https://rutube.ru/channel/23684374/,Фудблогер,118 300,25 030,"р.50 060,00","р.0,75","р.2,00"
5,Краснодар,Солодов Юрий Юрьевич,https://rutube.ru/channel/23722477/,"Лайфстайл, влог",24 500,47 000,"р.82 250,00","р.0,50","р.1,75"
6,Казань,Ахмадуллин Наиль Радикович,https://rutube.ru/channel/23694775/,"Путешествия, лайфстайл",18 500,27 524,под запрос,-,-
7,Воронеж,Рязанова Татьяна Алексеевна,https://rutube.ru/channel/19363597/,"Бьюти, лайфстайл",16 500,50 066,"р.87 615,00","р.0,75","р.2,00"
8,Воронеж,Лаврова Валерия Владимировна,https://rutube.ru/channel/26070201/,Художник,15 600,40 063,"р.70 115,00","р.0,75","р.2,00"


### Блогеры от NextUp

In [21]:
nextup_data_dict = {'nextup_instagram':'HREF',
                    'nextup_telegram':'HREF',
                    'nextup_youtube':'HREF',
                    'nextup_twitch':'HREF',
                    'nextup_vk':'HREF',
                   }

In [22]:
nextup_header_keyword = ['Имя блогера']
nextup_footer_keyword = []

In [23]:
nextup_df = read_csv_from_dict(nextup_data_dict,
                               2,
                               nextup_header_keyword,
                               nextup_footer_keyword
                              )

> Проверим результат

In [24]:
nextup_vk.head()

5,NaN,NaN,Имя блогера,Ссылка,Кол-во подписчиков,Статистика,ЦА по городам,ЦА по возрасту,ЦА по полу,Охват поста за 30 дней,Охват VK клип за 30 дней,Стоимость поста,CPV пост,Стоимость VK клип,CPV Клип,Статистика ссылка
6,NaN,П,Hardplay,https://vk.com/hp_legion,407 000,NaN,NaN,NaN,NaN,10 000,—,72 000 ₽,"7,20 ₽",—,—,NaN
7,NaN,П,Paradeev1ch,https://vk.com/paradeev1ch,414 000,NaN,NaN,NaN,NaN,45 000,-,78 500 ₽,NaN,71 500 ₽,NaN,NaN
8,NaN,П,Маш Милаш,https://vk.com/milash,92 000,NaN,NaN,NaN,NaN,NaN,NaN,79 000 ₽,NaN,71 500 ₽,NaN,NaN
9,NaN,Э,Ира (inst_rinna),https://vk.com/inst_rinnaaaa,113 000,статистика,Москва – 26%\nСПб – 11%\nКраснодар – 4%,"13–17 лет – 17,5%\n18–24 года – 15%\n25–34 год...",Мужской – 68%\nЖенский – 32%,60 000,40 000,72 000 ₽,"1,20 ₽",72 000 ₽,"1,80 ₽",https://drive.google.com/drive/folders/1-3rUVS...
10,NaN,П,Ярослав (Дилблин),https://vk.com/dilblinnn,87 000,статистика,Москва – 21%\nСПб – 11%\nУфа – 4%,13–17 лет – 35%\n18–24 года – 17%\n25–34 года ...,Мужской – 34%\nЖенский – 66%,100 000,–,80 000 ₽,"0,80 ₽",–,–,https://drive.google.com/drive/folders/15UUvEu...


### Блогеры от AvtorMedia

In [25]:
# nextup_artist_data_dict = {'nextup_artist':\
#                     'HREF'}

In [26]:
# nextup_artist_header_keyword = ['Сообщество']
# nextup_artist_footer_keyword = []

In [27]:
avtormedia_data_dict = {'avtormedia_youtube':'HREF',
                        'avtormedia_instagram':'HREF',
                        'avtormedia_telegram':'HREF',
                        #'avtormedia_tiktok':''
                        'avtormedia_vk':'HREF',
                        'avtormedia_zen':'HREF',
                        'avtormedia_podcast':'HREF'
                       }

In [28]:
avtormedia_header_keyword = ['Блогер']
avtormedia_footer_keyword = []

In [29]:
avtormedia_df = read_csv_from_dict(avtormedia_data_dict,
                                   0,
                                   avtormedia_header_keyword,
                                   avtormedia_footer_keyword
                                  )

> Проверим результат

In [30]:
avtormedia_telegram.head()

,Блогер,Тематика,Подписчики,Материалы\nи статистика,Средние просмотры,Короткое видео,Пост,Спешл,Цена,Спешл,...,Пакетное предложение,Цена,Охват,CPV,Пакетное предложение,Цена,Охват,CPV,Ссылка,Статистика ссылка
1,Илья Варламов @varlamov_news,новости,1 200 000,Ссылка,470 000,833 333 ₽,450 000 ₽,Сторис,500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov_news,https://drive.google.com/drive/u/1/folders/165...
2,Mamix,эксперименты,337 000,Ссылка,261 000,NaN,425 531 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/its_mamix,https://drive.google.com/drive/u/1/folders/12S...
3,Илья Варламов @varlamov,урбанистика\nобщество\nполитика,329 000,Ссылка,212 000,888 888 ₽,600 000 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov,https://drive.google.com/drive/u/1/folders/165...
4,Венедиктов,общество\nполитика,223 000,Ссылка,131 000,NaN,250 000 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/aavst2022,https://drive.google.com/drive/folders/1UtGUA_...
5,Даша Козловская,творчество\nбьюти\nлайфстайл,186 400,Ссылка,81 000,430 000 ₽,350 000 ₽,NaN,NaN,NaN,...,Фото-пост в Инст + пост в Telegram,1 402 009 ₽,362 000,"3,87",NaN,NaN,NaN,NaN,https://t.me/dashakozlouskayatg,https://drive.google.com/drive/u/1/folders/1cW...


### Блогеры от PLATFORMA.COMMUNITY

In [31]:
sheet_url = 'HREF'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_youtube = pd.read_csv(csv_export_url, header=1)

sheet_url = 'HREF'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_instagram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'HREF'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_telegram = pd.read_csv(csv_export_url, header=0)

In [32]:
platformacommunity_youtube.head()

,Вид контракта,Канал,Проект\nВедущий\nФормат,Тематика,Подписчики,Средние просмотры для рекламных интеграций,Статистика,"Pre-roll / OLV, от, руб., до учета налогов","CPV, pre-roll, от, руб., до учета налогов","MID-roll / OLV, от, руб., до учета налогов","Интеграция 1-ый слот, от, руб., до учета налогов","CPV, 1 слот, от, руб., до учета налогов","Интеграция 2-ой слот, от, руб., до учета налогов","CPV, 2 слот, от, руб., до учета налогов","Продакт плейсмент, от, руб., до учета налогов","Размещение в сообществе, от, руб., до учета налогов",Пример,Ссылка,Статистика ссылка
0,Э,Уральские Пельмени,Официальный канал шоу «Уральские Пельмени»,Специальные сборки номеров под рекламодателя,"2 550 000,00","2 000 000,00",Статистика,"1 200 000,00","0,60",NaN,"2 500 000,00","1,25",NaN,NaN,NaN,NaN,Тинькофф,https://www.youtube.com/c/ProPelmeny/featured,https://drive.google.com/drive/folders/1vQIyBd...
1,NaN,NaN,NaN,Эксклюзивные номера,NaN,"1 000 000,00",NaN,"700 000,00","0,70",NaN,NaN,"0,00",NaN,NaN,NaN,NaN,Пятерочка,NaN,NaN
2,NaN,NaN,NaN,Регулярные выкладки,NaN,"700 000,00",NaN,"550 000,00","0,79",NaN,NaN,NaN,NaN,NaN,NaN,NaN,HAVAL,NaN,NaN
3,NaN,NaN,NaN,"""ЕДЕМ С ПЕЛЬМЕНЯМИ"" travel шоу про гастроли и ...",NaN,"300 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"800 000,00",NaN,Орматек,NaN,NaN
4,NaN,ВПИСКА,Коля Редькин,Интервью,"1 830 000,00","2 500 000,00",Статистика,"1 500 000,00",NaN,по запросу,по запросу,#VALUE!,NaN,"0,00",по запросу,NaN,NaN,https://www.youtube.com/@VPISKA/featured,https://drive.google.com/drive/folders/1KxM-Mc...


### Блогер ТОПЛЕС

In [33]:
sheet_url = 'HREF'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_telegram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'HREF'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_instagram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'HREF'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_vk = pd.read_csv(csv_export_url, header=0)

In [34]:
toples_telegram.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Telegram-канал Топлес,NaN,NaN,NaN,NaN
2,NaN,https://t.me/toplesofficial,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Подписчики: 500К,NaN,NaN,NaN,NaN
5,NaN,Охват поста: 120-130К,NaN,NaN,NaN,Условия актуальны до 30.11.2023
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Формат публикации,NaN,NaN,Подробности формата,Стоимость (от)
8,NaN,1,Текстовый пост\n\nМатериалы клиента,- текст от клиента\n- визуал от клиента (по не...,"Материалы клиента, при необходимости адаптируе...",р.85 000
9,NaN,2,Текстовый пост\n\nПишем сами по брифу клиента,- текст по брифу\n- визуал от клиента (по необ...,Текст пишем в стиле канала по брифу клиента. Т...,р.100 000


## Подготовка данных


#### Функция, с помощью которой мы предварительно подготовим DataFrame для дальнейшей работы

In [35]:
def pretraining_df(df: pd.DataFrame, drop_keywords: str, drop_word: str, upd_columns: str) -> pd.DataFrame:

    """
    Функция принимает DataFrame - df, с которым мы делаем преобразования:
    • Редактируем названия столбцов: приводим к нижнему регистру, убираем лишние пробелы до и после текста, удаляем перенос \n и сноски *.
    • Удаляем столбцы по ключевым словам drop_keywords. В коде учтено, что их может и не быть.
    • Удаляем ключевые слова drop_word из названия столбцов. В коде учтено, что их может и не быть.
    • Столбцы upd_columns, в которых содержится текстовая информация для дальнейшего объединения разных DataFrame,
    приводим к единому формату по примеру первого пункта, а также заполняем NaN с помощью 

    
    ПАРАМЕТРЫ:
    df (DataFrame) - DataFrame, в котором мы будем вносить изменения
    drop_keywords (str) - список ключевых слов для удаления столбцов
    drop_word (str) - слово для удаления из названий столбцов
    upd_columns (str) - список имен столбцов для приведения к нижнему регистру и удаления символов
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df - DataFrame, в котором мы делали преобразования
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    key_word = [] # Список ключевых слов для поиска столбцов и их удаления
    drop_word = ' до ндс' # Список ключевых слов для их удаления из названия столбцов
    upd_columns = ['сообщество', 'шоу'] # Список столбцов с именами блогеров для приведения к единому формату
    upd_columns = [column.lower() for column in upd_columns]

    vk_bloggers = pretraining_df(vk_bloggers, key_word, drop_word, upd_columns)
    
    """
    
    drop_keywords = [keywords.lower() for keywords in drop_keywords] #
    drop_keywords = [keyword.strip() for keyword in drop_keywords]
    drop_word = drop_word.lower()
    upd_columns = [column.lower() for column in upd_columns] #
    
    
    df.columns = df.columns.str.strip()\
                            .str.lower()\
                            .str.replace('/\n', ', ')\
                            .str.replace('\n', ', ')\
                            .str.replace('*', '')
    
# Найдём столбцы по ключевым словам и их удалим их, так как в DataFrame у нас встречаются столбцы,
# которые не несут уникальной информации, например cpm, cpv и тд,
# и которые мы можем вычислить точнее через математические операции.


    if drop_keywords:
        columns_to_drop = [column for column in df.columns if isinstance(column, str) and any(keyword in column for keyword in drop_keywords)]
        df.drop(columns=columns_to_drop, axis=1, inplace=True)

        
    
# Удалим ключевые слова из названия столбцов
    if drop_word:
        df.columns = df.columns.str.replace(drop_word, '', regex=False)


# Столбцы, в которых содержится информация для дальнейшего объединения, приведём к единому формату
    for upd_column in upd_columns:
        df[upd_column] = df[upd_column].fillna('').apply(lambda x: str(x).strip().lower()\
                                                                         .replace('/\n', ', ')\
                                                                         .replace('\n', ', ')\
                                                                         .replace('*', '')\
                                                             if isinstance(x, str) else x)
        # Заменим пустую строку на NaN
        df[upd_column] = df[upd_column].replace('', np.nan)
    
    return df

## Система наименований столбцов

## Подготовка данных по блогерам от Didenok Team

#### Имеющиеся датафреймы:  

* didenok_instagram  
* didenok_youtube  
* didenok_telegram  
* didenok_vk_zen_show  
* didenok_vk_group  
<!-- * didenok_vk_ls   -->
* didenok_twitch  

> Сделаем список DataFrame, чтобы в дальнейшем работать с ним

In [36]:
didenok_df = [didenok_instagram,
              didenok_youtube,
              didenok_telegram,
              didenok_vk_zen_show,
              didenok_vk_group,
#               didenok_vk_ls,
              didenok_twitch]

In [37]:
# key_word = ['вид сотрудничества', 'cpv', 'cpu'] # Список ключевых слов для поиска столбцов и их удаления
# drop_word = ' (руб.) до ндс' # Список ключевых слов для их удаления из названия столбцов
# upd_columns = ['блогер', 'канал'] # Список столбцов с именами блогеров для приведения к единому формату 
# upd_columns = [column.lower() for column in upd_columns] #

> Так как до предобработки названий столбцов DataFrame название столбца с именем блогера может быть как "Блогер", так и "Канал", учтём это в цикле ниже

In [38]:
key_word = ['вид сотрудничества', 'cpv', 'cpu', 'err']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ' (руб.) до ндс'  # Список ключевых слов для их удаления из названия столбцов

for i in range(len(didenok_df)):
    df = didenok_df[i]  # Получаем текущий DataFrame по индексу
    if 'Блогер' in df.columns:
        upd_columns = ['блогер']  # Список столбцов с именами блогеров для приведения к единому формату
#         upd_columns = [column.lower() for column in upd_columns]
        pretraining_df(df, key_word, drop_word, upd_columns)
    elif 'Канал' in df.columns:
        upd_columns = ['канал']  # Список столбцов с именами блогеров для приведения к единому формату
#         upd_columns = [column.lower() for column in upd_columns]
        pretraining_df(df, key_word, drop_word, upd_columns)
    else:
        print('Не удалось найти столбец "Блогер" или "Канал" для преобразования DataFrame', {i})
    
    i += 1  # Увеличиваем i для перехода к следующему DataFrame

> Проверим результат

In [39]:
didenok_youtube.head()

7,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,стоимость за shorts до 1 мин,статистика,статистика ссылка
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,Статистика,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,Статистика,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,Статистика,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,Статистика,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,Статистика,https://drive.google.com/drive/folders/1VTEv65...


### Подготовка таблицы didenok_instagram

In [40]:
[col for col in didenok_instagram.columns]

['блогер',
 'ссылка',
 'категория, тематика блогера',
 'количество подписчиков',
 'средний охват фотопоста',
 'средний охват 1 сторис',
 'среднее кол-во просмотров reels',
 'среднее кол-во лайков',
 'среднее кол-во комментариев',
 'стоимость за фотопост',
 'стоимость за 1 сторис до 15 сек',
 'стоимость за серию из 3х сториc до 45 сек',
 'стоимость за reels',
 'статистика',
 'статистика']

In [41]:
bufer_columns = ['блогер',
                 'ссылка',
                 'тематика',
                 'количество подписчиков',
                 'среднее кол-во просмотров фотопоста',
                 'среднее кол-во просмотров 1 сторис',
                 'среднее кол-во просмотров reels / видеопоста',
                 'среднее кол-во лайков',
                 'среднее кол-во комментариев',
                 'стоимость фотопост',
                 'стоимость 1 сторис до 15 сек',
                 'стоимость серия из 3х сторис до 45 сек',
                 'стоимость reels / видеопост',
                 'статистика удалить',
                 'статистика']

In [42]:
didenok_instagram.columns = bufer_columns

In [43]:
didenok_instagram.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров reels / видеопоста,среднее кол-во лайков,среднее кол-во комментариев,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия из 3х сторис до 45 сек,стоимость reels / видеопост,статистика удалить,статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,Статистика,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,-,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,Статистика,https://drive.google.com/drive/u/1/folders/11S...
11,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,Статистика,https://drive.google.com/drive/u/1/folders/1M3...
12,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,Статистика,https://drive.google.com/drive/folders/1haDuDs...


> Удалим столбец со статистикой, в котором не высветились гиперссылки

In [44]:
didenok_instagram.drop(columns=['статистика удалить'], axis=1, inplace= True)

> Удалим первую строку, которая не показывает что-то по блогерам, а являестся побочной строкой названия столбцом.  
> [Она то появляется, то исчезает, поэтому оставим как удаление пропуска]

In [45]:
didenok_instagram.drop(didenok_instagram[didenok_instagram['блогер'] == ''].index, inplace=True)

In [46]:
didenok_instagram

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров reels / видеопоста,среднее кол-во лайков,среднее кол-во комментариев,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия из 3х сторис до 45 сек,стоимость reels / видеопост,статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
11,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...
12,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,https://drive.google.com/drive/folders/1haDuDs...
13,антон шастун,https://www.instagram.com/anton.shastoon/,лайфстайл/юмор,2 000 000,300 000,80 000,130 000,30 000,-,р.715 000,р.400 000,р.880 000,р.715 000,https://drive.google.com/drive/folders/1wmz871...
14,софа купер,https://www.instagram.com/sophakuper1/,лайфстайл/бьюти,1 700 000,130 000,50 000,130 000,9 000,50,р.185 000,р.75 000,р.170 000,р.240 000,https://drive.google.com/drive/folders/1UXVo4H...
15,мария омг,https://www.instagram.com/mariya.omg/,дети/подростки,1 600 000,60 000,25 000,50 000,10 000,85,р.80 000,р.35 000,р.75 000,р.115 000,https://drive.google.com/drive/folders/1CngKKI...
16,влад кобяков,https://www.instagram.com/kobyakov_official/,лайфстайл/гейминг,1 500 000,250 000,40 000,100 000,30 000,75,р.145 000,р.100 000,р.145 000,р.195 000,https://drive.google.com/drive/folders/1S9ESZI...
17,саша айс,https://www.instagram.com/isashaice/,лайфстайл/бьюти,1 300 000,130 000,50 000,130 000,9 000,50,р.185 000,р.75 000,р.170 000,р.240 000,https://drive.google.com/drive/folders/14T0VOj...


> Добавим указание на название соц.сети в название столбцов,чтобы в дальнейшем правильно объединять и искать нужные столбцы, в том числе в боте

##### Функция для указания названия соц.сети в названии столбцов

In [47]:
def type_social_network(df: pd.DataFrame, social_network: str) -> pd.DataFrame:
    
    
    """
    Функция принимает DataFrame - df, в название столбцов которого мы добавим название соц сети social_network
    
    ПАРАМЕТРЫ:
    df - DataFrame, в котором делаем преобразования.
    social_network - название соц. сети, которое мы добавим в название столбцов
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (DataFrame) с обновлёнными названиями столбцов
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    type_social_network(didenok_instagram, 'Instagram')
    """
    
    bufer_lst = []
    
    for element in df.columns.tolist():
        bufer_lst.append(f'{social_network} ' + element)
        
    df.columns = bufer_lst
    
    return df

In [48]:
type_social_network(didenok_instagram, 'Instagram')

,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
11,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...
12,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,https://drive.google.com/drive/folders/1haDuDs...
13,антон шастун,https://www.instagram.com/anton.shastoon/,лайфстайл/юмор,2 000 000,300 000,80 000,130 000,30 000,-,р.715 000,р.400 000,р.880 000,р.715 000,https://drive.google.com/drive/folders/1wmz871...
14,софа купер,https://www.instagram.com/sophakuper1/,лайфстайл/бьюти,1 700 000,130 000,50 000,130 000,9 000,50,р.185 000,р.75 000,р.170 000,р.240 000,https://drive.google.com/drive/folders/1UXVo4H...
15,мария омг,https://www.instagram.com/mariya.omg/,дети/подростки,1 600 000,60 000,25 000,50 000,10 000,85,р.80 000,р.35 000,р.75 000,р.115 000,https://drive.google.com/drive/folders/1CngKKI...
16,влад кобяков,https://www.instagram.com/kobyakov_official/,лайфстайл/гейминг,1 500 000,250 000,40 000,100 000,30 000,75,р.145 000,р.100 000,р.145 000,р.195 000,https://drive.google.com/drive/folders/1S9ESZI...
17,саша айс,https://www.instagram.com/isashaice/,лайфстайл/бьюти,1 300 000,130 000,50 000,130 000,9 000,50,р.185 000,р.75 000,р.170 000,р.240 000,https://drive.google.com/drive/folders/14T0VOj...


### Подготовка таблицы didenok_youtube

In [49]:
didenok_youtube.head()

7,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,стоимость за shorts до 1 мин,статистика,статистика ссылка
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,Статистика,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,Статистика,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,Статистика,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,Статистика,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,Статистика,https://drive.google.com/drive/folders/1VTEv65...


> Переименуем столбцы

In [50]:
[col for col in didenok_youtube.columns]

['канал',
 'ссылка',
 'категория, тематика канала',
 'количество подписчиков',
 'среднее кол-во просмотров',
 'среднее кол-во просмотров shorts',
 'среднее кол-во лайков',
 'среднее кол-во комментариев',
 'стоимость за интеграцию до 1 мин',
 'стоимость за продакт плейсмент',
 'стоимость за преролл до 30 сек',
 'стоимость за мидролл до 30 сек',
 'стоимость за shorts до 1 мин',
 'статистика',
 'статистика ссылка']

In [51]:
try:
    bufer_columns = ['блогер',
                     'ссылка',
                     'тематика',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'среднее кол-во просмотров shorts',
                     'среднее кол-во лайков',
                     'среднее кол-во комментариев',
                     'стоимость интеграция 1 слот',
                     'стоимость продакт плейсмент',
                     'стоимость преролл до 30 сек',
                     'стоимость мидролл до 30 сек',
                     'стоимость shorts до 1 мин',
                     'статистика удалить',
                     'м',
                     'ж',
                     '13-17',
                     '18-24',
                     '25-34',
                     '35-44',
                     '45-54',
                     '55+',
                     'ца гео рф',
                     'статистика ссылка']
    
    didenok_youtube.columns = bufer_columns
    
    didenok_youtube['ца по возрасту'] = didenok_youtube.apply(lambda row:
                                                              f"13-17 - {row['13-17']}, " +
                                                              f"18-24 - {row['18-24']}, " +
                                                              f"25-34 - {row['25-34']}, " +
                                                              f"35-44 - {row['35-44']}, " +
                                                              f"45-54 - {row['45-54']}" +
                                                              f"55+ - {row['55+']}", axis=1)
    
    didenok_youtube.drop(columns=['13-17',
                              '18-24',
                              '25-34',
                              '35-44',
                              '45-54',
                              '55+'], axis=1, inplace= True)
    didenok_youtube['ца по полу'] = didenok_youtube.apply(lambda row:
    f"Мужской - {row['м']}, " +
    f"Женской - {row['ж']}", axis=1)
    didenok_youtube.drop(columns=['м', 'ж'], axis=1, inplace= True)
    
    
except:
    bufer_columns = ['блогер',
                     'ссылка',
                     'тематика',
                     'количество подписчиков',
                     'среднее кол-во просмотров видео',
                     'среднее кол-во просмотров shorts',
                     'среднее кол-во лайков',
                     'среднее кол-во комментариев',
                     'стоимость интеграция 1 слот',
                     'стоимость продакт плейсмент',
                     'стоимость преролл до 30 сек',
                     'стоимость мидролл до 30 сек',
                     'стоимость shorts до 1 мин',
                     'статистика удалить',
                     'статистика ссылка']
    
    didenok_youtube.columns = bufer_columns

In [52]:
didenok_youtube.drop(columns=['статистика удалить'], axis=1, inplace= True)
didenok_youtube.rename(columns = {'статистика ссылка':'статистика'}, inplace = True)

In [53]:
didenok_youtube.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,https://drive.google.com/drive/folders/1VTEv65...


> У нас есть столбцы с возрастом и полу в процентном соотношении - объединим их в единый столбец.  
[Данные столбцы то появляются, то исчезают, поэтому на будущее оставим код]

> Удалим пустые строки

In [54]:
didenok_youtube.drop(didenok_youtube[didenok_youtube['блогер'] == ''].index, inplace=True)

In [55]:
didenok_youtube

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,https://drive.google.com/drive/folders/1VTEv65...
13,катя клэп,https://www.youtube.com/user/thekateclapp,лайфстайл/бьюти,7 130 000,700 000,-,60 000,1 000,р.1 330 000,-,-,-,-,https://drive.google.com/drive/folders/1LU4q-1...
14,катя адушкина (1 слот),https://www.youtube.com/@KatyaAdushkinaa,лайфстайл,5 390 000,500 000,4 000,20 000,500,р.640 000,р.400 000,р.440 000,р.370 000,р.250 000,https://drive.google.com/drive/folders/1aSmonK...
15,катя адушкина (2 слот),https://www.youtube.com/@KatyaAdushkinaa,лайфстайл,5 390 000,500 000,4 000,20 000,500,р.490 000,-,-,-,-,https://drive.google.com/drive/folders/1aSmonK...
16,supercrastan (1 слот),https://www.youtube.com/user/SuperCrastan,научпоп/DIY/шоу,3 430 000,750 000,40 000,7 000,500,р.980 000,-,р.735 000,р.615 000,р.735 000,https://drive.google.com/drive/folders/1dVEwSt...
17,supercrastan (2 слот),https://www.youtube.com/user/SuperCrastan,научпоп/DIY/шоу,3 430 000,750 000,40 000,7 000,500,р.860 000,-,-,-,-,https://drive.google.com/drive/folders/1dVEwSt...


> У нас есть проблема в том, что часть по одному и тому же блогеру выведены в отдельные строки как 1 и 2 слот

In [56]:
didenok_youtube.loc[(didenok_youtube['блогер'].str.contains('1 слот')) | (didenok_youtube['блогер'].str.contains('2 слот'))]

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
14,катя адушкина (1 слот),https://www.youtube.com/@KatyaAdushkinaa,лайфстайл,5 390 000,500 000,4 000,20 000,500,р.640 000,р.400 000,р.440 000,р.370 000,р.250 000,https://drive.google.com/drive/folders/1aSmonK...
15,катя адушкина (2 слот),https://www.youtube.com/@KatyaAdushkinaa,лайфстайл,5 390 000,500 000,4 000,20 000,500,р.490 000,-,-,-,-,https://drive.google.com/drive/folders/1aSmonK...
16,supercrastan (1 слот),https://www.youtube.com/user/SuperCrastan,научпоп/DIY/шоу,3 430 000,750 000,40 000,7 000,500,р.980 000,-,р.735 000,р.615 000,р.735 000,https://drive.google.com/drive/folders/1dVEwSt...
17,supercrastan (2 слот),https://www.youtube.com/user/SuperCrastan,научпоп/DIY/шоу,3 430 000,750 000,40 000,7 000,500,р.860 000,-,-,-,-,https://drive.google.com/drive/folders/1dVEwSt...
21,контакты (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,450 000,-,20 000,1 000,р.950 000,-,р.500 000,р.400 000,-,https://drive.google.com/drive/u/1/folders/1dz...
22,контакты (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,450 000,-,20 000,1 000,р.800 000,-,р.500 000,р.400 000,-,NaN
23,цитаты (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,100 000,-,7 000,500,р.650 000,-,р.400 000,р.300 000,-,NaN
24,цитаты (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,100 000,-,7 000,500,р.500 000,-,р.400 000,р.300 000,-,NaN
25,не игры (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,500 000,-,30 000,1 000,р.950 000,-,р.500 000,р.400 000,-,NaN
26,не игры (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,500 000,-,30 000,1 000,р.800 000,-,р.500 000,р.400 000,-,NaN


> Исправим данную проблему, переведя данные цены в отдельный столбец и объединив строки

In [57]:
def new_position_in_columns(df: pd.DataFrame, columns_with_name: str, columns_with_position: str) -> pd.DataFrame:
    
    """
    
    ПАРАМЕТРЫ:
    
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df - DataFrame, в котором мы делали преобразования
    columns_with_name (str) - 
    columns_with_position (str) -
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    didenok_youtube = new_position_in_columns(didenok_youtube, 'блогер', 'стоимость за интеграцию до 1 мин')
    
    """
    
    df[['блогер upd', 'слот']] = df[columns_with_name].str.extract(r'(.+)\s?\((\d+ слот)?\)')
    df['блогер upd'] = df['блогер upd'].fillna(df[columns_with_name])
    
    rows_to_delete = []

# Итерируемся по строкам DataFrame
    for index, row in df.iterrows():
        if '2 слот' in row['блогер']:
            blogger_name = row['блогер'].split(' (')[0]
        # Проверяем, есть ли строка с '1 слот' для этого блогера
            if any(df[(df['блогер'] != row['блогер']) & (df['блогер'].str.contains(blogger_name))]['блогер'].str.contains('1 слот')):
            # Находим индексы строк с совпадающим именем блогера
                matching_indices = df[df['блогер'].str.contains(blogger_name)].index
            # Обновляем данные в столбце 'стоимость за интеграцию 2 слот до 1 мин'
                df.loc[matching_indices, f'{columns_with_position} 2 слот'] = row[columns_with_position]
            # Добавляем индекс строки для удаления
                rows_to_delete.append(index)

# Удаляем строки с 2 слотом
    df.drop(rows_to_delete, inplace=True)
    
    df.drop(columns=['слот', 'блогер'], inplace=True)

# Переименовываем столбец 'блогер upd' в 'блогер'
    df.rename(columns={'блогер upd': 'блогер'}, inplace=True)

# Отчисти имена блогеров от лишних пробелов по бокам имени
    df['блогер'] = df['блогер'].str.strip()

    df.reset_index(drop=True, inplace=True)
    
    return df

In [58]:
didenok_youtube = new_position_in_columns(didenok_youtube, 'блогер', 'стоимость интеграция 1 слот')

In [59]:
# Проверим

didenok_youtube.loc[(didenok_youtube['блогер'].str.contains('1 слот')) | (didenok_youtube['блогер'].str.contains('2 слот'))]

,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика,блогер,стоимость интеграция 1 слот 2 слот


In [60]:
didenok_youtube.rename(columns = {'стоимость интеграция 1 слот 2 слот':'стоимость интеграция 2 слот'
                                  }, inplace = True)

### Нулевое объединение

> Сейчас мы объединим не два DataFrame из источников выше, а сделаем первый DataFrame, который станет основой для будущих объединений.  
> Перед объединением исправим неточность в наименовании блогера.

In [61]:
didenok_instagram.loc\
    [didenok_instagram['Instagram блогер'] == 'амина тендерлибае', 'Instagram блогер']  = 'амина tenderlybae'

didenok_instagram.loc\
    [didenok_instagram['Instagram блогер'] == 'helenavi', 'Instagram блогер']  = 'helenavi и gopronin'

> У нас есть два столбца, которые нам отдельно важны - это Никнейм блогера и его Тематика. Создадим единый DataFrame с такими столбцами, которые будем заполнять по мере объединения столбцов

In [62]:
bufer = pd.DataFrame(columns=['блогер','тематика'])
bufer

,блогер,тематика


> Объединим два DataFrame по вертикали с помощью метода concat (добавляет строки из didenok_instagram в bufer, пропуски в bufer в дальнейшем заполним)

In [63]:
bufer = pd.concat([bufer, didenok_instagram], ignore_index=True)

In [64]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
0,NaN,NaN,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
1,NaN,NaN,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
2,NaN,NaN,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
3,NaN,NaN,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...
4,NaN,NaN,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,https://drive.google.com/drive/folders/1haDuDs...
5,NaN,NaN,антон шастун,https://www.instagram.com/anton.shastoon/,лайфстайл/юмор,2 000 000,300 000,80 000,130 000,30 000,-,р.715 000,р.400 000,р.880 000,р.715 000,https://drive.google.com/drive/folders/1wmz871...
6,NaN,NaN,софа купер,https://www.instagram.com/sophakuper1/,лайфстайл/бьюти,1 700 000,130 000,50 000,130 000,9 000,50,р.185 000,р.75 000,р.170 000,р.240 000,https://drive.google.com/drive/folders/1UXVo4H...
7,NaN,NaN,мария омг,https://www.instagram.com/mariya.omg/,дети/подростки,1 600 000,60 000,25 000,50 000,10 000,85,р.80 000,р.35 000,р.75 000,р.115 000,https://drive.google.com/drive/folders/1CngKKI...
8,NaN,NaN,влад кобяков,https://www.instagram.com/kobyakov_official/,лайфстайл/гейминг,1 500 000,250 000,40 000,100 000,30 000,75,р.145 000,р.100 000,р.145 000,р.195 000,https://drive.google.com/drive/folders/1S9ESZI...
9,NaN,NaN,саша айс,https://www.instagram.com/isashaice/,лайфстайл/бьюти,1 300 000,130 000,50 000,130 000,9 000,50,р.185 000,р.75 000,р.170 000,р.240 000,https://drive.google.com/drive/folders/14T0VOj...


> Те пропуски в данных двух столбцах, что мы имеем сейчас, мы будем заполнять с помощью похожих столбцов из присоединённых датафреймов.  
  
> Сделаем для этого отдельную функцию

#### Функция, с помощью которой мы заполним Имя и Тематику блогера в едином DataFrame

In [65]:
def bufer_name_category(bufer: pd.DataFrame, social_network: str) -> pd.DataFrame:
    
    """
    Функция принимает DataFrame - bufer, столбцы 'блогер' и 'тематика' которого мы заполняем с помощью данных из новых колонок
    '{social_network} блогер' и '{social_network} тематика'.
    Учитываем, что столбец 'тематика' не всегда есть в присоединяемом столбце

    
    ПАРАМЕТРЫ:
    bufer (DataFrame) - единый DataFrame, к которому мы присоединяем другие DataFrame
    social_network (str) - название соц сети, по которому мы будем искать вновь присоединённые колонки
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    DataFrame - bufer, в котором мы заполнили столбцы 'блогер' и 'тематика'
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    bufer = bufer_name_category(bufer, 'Instagram')
    """
    
    bufer['блогер'] = bufer['блогер'].fillna(bufer[f'{social_network} блогер'])
    
    # Учтём, что столбец 'тематика' не всегда есть в присоединяемом столбце

    if f'{social_network} тематика' in bufer.columns:
        bufer['тематика'] = bufer['тематика'].fillna(bufer[f'{social_network} тематика'])
    
    return bufer

In [66]:
bufer = bufer_name_category(bufer, 'Instagram')

In [67]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,200,р.120 000,р.60 000,р.135 000,р.155 000,https://drive.google.com/drive/folders/1haDuDs...


### Первое объединение

#### Функция для объединения DataFrame

> Так как мы будем объединять DataFrame, в которых как могут быть общие столбцы, так и нет, то для корректной работы merge мы напишем специальную функцию, которая будет как указывать в названии новых DataFrame название соц. сети, объединять по общим столбцам, а если их нет, то по имени блогеров. Также отдельно мы будем выводить количество дубликатов в объединённом DataFrame

In [68]:
def merge_bufer_df(bufer: pd.DataFrame, df: pd.DataFrame, social_network: str) -> pd.DataFrame:
    
    """
    Указываем название соц сети в название столбцов с помощью ключа social_network, далее делаем объединение по двум сценариям:
    1) У нас есть общие столбцы с учётом добавления названия соц сети - тогда объединяем по ним с помощью merge
    2) У нас нет общих столбов - тогда объединяем по имени блогера из bufer['блогер'] и нового DataFrame
    
    Далее мы заполняем столбцы 'блогер' и 'тематика' в едином DataFrame с учётом, что 'тематика' не всегда есть в присоединяемом DataFram
    
    ПАРАМЕТРЫ:
    - bufer (DataFrame): наш единый DataFrame, к которому мы присоединяем новый DataFrame (df)
    - df (DataFrame): DataFrame, который мы будем присоединять
    - social_network (str): Ключ-название соц сети, по которому мы будем искать столбцы с именем и тематикой блогеров, кототорые присоединили

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    merged_bufer: единый DataFrame, в который мы объединили два DataFrame
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    bufer = merge_bufer_df(bufer, didenok_youtube, 'YouTube')

    """
    
    list_columns_no_social_network = []
    
    for element in df.columns.tolist():
        list_columns_no_social_network.append(f'{social_network} ' + element)
        
    df.columns = list_columns_no_social_network
    
    common_columns = bufer.columns.intersection(df.columns)
    
# В Pandas нельзя использовать if для проверки истинности объекта Index, поскольку он считается неоднозначным.
# Вместо этого мы будем использовать if not common_columns.empty: для проверки, является ли common_columns пустым или нет.
    
    if not common_columns.empty:
        merged_bufer = bufer.merge(df,
                                   left_on=common_columns.tolist(),
                                   right_on=common_columns.tolist(),
                                   how='outer')
    
    else:
        merged_bufer = bufer.merge(df, left_on='блогер', right_on=f'{social_network} блогер', how='outer')

    
    merged_bufer['блогер'] = merged_bufer['блогер'].fillna(merged_bufer[f'{social_network} блогер'])

# Учтём, что столбец 'тематика' не всегда есть в присоединяемом столбце
    if f'{social_network} тематика' in merged_bufer.columns:
        merged_bufer['тематика'] = merged_bufer['тематика'].fillna(merged_bufer[f'{social_network} тематика'])
        
        
    duplicates_merged_bufer = merged_bufer[merged_bufer.duplicated('блогер', keep=False)]

    # Чтобы посчитать количество дубликатов, можно использовать len() или метод .shape
    count_duplicates = len(duplicates_merged_bufer)  # или duplicates.shape[0]

    print("Количество дубликатов:", count_duplicates)
    
    return merged_bufer

In [69]:
bufer = merge_bufer_df(bufer, didenok_youtube, 'YouTube')

Количество дубликатов: 0


In [70]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube среднее кол-во лайков,YouTube среднее кол-во комментариев,YouTube стоимость интеграция 1 слот,YouTube стоимость продакт плейсмент,YouTube стоимость преролл до 30 сек,YouTube стоимость мидролл до 30 сек,YouTube стоимость shorts до 1 мин,YouTube статистика,YouTube блогер,YouTube стоимость интеграция 2 слот
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,60 000,1 000,р.1 330 000,-,-,-,-,https://drive.google.com/drive/folders/1LU4q-1...,катя клэп,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...,влад бумага а4,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,20 000,500,р.640 000,р.400 000,р.440 000,р.370 000,р.250 000,https://drive.google.com/drive/folders/1aSmonK...,катя адушкина,р.490 000
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,26 000,500,р.560 000,р.280 000,р.250 000,р.225 000,-,https://drive.google.com/drive/folders/13YIWAD...,амина tenderlybae,NaN
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,https://drive.google.com/drive/folders/1_CHbok...,vikishow,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,александр теслонд,лайфстайл/влоги,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10 000,500,р.160 000,р.115 000,р.115 000,р.90 000,р.80 000,https://drive.google.com/drive/folders/1b6RWd4...,александр теслонд,р.115 000
69,позавчерашние новости,новости/интервью,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10 000,500,р.1 000 000,р.500 000,р.650 000,р.500 000,-,https://drive.google.com/drive/folders/1Xoon_l...,позавчерашние новости,р.900 000
70,чай с жасмин,религия/психология,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3 000,500,р.160 000,р.130 000,р.130 000,р.115 000,р.100 000,https://drive.google.com/drive/folders/1z_vjeZ...,чай с жасмин,NaN
71,dimixer live,новости/события,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100,10,р.70 000,р.35 000,р.35 000,р.25 000,р.40 000,https://drive.google.com/drive/folders/1r3_knq...,dimixer live,NaN


> На данный момент мы не будем удалять дублирующие столбцы с именем и категорией блогеров из просоединённых столбцов, так как они нам понадобятся в дальнейшем для бота и для исследования

### Подготовка таблицы didenok_telegram

In [71]:
didenok_telegram.head()

7,блогер,ссылка,"категория, тематика блогера",количество подписчиков,среднее кол-во просмотров поста,среднее кол-во реакций и комментариев,стоимость за фотопост + текст,стоимость за видеопост + текст,стоимость за кружочек до 60 сек,стоимость за текстовое сообщение без фото,статистика,статистика ссылка
8,влад бумага а4,https://t.me/A4omg,лайфстайл/гейминг,753 000,500 000,100 000,р.1 000 000,р.1 500 000,р.875 000,р.625 000,Статистика,https://tgstat.ru/channel/@A4omg/stat
9,влад глент,https://t.me/vladglenttg,техно/лайфстайл,515 000,270 000,15 000,р.440 000,р.565 000,р.565 000,р.440 000,Статистика,https://tgstat.ru/channel/@vladglenttg/stat
10,антон шастун,https://t.me/shastoonchannel,лайфстайл/юмор,273 000,200 000,15 000,р.645 000,р.645 000,р.500 000,-,Статистика,https://drive.google.com/drive/folders/14xxWvL...
11,амина тендерлибае,https://t.me/plsbetenderly,твич/лайфстайл,256 000,80 000,3 000,р.240 000,р.390 000,р.310 000,р.170 000,Статистика,https://drive.google.com/drive/folders/1Gg5LC6...
12,катя клэп,https://t.me/KateClappTelegram,лайфстайл/бьюти,151 000,95 000,5 000,р.285 000,р.650 000,р.395 000,-,-,NaN


In [72]:
didenok_telegram.drop(columns=['статистика'], axis=1, inplace= True)

In [73]:
didenok_telegram.columns

Index(['блогер', 'ссылка', 'категория, тематика блогера',
       'количество подписчиков', 'среднее кол-во просмотров поста',
       'среднее кол-во реакций и комментариев',
       'стоимость за фотопост + текст', 'стоимость за видеопост + текст',
       'стоимость за кружочек до 60 сек',
       'стоимость за текстовое сообщение без фото', 'статистика ссылка'],
      dtype='object', name=7)

In [74]:
didenok_telegram.rename(columns = {'категория, тематика блогера':'тематика',
                                   'статистика ссылка':'статистика',
                                   'среднее кол-во реакций и комментариев':'средняя вовлечённость',
                                   'стоимость за фотопост + текст':'стоимость фотопост',
                                   'стоимость за видеопост + текст':'стоимость видеопост',
                                   'стоимость за кружочек до 60 сек':'стоимость кружочек до 60 сек',
                                   'стоимость за текстовое сообщение без фото':'стоимость текстовое сообщение без фото'
                                  }, inplace = True)

### Второе объединение

> Исправим неправильное наименование блогеров

In [75]:
didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'амина тендерлибае', 'блогер']  = 'амина tenderlybae'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'володя ххл', 'блогер']  = 'володя xxl'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'маша омг', 'блогер']  = 'мария омг'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'илья бёрн', 'блогер']  = 'илья берн'

> Объединяем

In [76]:
bufer = merge_bufer_df(bufer, didenok_telegram, 'Telegram')

Количество дубликатов: 0


In [77]:
bufer.tail()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Telegram ссылка,Telegram тематика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram средняя вовлечённость,Telegram стоимость фотопост,Telegram стоимость видеопост,Telegram стоимость кружочек до 60 сек,Telegram стоимость текстовое сообщение без фото,Telegram статистика
69,позавчерашние новости,новости/интервью,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,чай с жасмин,религия/психология,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,dimixer live,новости/события,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,несусвет,интервью/шоу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,маша нарциссизм,лайфстайл/бьюти,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://t.me/notmasha1,лайфстайл/бьюти,37 000,16 000,500,р.35 000,р.50 000,р.35 000,р.20 000,https://tgstat.ru/channel/@notmasha1/stat


### Подготовка таблицы didenok_vk_zen_show

> Мы видим, что помимо рекламы в VK, в таблице также есть строки с рекламой в Дзен Шоу.  
> Сделаем отдельный датафрейм для них, а из основного датафрейма удалим их.

In [78]:
didenok_vk_zen_show

6,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,статистика,статистика ссылка
7,громкий вопрос (1 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,Статистика,https://drive.google.com/drive/u/1/folders/1nx...
8,громкий вопрос (2 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 600 000,NaN,NaN,NaN,NaN,NaN
9,шоу истории (1 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,NaN,NaN
10,шоу истории (2 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 200 000,NaN,NaN,NaN,NaN,NaN
11,что делаем? куда идём? (1 слот),https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,NaN,NaN
12,что делаем? куда идём? (2 слот),https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.2 700 000,NaN,NaN,NaN,NaN,NaN
13,тейблтайм (1 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,NaN,NaN
14,тейблтайм (2 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 000 000,р.1 300 000,NaN,NaN,NaN,NaN,NaN
15,контакты (1 слот),https://vk.com/shastoon.channel,шоу/юмор/развлечения,358 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,Статистика,https://drive.google.com/drive/u/1/folders/1US...
16,контакты (2 слот),https://vk.com/shastoon.channel,шоу/юмор/развлечения,358 000,1 600 000,р.1 600 000,NaN,NaN,NaN,NaN,NaN


> Заполним столбец "Статистика"

In [79]:
didenok_vk_zen_show[['статистика ссылка']] = \
            didenok_vk_zen_show[['статистика ссылка']].fillna(method='ffill')

In [80]:
didenok_vk_zen_show.drop(columns=['статистика'], axis=1, inplace= True)
didenok_vk_zen_show.rename(columns = {'статистика ссылка':'статистика'}, inplace = True)

In [81]:
didenok_vk_zen_show.rename(columns = {'канал':'блогер',
                                      'категория, тематика блогера':'тематика',
                                      'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
                                      'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
                                      'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
                                      'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
                                     }, inplace = True)

> Переведём позиции со строк "2 слот" в отдельный столбец

In [82]:
didenok_vk_zen_show = new_position_in_columns(didenok_vk_zen_show, 'блогер', 'стоимость интеграция до 1 мин')
didenok_vk_zen_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
                                  }, inplace = True)

> Используем функцию .str.contains() для поиска подстроки в строках столбца DataFrame, а именно 'https://dzen.ru' в ссылке

In [83]:
# didenok_zen_show = didenok_vk_zen_show[didenok_vk_zen_show['ссылка'].str.contains('https://dzen.ru', na=False)]

In [84]:
# didenok_zen_show

> Используем ту же логику, что и выше, но теперь исключим строки, где в ссылке встречается 'https://dzen.ru', то есть оставим только 'https://vk.com/'

In [85]:
#Инверсия условия ~ здесь означает "не содержит", то есть выбираются строки,
#которые не содержат 'https://dzen.ru' в столбце 'ссылка'.
#Таким образом, didenok_vk_show будет содержать строки, которые не соответствуют заданному условию.

# didenok_vk_show = didenok_vk_zen_show[~didenok_vk_zen_show['ссылка'].str.contains('https://dzen.ru', na=False)]

> Переименуем столбцы, чтобы подвести их под единый принятый формат

In [86]:
# didenok_vk_show.columns

In [87]:
# didenok_vk_show.rename(columns = {'канал':'блогер',
#                                   'категория, тематика блогера':'тематика',
#                                   'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
#                                   'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
#                                   'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
#                                   'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
#                                  }, inplace = True)

# didenok_zen_show.rename(columns = {'канал':'блогер',
#                                   'категория, тематика блогера':'тематика',
#                                   'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
#                                   'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
#                                   'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
#                                   'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
#                                  }, inplace = True)

In [88]:
# didenok_zen_show = new_position_in_columns(didenok_zen_show, 'блогер', 'стоимость интеграция до 1 мин')

In [89]:
# didenok_zen_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
#                                   }, inplace = True)

In [90]:
# Проверим, что получилось
# didenok_zen_show.head()

In [91]:
# didenok_vk_show = new_position_in_columns(didenok_vk_show, 'блогер', 'стоимость интеграция до 1 мин')

In [92]:
# didenok_vk_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
#                                   }, inplace = True)


In [93]:
#Проверим, что получилось
didenok_vk_zen_show.head()

6,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость интеграция до 1 мин,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,статистика,блогер,стоимость интеграция 2 слот
0,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1nx...,громкий вопрос,р.1 600 000
1,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,https://drive.google.com/drive/u/1/folders/1nx...,шоу истории,р.1 200 000
2,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,NaN
3,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.2 700 000,NaN,NaN,NaN,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,NaN
4,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000


> У нас получилась интересная ситуация: у блогера "что делаем? куда идём?" не сработала функция.  
> Сделаем проверку:

In [94]:
string1 = didenok_vk_zen_show.loc[2, 'блогер']
string2 = didenok_vk_zen_show.loc[3, 'блогер']

print(repr(string1))
print(repr(string2))

'что делаем? куда идём?'
'что делаем? куда идём?'


In [95]:
# Затем вы можете сравнить их
if repr(string1) == repr(string2):
    print("Строки идентичны")
else:
    print("Строки различаются")

Строки идентичны


> Объединим данные строки:

In [96]:
indexes_to_remember = []

# Проверяем условие и сохраняем индексы в список
for index, row in didenok_vk_zen_show.iterrows():
    if row['блогер'] == 'что делаем? куда идём?':
        indexes_to_remember.append(index)

# Получаем последний и первый индексы
last_index = indexes_to_remember[-1]
first_index = indexes_to_remember[0]

# Заменяем значение в столбце 'стоимость за интеграцию до 1 мин 2 слот' в строке с первым индексом
didenok_vk_zen_show.at[first_index, 'стоимость интеграция 2 слот'] = \
            didenok_vk_zen_show.at[last_index, 'стоимость интеграция до 1 мин']

didenok_vk_zen_show = didenok_vk_zen_show.drop(last_index)

In [97]:
didenok_vk_zen_show

6,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость интеграция до 1 мин,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,статистика,блогер,стоимость интеграция 2 слот
0,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1nx...,громкий вопрос,р.1 600 000
1,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,https://drive.google.com/drive/u/1/folders/1nx...,шоу истории,р.1 200 000
2,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,р.2 700 000
4,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000
5,https://vk.com/shastoon.channel,шоу/юмор/развлечения,358 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1US...,контакты,р.1 600 000
6,https://vk.com/shastoon.channel,шоу/юмор/развлечения,358 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1US...,не игры,р.1 600 000
7,https://vk.com/shastoon.channel,подкаст/юмор,358 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1US...,цитаты,р.1 600 000
8,https://vk.com/improvdvij,шоу/юмор/развлечения,61 000,700 000,р.900 000,р.500 000,р.500 000,р.400 000,https://drive.google.com/drive/u/1/folders/1sk...,jam,р.700 000


### Третье объединение

In [98]:
# didenok_zen_show.rename(columns = {'категория, тематика канала':'тематика'}, inplace = True)

In [99]:
# bufer = merge_bufer_df(bufer, didenok_zen_show, 'Дзен Шоу')

In [100]:
# bufer.tail()

### Четвёртое объединение

In [101]:
didenok_vk_zen_show.rename(columns = {'категория, тематика канала':'тематика'}, inplace = True)

In [102]:
bufer = merge_bufer_df(bufer, didenok_vk_zen_show, 'VK Видео')

Количество дубликатов: 0


In [103]:
bufer.tail()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK Видео тематика,VK Видео количество подписчиков,VK Видео среднее кол-во просмотров ролика,VK Видео стоимость интеграция до 1 мин,VK Видео стоимость продакт плейсмент,VK Видео стоимость преролл до 30 сек,VK Видео стоимость мидролл до 30 сек,VK Видео статистика,VK Видео блогер,VK Видео стоимость интеграция 2 слот
72,несусвет,интервью/шоу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,маша нарциссизм,лайфстайл/бьюти,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,что делаем? куда идём?,шоу/юмор/тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,р.2 700 000
75,тейблтайм,шоу/юмор/развлечения,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000
76,jam,шоу/юмор/развлечения,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/развлечения,61 000,700 000,р.900 000,р.500 000,р.500 000,р.400 000,https://drive.google.com/drive/u/1/folders/1sk...,jam,р.700 000


### Подготовка таблицы didenok_vk_group

In [104]:
didenok_vk_group.head()

7,блогер,ссылка,"категория, тематика блогера",количество подписчиков,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,среднее кол-во лайков,среднее кол-во комментариев,стоимость за фотопост,стоимость за видеопост,стоимость за вк клип,статистика,статистика ссылка
8,влад бумага а4,https://vk.com/a4,лайфстайл/гейминг,1 540 000,200 000,250 000,6 000,180,р.750 000,-,р.1 625 000,обновляем,NaN
9,амина тендерлибае,https://vk.com/tenderlybae,твич/лайфстайл,739 000,250 000,80 000,10 000,50,р.210 000,р.300 000,р.300 000,Статистика,https://drive.google.com/drive/folders/12PNvkW...
10,катя адушкина,https://vk.com/katyaadushkina,музыка/лайфстайл/танцы,1 000 000,250 000,150 000,4 000,10,р.155 000,р.185 000,р.185 000,Статистика,https://drive.google.com/drive/folders/1MnkkeU...
11,даня крастер,https://vk.com/bnbproduction,DIY,275 000,30 000,30 000,200,10,р.200 000,р.370 000,р.370 000,Статистика,https://drive.google.com/drive/folders/1dVEwSt...
12,vikishow,https://vk.com/vikishow,лайфстайл/дети/подростки,478 000,70 000,50 000,2 000,20,р.70 000,р.115 000,р.115 000,Статистика,https://drive.google.com/drive/folders/14MQXd3...


In [105]:
didenok_vk_group.drop(columns=['статистика'], axis=1, inplace= True)

In [106]:
didenok_vk_group.columns

Index(['блогер', 'ссылка', 'категория, тематика блогера',
       'количество подписчиков', 'среднее кол-во просмотров поста',
       'среднее кол-во просмотров вк клипа', 'среднее кол-во лайков',
       'среднее кол-во комментариев', 'стоимость за фотопост',
       'стоимость за видеопост', 'стоимость за вк клип', 'статистика ссылка'],
      dtype='object', name=7)

In [107]:
didenok_vk_group.rename(columns = {'категория, тематика блогера':'тематика',
                                   'статистика ссылка':'статистика',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост',
                                   'стоимость за вк клип':'стоимость вк клип'}, inplace = True)

### Пятое объединение

> Исправим одну неточность в имени блогера

In [108]:
didenok_vk_group.loc\
    [didenok_vk_group['блогер'] == 'амина тендерлибае', 'блогер']  = 'амина tenderlybae'

In [109]:
bufer = merge_bufer_df(bufer, didenok_vk_group, 'VK группа')

Количество дубликатов: 0


In [110]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK группа тематика,VK группа количество подписчиков,VK группа среднее кол-во просмотров поста,VK группа среднее кол-во просмотров вк клипа,VK группа среднее кол-во лайков,VK группа среднее кол-во комментариев,VK группа стоимость фотопост,VK группа стоимость видеопост,VK группа стоимость вк клип,VK группа статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,лайфстайл/гейминг,1 540 000,200 000,250 000,6 000,180,р.750 000,-,р.1 625 000,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,музыка/лайфстайл/танцы,1 000 000,250 000,150 000,4 000,10,р.155 000,р.185 000,р.185 000,https://drive.google.com/drive/folders/1MnkkeU...
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,твич/лайфстайл,739 000,250 000,80 000,10 000,50,р.210 000,р.300 000,р.300 000,https://drive.google.com/drive/folders/12PNvkW...
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,лайфстайл/дети/подростки,478 000,70 000,50 000,2 000,20,р.70 000,р.115 000,р.115 000,https://drive.google.com/drive/folders/14MQXd3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,несусвет,интервью/шоу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,маша нарциссизм,лайфстайл/бьюти,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,что делаем? куда идём?,шоу/юмор/тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,тейблтайм,шоу/юмор/развлечения,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Подготовка таблицы didenok_vk_ls

In [111]:
# didenok_vk_ls
# didenok_vk_ls.drop(columns=['статистика'], axis=1, inplace= True)
# didenok_vk_ls.rename(columns = {'категория, тематика блогера':'тематика',
#                                'статистика ссылка':'статистика'}, inplace = True)

# ### Шестое объединение
# bufer = merge_bufer_df(bufer, didenok_vk_ls, 'VK личная страница')
# bufer.tail()

### Подготовка таблицы didenok_twitch

In [112]:
didenok_twitch

7,блогер,сcылка,категория,количество подписчиков,среднее кол-во зрителей,"уникальные зрители , (за 30 дней)",статистика,баннер + ссылка в описании 1 мес,оверлей использование логотипа товара/бренда в процессе трансляции,нативное голосовое упоминание рекламируемого товара в процессе трансляции,ссылка в чат бот 1 стрим,демонстрация-обзор товара на трансляции,статистика ссылка
8,амина tenderlybae,twitch.tv/tenderlybae,Just Chatting / IRL / Multigaming,1 300 000,6 500,546 000,Статистика,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA


In [113]:
didenok_twitch.drop(columns=['статистика'], axis=1, inplace= True)

In [114]:
didenok_twitch.rename(columns = {'сcылка':'ссылка',
                                 'уникальные зрители , (за 30 дней)':'уникальные зрители (30 дней)',
                                 'баннер + ссылка в описании 1 мес':'стоимость за баннер + ссылка в описании 1 мес',
                                 'оверлей использование логотипа товара/бренда в процессе трансляции':'стоимость оверлей логотипа товара/бренда в процессе трансляции',
                                 'нативное голосовое упоминание рекламируемого товара в процессе трансляции':'стоимость нативное голосовое упоминание товара в трансляции',
                                 'ссылка в чат бот 1 стрим ':'стоимость ссылка в чат-боте на 1 стрим',
                                 'демонстрация-обзор товара на трансляции':'стоимость демонстрация/обзор товара на трансляции',
                                 'статистика':'статистика',
                                 'категория':'тематика',
                                 'статистика ссылка':'статистика'}, inplace = True)

### Седьмое объединение

In [115]:
bufer = merge_bufer_df(bufer, didenok_twitch, 'Twitch')

Количество дубликатов: 0


In [116]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Twitch тематика,Twitch количество подписчиков,Twitch среднее кол-во зрителей,Twitch уникальные зрители (30 дней),Twitch стоимость за баннер + ссылка в описании 1 мес,Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции,Twitch стоимость нативное голосовое упоминание товара в трансляции,Twitch стоимость ссылка в чат-боте на 1 стрим,Twitch стоимость демонстрация/обзор товара на трансляции,Twitch статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,Just Chatting / IRL / Multigaming,1 300 000,6 500,546 000,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,несусвет,интервью/шоу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,маша нарциссизм,лайфстайл/бьюти,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,что делаем? куда идём?,шоу/юмор/тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,тейблтайм,шоу/юмор/развлечения,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> Мы объединили все таблицы DIDENOK, осталось добавить два столбца: налог и контакты менеджера

In [117]:
nalog = 'НДС (20%)'
manager = 'info@didenokteam.com'

In [118]:
bufer['налог'] = nalog
bufer['контакты менеджера'] = manager

In [119]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Twitch среднее кол-во зрителей,Twitch уникальные зрители (30 дней),Twitch стоимость за баннер + ссылка в описании 1 мес,Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции,Twitch стоимость нативное голосовое упоминание товара в трансляции,Twitch стоимость ссылка в чат-боте на 1 стрим,Twitch стоимость демонстрация/обзор товара на трансляции,Twitch статистика,налог,контакты менеджера
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,6 500,546 000,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA,НДС (20%),info@didenokteam.com
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com


## Подготовка данных по блогерам от VK

In [120]:
vk_bloggers.head(10)

,Сообщество,Шоу,KPI (стартов просмотра на 1 эпизод на 30-е сутки после релиза),Стоимость интеграции 1-й слот до НДС,Стоимость интеграции 2-й слот до НДС,Продакт плейсмент (30 сек) до НДС,Стоимость рекламной заставки (пре-ролл до 20 сек) до НДС,Стоимость рекламной заставки (мид-ролл до 20 сек) до НДС,Статистика,Контакты менеджера по продажам,Ссылка,Статистика ссылка
1,Азамат Мусагалиев,Большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,Статистика,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,Это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,NaN,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,Я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,NaN,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,Шоу памяти Азамата Мусагалиева NEW,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,NaN,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,Пикабу,Чебатков читает Пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...
6,Женский форум,Женский форум,1 000 000,р.1 800 000,р.1 500 000,р.600 000,р.1 000 000,нет,Статистика,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/zhenforum,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%96...
7,Варвара Щербакова по вашим интересам!,В гостях у Вари,1 000 000,р.1 500 000,р.1 200 000,р.500 000,р.850 000,нет,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/varjauletela_group,https://drive.google.com/drive/u/1/folders/1Q1...
8,Иван Абрамов,Подкаст Абрамова,2 000 000,по запросу,по запросу,нет,по запросу,по запросу,Статистика,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/ivan_abramov_standup,https://disk.yandex.ru/i/yt1TPXEBnJ9YCw
9,Иван Абрамов,Родная речь,1 500 000,р.1 000 000,р.850 000,по запросу,р.700 000,р.500 000,NaN,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/ivan_abramov_standup,NaN
10,Комьюнити,По хатам/по тачкам,1 500 000,р.650 000,нет,по запросу,р.550 000,р.350 000,NaN,NaN,https://vk.com/community_show,NaN


> Сделаем предобработку DataFrame

In [121]:
key_word = [] # Список ключевых слов для поиска столбцов и их удаления
drop_word = ' до НДС' # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Сообщество', 'Шоу'] # Список столбцов с именами блогеров для приведения к единому формату

In [122]:
vk_bloggers = pretraining_df(vk_bloggers, key_word, drop_word, upd_columns)

> Удалим сразу не нужный нам столбец "Статистика"

In [123]:
vk_bloggers.columns

Index(['сообщество', 'шоу',
       'kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)',
       'стоимость интеграции 1-й слот', 'стоимость интеграции 2-й слот',
       'продакт плейсмент (30 сек)',
       'стоимость рекламной заставки (пре-ролл до 20 сек)',
       'стоимость рекламной заставки (мид-ролл до 20 сек)', 'статистика',
       'контакты менеджера по продажам', 'ссылка', 'статистика ссылка'],
      dtype='object', name=0)

In [124]:
vk_bloggers.drop(columns=['статистика'], axis=1, inplace=True)

vk_bloggers.rename(columns = {'статистика ссылка':'статистика',
                              'стоимость интеграции 1-й слот':'стоимость интеграция 1 слот',
                              'стоимость интеграции 2-й слот':'стоимость интеграция 2 слот',
                              'продакт плейсмент (30 сек)':'стоимость продакт плейсмент',
                              'стоимость рекламной заставки (пре-ролл до 20 сек)':'стоимость преролл до 20 сек',
                              'стоимость рекламной заставки (мид-ролл до 20 сек)':'стоимость мидролл до 20 сек'}, inplace = True)

In [125]:
vk_bloggers.head()

,сообщество,шоу,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика
1,азамат мусагалиев,большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,шоу памяти азамата мусагалиева new,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,пикабу,чебатков читает пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


> Нам необходимо заполнить данные - так как при часть ячеек в исходной таблице объеденены, то правильное значение ставиться в верхнюю строку, а остальные становятся NaN, и мы заполним значения "сверху вниз".
  
> Мы заполним столбцы 'Сообщество', 'Ссылка на канал ', 'Ссылка' и 'Шоу'  
> Столбец 'Ссылка на канал' то появляется, то исчезает, так что учтём и его.

> К сожалению, мы не сможем сразу заполнить столбец 'Статистика ссылка' и 'ссылка', так как у некоторых каналов нет Статистики и ссылки, и если мы заполним сверху вниз, то у нас получится неверные ссылки - отдельно решим эту проблему

> В функции pretraining_df мы используем методы .strip(), .lower() и .replace() непосредственно на значениях NaN, которые мы заполнили через .fillna(''). Если не заполнить пропуски, то Python пытается их выполнить, что может привести к неожиданным результатам <em>[В данном проекте, например, символ "*" из значения NaN был заменен на "8" методом .replace("*")]</em>.
Поэтому надо превратить строки обратно в NaN, чтобы сделать заполнение.

In [126]:
vk_bloggers['ссылка'] = vk_bloggers['ссылка'].replace('', np.nan)
vk_bloggers['контакты менеджера по продажам'] = vk_bloggers['контакты менеджера по продажам'].replace('', np.nan)

In [127]:
try:
    vk_bloggers[['сообщество', 'ссылка', 'шоу', 'контакты менеджера по продажам']] = \
            vk_bloggers[['сообщество', 'ссылка', 'шоу', 'контакты менеджера по продажам']].fillna(method='ffill')
    
except:
    vk_bloggers[['сообщество', 'шоу', 'контакты менеджера по продажам']] = \
            vk_bloggers[['сообщество', 'шоу', 'контакты менеджера по продажам']].fillna(method='ffill')

> Чтобы заполнить столбец 'статистика', сделаем групировку по столбцу 'сообщество', так как у них одинаковые каналы, а значит и одинаковая статистика, выберем только столбец 'статистика' для работы и заполним с помощью .transform('first'), но учтём, что в столбце "ссылка" у нас есть исключение

In [128]:
vk_bloggers['статистика'] = vk_bloggers.groupby('сообщество')['статистика'].transform('first')

In [129]:
vk_bloggers['ссылка'] = vk_bloggers.groupby('сообщество')['ссылка']\
                        .transform(lambda group: group.iloc[0] if 'ок video' not in group.name else group)

In [130]:
# Проверяем
vk_bloggers.head()

,сообщество,шоу,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика
1,азамат мусагалиев,большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,азамат мусагалиев,это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
3,азамат мусагалиев,я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
4,азамат мусагалиев,шоу памяти азамата мусагалиева new,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
5,пикабу,чебатков читает пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


> В основной таблице bufer название блогера или его шоу указываются в едином столбце "блогер".  
> В таблице vk_bloggers имя блогера и его шоу разделены по разным столбцам.  
> Мы объединим их, чтобы в дальшейшем мы могли использовать поиск в боте

In [131]:
vk_bloggers['блогер'] = vk_bloggers['сообщество'] + ' - ' + vk_bloggers['шоу']
vk_bloggers.drop(columns=['сообщество', 'шоу'], axis= 1 , inplace= True )

In [132]:
#Проверяем
vk_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
1,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - большое шоу
2,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - это про меня
3,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - я себя знаю
4,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - шоу памяти азамата мусагал...
5,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...,пикабу - чебатков читает пикабу


> Мы видим, что помимо рекламы в VK, в таблице также есть строки с рекламой в Одноклассниках.  
> Сделаем отдельный датафрейм для них, а из основного датафрейма удалим их.

In [133]:
ok_bloggers = vk_bloggers[vk_bloggers['блогер'].str.contains('ок video')]

In [134]:
ok_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
83,5 000 000,р.1 150 000,по запросу,нет,р.800 000,нет,"Анна Ипатова, TG: @Anna_ipp, a.ipatova@vk.team",https://ok.ru/oknutye,None,ок video - в мире домашних животных
84,5 000 000,р.700 000,по запросу,по запросу,нет,нет,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://ok.ru/travelok,None,"ок video - ок, мама, поехали!"
85,2 500 000,р.800 000,р.700 000,по запросу,р.500 000,р.400 000,Алексей Леднев - alexey.lednev@vk.team,https://ok.ru/travelok,None,ок video - психология кино
86,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,Алексей Леднев - alexey.lednev@vk.team,https://ok.ru/oknutye,None,ок video - полтора спортсмена
87,5 000 000,р.1 100 000,по запросу,р.800 000,р.500 000,нет,Алексей Леднев - alexey.lednev@vk.team,https://ok.ru/gotovimok,None,ок video - шеф против любителя


In [135]:
# Проверим на дубликаты:

duplicates = ok_bloggers[ok_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер


> Сделаем DataFrame с блогерами из VK, удалив строки с блогерами из Одноклассников из исходного DataFrame

In [136]:
vk_bloggers = vk_bloggers[~vk_bloggers['блогер'].str.contains('ок video')]

In [137]:
vk_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
1,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - большое шоу
2,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - это про меня
3,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - я себя знаю
4,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - шоу памяти азамата мусагал...
5,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...,пикабу - чебатков читает пикабу


In [138]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
27,600 000,Интеграция с участием актеров: 350 000р,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей
28,NaN,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей
29,700 000,Интеграция с участием актеров: 600 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена
30,NaN,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена
32,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик


> Как мы видим, часть ячеек заполнено как тестовыми, так и числовыми значениями "Интеграция с участием блогера: 650 000р", то есть указана и позиция (базово она у нас указана в столбце), и цены (базово они у нас указаны в строках). Нам надо перевести данные значения в отдельные столбцы, объединив похожие позиции

In [139]:
def custom_split_text(df: pd.DataFrame, column1, column2) -> pd.DataFrame:
    
    """
    Функция форматирует информацию по строкам, создавая новые столбцы на основе содержимого 
    ячеек в указанных столбцах. Основные шаги включают в себя извлечение числовых значений и
    текстовых описаний из указанных столбцов, их дальнейшую обработку и создание новых столбцов
    с отформатированными данными, где мы выделяем позицию, указанную в ячейках, и цену на данную позицию.
    В результате, функция предназначена для структурирования данных и улучшения их читаемости.
    
    
    ПАРАМЕТРЫ:
    - df (pd.DataFrame): Исходный DataFrame, содержащий данные.
    - column1 (str): Название первого столбца, из которого нужно извлечь информацию.
    - column2 (str): Название второго столбца, из которого нужно извлечь информацию.


    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    - filtered_df (pandas DataFrame): Новый DataFrame, содержащий обработанные данные, включая извлеченные числа и описания.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    filtered_df (pandas DataFrame): Новый DataFrame, содержащий обработанные данные,
    включая извлеченные числа и описания.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    filtered_df = custom_split_text(vk_bloggers, 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот')

    """
    
    duplicates_filter = df[df.duplicated('блогер', keep=False)]

    filtered_df = duplicates_filter[['блогер', 
                                     column1,
                                     column2]]    


    for index in filtered_df.index:
        text = filtered_df.at[index, column1]

        # Поиск чисел в тексте
        numbers = re.findall(r'\d[\d\s]+', text)

        if numbers:
            # Извлечение первого найденного числа
            number = re.sub(r'[^0-9\s]', '', numbers[0])
            # Извлечение описания (текста без чисел)
            description = re.sub(r'[0-9]', '', text).strip()
            # Сохраняем числа и описание
            filtered_df.at[index, 'Позиция 1'] = description            
            filtered_df.at[index, 'Цена 1'] = number
        else:
            # Если нет чисел, сохраняем текст в столбце 'Позиция 1' и пустую строку в столбце 'цена 1'
            filtered_df.at[index, 'Позиция 1'] = text
            filtered_df.at[index, 'Цена 1'] = ''
            
    for index in filtered_df.index:
        text = filtered_df.at[index, column2]

        # Поиск чисел в тексте
        numbers = re.findall(r'\d[\d\s]+', text)

        if numbers:
            # Извлечение первого найденного числа
            number = re.sub(r'[^0-9\s]', '', numbers[0])
            # Извлечение описания (текста без чисел)
            description = re.sub(r'[0-9]', '', text).strip()
            # Сохраняем числа и описание
            filtered_df.at[index, 'Цена 2'] = number
            filtered_df.at[index, 'Позиция 2'] = description
        else:
            # Если нет чисел, сохраняем текст в столбце 'Позиция 1' и пустую строку в столбце 'цена 1'
            filtered_df.at[index, 'Позиция 2'] = text
            filtered_df.at[index, 'Цена 2'] = ''            
            
    for i in range(1, 3):
            # Удаление "р" и символов ".:()" из столбцов 'Позиция'
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].str.replace(r'\bр\.?\b', '', regex=True)
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].str.replace(r'[.:()]', '', regex=True)
            # Замена пустых строк на NaN
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].replace('', np.nan)
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].replace('по запросу', np.nan)    

        filtered_df[f'Цена {i}'] = filtered_df[f'Цена {i}'].replace('', np.nan)    
    
    return filtered_df

> Выведем новые значения в отдельный DataFrame

In [140]:
filtered_df = custom_split_text(vk_bloggers, 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот')

In [141]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350 000,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550 000,Интеграция с участием блогера,450 000
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600 000,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750 000,Интеграция с участием блогера,650 000
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1 150 000,Интеграция с участием актеров,1 050 000


> Сохраним индексы на будущее

In [142]:
indexes_to_filtered_df = list(filtered_df.index)

In [143]:
unique_position = filtered_df['блогер'].str.strip().unique()
len(unique_position)

10

In [144]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350 000,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550 000,Интеграция с участием блогера,450 000
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600 000,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750 000,Интеграция с участием блогера,650 000
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1 150 000,Интеграция с участием актеров,1 050 000


In [145]:
unique_position = filtered_df['Позиция 1'].str.strip().unique()
unique_position

array(['Интеграция с участием актеров', 'Интеграция с участием блогера',
       'Интеграция с участием комиков из выпуска',
       'Интеграция с участием Сергея Орлова', nan,
       'Интеграция с участием ведущих', 'Пушной, Крастер, Старый',
       'Мамикс', 'интеграция до  сек', 'устник до  сек'], dtype=object)

In [146]:
filtered_df.columns

Index(['блогер', 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот',
       'Позиция 1', 'Цена 1', 'Позиция 2', 'Цена 2'],
      dtype='object', name=0)

> Отчистим столбец с ценами от букв и сторонних символов, оставив только цифры

In [147]:
filtered_df['Позиция 1'] = filtered_df['Позиция 1'].fillna('')
filtered_df['Цена 1'].fillna(0, inplace=True)
# Уберем неразрывные пробелы из значений столбца 'Цена 1'
filtered_df['Цена 1'] = filtered_df['Цена 1'].str.replace('\xa0', '')
filtered_df['Цена 1'] = filtered_df['Цена 1'].str.replace(' ', '')
# Теперь преобразуем значения в числа с плавающей точкой
filtered_df['Цена 1'] = filtered_df['Цена 1'].astype(float)


filtered_df['Позиция 2'] = filtered_df['Позиция 2'].fillna('')
filtered_df['Цена 2'].fillna(0, inplace=True)
# Уберем неразрывные пробелы из значений столбца 'Цена 1'
filtered_df['Цена 2'] = filtered_df['Цена 2'].str.replace('\xa0', '')
filtered_df['Цена 2'] = filtered_df['Цена 2'].str.replace(' ', '')
# Теперь преобразуем значения в числа с плавающей точкой
filtered_df['Цена 2'] = filtered_df['Цена 2'].astype(float)


filtered_df['Позиция 1'] = filtered_df['Позиция 1'].str.strip()
filtered_df['Позиция 2'] = filtered_df['Позиция 2'].str.strip()

In [148]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350000.0,,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550000.0,Интеграция с участием блогера,450000.0
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600000.0,,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750000.0,Интеграция с участием блогера,650000.0
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1150000.0,Интеграция с участием актеров,1050000.0


In [149]:
def get_column_values_filtered_df(df: pd.DataFrame, key: int, column_with_position: str, column_with_price: str, column_list: str) -> pd.DataFrame:
    
    
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
    df - DataFrame, в котором делаем преобразования;
    key - Ключ в виде int, который нужно добавить к названию, чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
    column_with_position - Столбец, в котором указаны позиции;
    column_with_price - Столбец, в котором указаны цены на позиции;
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
    Создаём список с уникальными значениями позиций, приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
    spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                               .str.lower()\
                                               .str.replace('/\n', ', ')\
                                               .str.replace('\n', ', ')\
                                               .str.replace('*', '')\
                                               .unique()
    
    #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
    #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
    spisok = spisok[~pd.isna(spisok)]
        
        
    # Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
    
    # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
        df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
        where(df[column_with_position] == unique_position[i], other='')
    
    
    Удаляем исходные столбцы, чтобы не было повтора:
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)
    
    
    Возвращаем обновлённый DataFrame
    return df    
                
    """
#     column_list = []

    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]
    
    for i in range(len(unique_position)):
        
        new_column_name = f'{unique_position[i]}_{key} слот'
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')

            
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [150]:
column_list = []

for key in range(1, 3):
    column_with_position = f'Позиция {key}'
    price_column = f'Цена {key}'
    filtered_df_new = get_column_values_filtered_df(filtered_df, key, column_with_position, price_column, column_list)

In [151]:
filtered_df_new.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",...,устник до сек_1 слот,_2 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,-_2 слот,"Пушной, Крастер, Старый_2 слот",нет_2 слот
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
filtered_df_new.columns

Index(['блогер', 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот',
       'Интеграция с участием актеров_1 слот',
       'Интеграция с участием блогера_1 слот',
       'Интеграция с участием комиков из выпуска_1 слот',
       'Интеграция с участием Сергея Орлова_1 слот', '_1 слот',
       'Интеграция с участием ведущих_1 слот',
       'Пушной, Крастер, Старый_1 слот', 'Мамикс_1 слот',
       'интеграция до  сек_1 слот', 'устник до  сек_1 слот', '_2 слот',
       'Интеграция с участием блогера_2 слот',
       'Интеграция с участием актеров_2 слот',
       'Интеграция с участием комиков из выпуска_2 слот',
       'Интеграция с участием Сергея Орлова_2 слот',
       'Интеграция с участием ведущих_2 слот', '-_2 слот',
       'Пушной, Крастер, Старый_2 слот', 'нет_2 слот'],
      dtype='object', name=0)

> Удалим столбцы, не несущие дополнительной информации

In [153]:
filtered_df_new.drop(columns=['стоимость интеграция 1 слот',
                              'стоимость интеграция 2 слот',
                              '_1 слот',
                              '_2 слот',
                              '-_2 слот',
                              'нет_2 слот'], axis=1, inplace=True)

In [154]:
filtered_df_new.head()

,блогер,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",Мамикс_1 слот,интеграция до сек_1 слот,устник до сек_1 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,"Пушной, Крастер, Старый_2 слот"
27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,bubblegunshow - 9 панчей,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN
29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,bubblegunshow - арена,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN
32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN


In [155]:
filtered_df_new.reset_index(inplace=True)
filtered_df_new.head()

,index,блогер,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",Мамикс_1 слот,интеграция до сек_1 слот,устник до сек_1 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,"Пушной, Крастер, Старый_2 слот"
0,27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28,bubblegunshow - 9 панчей,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN
2,29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30,bubblegunshow - арена,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN
4,32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN


> Сделаем два DataFrame с чётными и нечётными индексами, так как в них дублируются блогеры

In [156]:
filtered_df_1 = filtered_df_new[filtered_df_new.index % 2 == 0]
filtered_df_2 = filtered_df_new[filtered_df_new.index % 2 != 0]

In [157]:
indexes_to_filtered_df_1 = filtered_df_1['index'].tolist()
indexes_to_filtered_df_1

[27, 29, 32, 36, 38, 42, 55, 58, 66, 70]

In [158]:
indexes_to_filtered_df_2 = filtered_df_2['index'].tolist()
indexes_to_filtered_df_2

[28, 30, 33, 37, 39, 43, 56, 59, 67, 71]

> С помощью merge объединим два получившихся DataFrame, чтобы не было дубликатов по блогерам

In [159]:
filtered_df_merge = pd.merge(filtered_df_1, filtered_df_2, on='блогер', how='outer')

In [160]:
filtered_df_merge.columns

Index(['index_x', 'блогер', 'Интеграция с участием актеров_1 слот_x',
       'Интеграция с участием блогера_1 слот_x',
       'Интеграция с участием комиков из выпуска_1 слот_x',
       'Интеграция с участием Сергея Орлова_1 слот_x',
       'Интеграция с участием ведущих_1 слот_x',
       'Пушной, Крастер, Старый_1 слот_x', 'Мамикс_1 слот_x',
       'интеграция до  сек_1 слот_x', 'устник до  сек_1 слот_x',
       'Интеграция с участием блогера_2 слот_x',
       'Интеграция с участием актеров_2 слот_x',
       'Интеграция с участием комиков из выпуска_2 слот_x',
       'Интеграция с участием Сергея Орлова_2 слот_x',
       'Интеграция с участием ведущих_2 слот_x',
       'Пушной, Крастер, Старый_2 слот_x', 'index_y',
       'Интеграция с участием актеров_1 слот_y',
       'Интеграция с участием блогера_1 слот_y',
       'Интеграция с участием комиков из выпуска_1 слот_y',
       'Интеграция с участием Сергея Орлова_1 слот_y',
       'Интеграция с участием ведущих_1 слот_y',
       'Пушн

> Удалим пустные столбцы

In [161]:
columns_with_all_nan = filtered_df_merge.columns[filtered_df_merge.isna().all()].tolist()
filtered_df_merge.drop(columns=columns_with_all_nan, axis=1, inplace=True)

In [162]:
filtered_df_merge.head()

,index_x,блогер,Интеграция с участием актеров_1 слот_x,Интеграция с участием комиков из выпуска_1 слот_x,"Пушной, Крастер, Старый_1 слот_x",интеграция до сек_1 слот_x,Интеграция с участием актеров_2 слот_x,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",index_y,Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
0,27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,28,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
1,29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,30,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
2,32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,33,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
3,36,bubblegunshow - удиви меня,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,37,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
4,38,bubblegunshow - сходка,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,39,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN


> Удалим столбцы со старыми индексами

In [163]:
filtered_df_merge.drop(columns=['index_x', 'index_y'], axis=1, inplace=True)

In [164]:
filtered_df_merge.head()

,блогер,Интеграция с участием актеров_1 слот_x,Интеграция с участием комиков из выпуска_1 слот_x,"Пушной, Крастер, Старый_1 слот_x",интеграция до сек_1 слот_x,Интеграция с участием актеров_2 слот_x,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
0,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
1,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
2,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
3,bubblegunshow - удиви меня,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
4,bubblegunshow - сходка,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN


> Объединим получившийся DataFrame с исходным по столбцу "блогер"

In [165]:
vk_bloggers = vk_bloggers.merge(filtered_df_merge, on='блогер', how='outer')

In [166]:
# vk_bloggers.drop(indexes_to_filtered_df_1, axis=0, inplace=True)

In [167]:
indexes_to_filtered_df_2

[28, 30, 33, 37, 39, 43, 56, 59, 67, 71]

In [168]:
indexes_to_filtered_df_1

[27, 29, 32, 36, 38, 42, 55, 58, 66, 70]

In [169]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
26,600 000,Интеграция с участием актеров: 350 000р,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
27,NaN,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
28,700 000,Интеграция с участием актеров: 600 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
29,NaN,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
31,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик,...,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
32,NaN,Интеграция с участием блогера: 1 550 000р,Интеграция с участием блогера: 1 350 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик,...,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
35,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - удиви меня,...,NaN,NaN,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
36,NaN,Интеграция с участием блогера: 1 450 000р,Интеграция с участием блогера: 1 250 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - удиви меня,...,NaN,NaN,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
37,700 000,Интеграция с участием актеров: 650 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - сходка,...,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN
38,NaN,Интеграция с участием блогера: 1 250 000р,Интеграция с участием блогера: 1 150 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - сходка,...,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN


> Отчистим столбцы 'стоимость интеграции 1-й слот', 'стоимость интеграции 2-й слот' от тех значений, которые мы перевели в отдельные столбцы

In [170]:
# Создаем паттерн для поиска строк, содержащих 'интеграция', 'устник' и '('
pattern = r'интеграция|устник|\(.*\)'  # Это регулярное выражение будет искать любое из указанных слов или '('

# Определите список слотов, которые вы хотите обработать
slots = ['стоимость интеграция 1 слот', 'стоимость интеграция 2 слот']

for slot_number, slot_column in enumerate(slots, start=1):
    
    for i in range (1,3):
        vk_bloggers[f'стоимость интеграция {i} слот'] = vk_bloggers[f'стоимость интеграция {i} слот'].str.lower()
    
    # Применяем фильтрацию с использованием регулярного выражения
    filtered_duplicates = vk_bloggers[slot_column].str.contains(pattern, case=False)

    # Извлекаем значения в виде списка
    duplicates_position = vk_bloggers[filtered_duplicates][slot_column].tolist()

    # Заменяем найденные значения на np.nan
    for i in range(len(duplicates_position)):
        vk_bloggers[slot_column] = vk_bloggers[slot_column].replace(duplicates_position[i], np.nan)

In [171]:
duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
26,600 000,NaN,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
28,700 000,NaN,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
31,1 600 000,NaN,NaN,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик,...,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
32,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик,...,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
35,1 600 000,NaN,NaN,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - удиви меня,...,NaN,NaN,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
36,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - удиви меня,...,NaN,NaN,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
37,700 000,NaN,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - сходка,...,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN
38,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - сходка,...,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN


> Удалим дубликаты из основного DataFrame vk_bloggers по индексам

In [172]:
indexes_to_duplicates = duplicates[duplicates.index % 2 == 0]
indexes_to_duplicates = indexes_to_duplicates.index.tolist()
indexes_to_duplicates

[26, 28, 32, 36, 38, 42, 54, 58, 66, 70]

In [173]:
vk_bloggers.drop(indexes_to_duplicates, inplace=True)

> Отлично, мы избавились от дубликатов и от неправильных значений в столбцe с ценами на интеграции!

In [174]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y


> В тоже время мы сохранили все значение

In [175]:
vk_bloggers.loc[vk_bloggers['блогер'] == 'vk video - кошмарный сон а4']

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
69,2 000 000,NaN,нет,нет,р.1 500 000,по запросу,Варвара Каинова - v.kainova@vk.team,https://vk.com/vkvideo,https://drive.google.com/drive/folders/1BpIYrN...,vk video - кошмарный сон а4,...,NaN,NaN,NaN,NaN,NaN,NaN,2500000.0,NaN,NaN,NaN


> Переименуем столбцы, чтобы подвести их к формату DataFrame bufer

In [176]:
[col for col in vk_bloggers]

['kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)',
 'стоимость интеграция 1 слот',
 'стоимость интеграция 2 слот',
 'стоимость продакт плейсмент',
 'стоимость преролл до 20 сек',
 'стоимость мидролл до 20 сек',
 'контакты менеджера по продажам',
 'ссылка',
 'статистика',
 'блогер',
 'Интеграция с участием актеров_1 слот_x',
 'Интеграция с участием комиков из выпуска_1 слот_x',
 'Пушной, Крастер, Старый_1 слот_x',
 'интеграция до  сек_1 слот_x',
 'Интеграция с участием актеров_2 слот_x',
 'Интеграция с участием комиков из выпуска_2 слот_x',
 'Пушной, Крастер, Старый_2 слот_x',
 'Интеграция с участием блогера_1 слот_y',
 'Интеграция с участием Сергея Орлова_1 слот_y',
 'Интеграция с участием ведущих_1 слот_y',
 'Мамикс_1 слот_y',
 'устник до  сек_1 слот_y',
 'Интеграция с участием блогера_2 слот_y',
 'Интеграция с участием Сергея Орлова_2 слот_y',
 'Интеграция с участием ведущих_2 слот_y']

In [177]:
vk_bloggers.rename(columns = {'kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)':'среднее кол-во просмотров',
                              'контакты менеджера по продажам':'контакты менеджера',
                              'Интеграция с участием актеров_1 слот_x':'стоимость интеграция с участием актеров 1 слот',
                              'Интеграция с участием комиков из выпуска_1 слот_x':'стоимость интеграция с участием комиков из выпуска 1 слот',
                              'Пушной, Крастер, Старый_1 слот_x':'стоимость интеграция с участием Пушного, Крастера, Старого 1 слот',
                              'интеграция до  сек_1 слот_x':'стоимость интеграция до 60 секунд 1 слот',
                              'Интеграция с участием актеров_2 слот_x':'стоимость интеграция с участием актеров 2 слот',
                              'Интеграция с участием комиков из выпуска_2 слот_x':'стоимость интеграция с участием комиков из выпуска 2 слот',
                              'Пушной, Крастер, Старый_2 слот_x':'стоимость интеграция с участием Пушного, Крастера, Старого 2 слот',
                              'Интеграция с участием блогера_1 слот_y':'стоимость интеграция с участием блогера 1 слот',
                              'Интеграция с участием Сергея Орлова_1 слот_y':'стоимость интеграция с участием Сергея Орлова 1 слот',
                              'Интеграция с участием ведущих_1 слот_y':'стоимость интеграция с участием ведущих 1 слот',
                              'Мамикс_1 слот_y':'стоимость интеграция с участием Мамикса 1 слот',
                              'устник до  сек_1 слот_y':'стоимость устник до 30 сек 1 слот',
                              'Интеграция с участием блогера_2 слот_y':'стоимость интеграция с участием блогера 2 слот',
                              'Интеграция с участием Сергея Орлова_2 слот_y':'стоимость интеграция с участием Сергея Орлова 2 слот',
                              'Интеграция с участием ведущих_2 слот_y':'стоимость интеграция с участием ведущих 2 слот'}, inplace = True)

In [178]:
bufer_columns = ['среднее кол-во просмотров',
                 'стоимость интеграция 1 слот',
                 'стоимость интеграция 2 слот',
                 'стоимость продакт плейсмент',
                 'стоимость преролл до 20 сек',
                 'стоимость мидролл до 20 сек',
                 'контакты менеджера',
                 'ссылка',
                 'статистика',
                 'блогер']

In [179]:
ok_bloggers.columns = bufer_columns

### Восьмое объединение

In [180]:
bufer = merge_bufer_df(bufer, vk_bloggers, 'VK Видео')

Количество дубликатов: 0


In [181]:
# Заполним столбец "контакты менеджера" данными из столбца "VK Видео контакты менеджера"
bufer['контакты менеджера'] = bufer['контакты менеджера'].fillna(bufer['VK Видео контакты менеджера'])

# Удалить столбец "VK Видео контакты менеджера"
bufer.drop(['VK Видео контакты менеджера'], axis=1, inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

### Девятое объединение

In [182]:
common_columns_vk_bloggers = bufer.columns.intersection(ok_bloggers.columns)
print(common_columns_vk_bloggers)

Index(['блогер', 'контакты менеджера'], dtype='object')


> У нас нет общих столбцов, так как к тем, что выше, мы добавим название соц сети ещё, но мы это предугадали в функции merge_bufer_df

In [183]:
bufer = merge_bufer_df(bufer, ok_bloggers, 'OK Шоу')

Количество дубликатов: 0


In [184]:
# Заполним столбец "контакты менеджера" данными из столбца "VK Видео контакты менеджера"
bufer['контакты менеджера'] = bufer['контакты менеджера'].fillna(bufer['OK Шоу контакты менеджера'])

# Удалить столбец "OK Шоу контакты менеджера"
bufer.drop(['OK Шоу контакты менеджера'], axis=1, inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

In [185]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK Видео стоимость интеграция с участием ведущих 2 слот,OK Шоу среднее кол-во просмотров,OK Шоу стоимость интеграция 1 слот,OK Шоу стоимость интеграция 2 слот,OK Шоу стоимость продакт плейсмент,OK Шоу стоимость преролл до 20 сек,OK Шоу стоимость мидролл до 20 сек,OK Шоу ссылка,OK Шоу статистика,OK Шоу блогер
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,ок video - полтора спортсмена,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,https://ok.ru/oknutye,None,ок video - полтора спортсмена
161,ок video - шеф против любителя,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,р.800 000,р.500 000,нет,https://ok.ru/gotovimok,None,ок video - шеф против любителя
162,ок video - кинокухня,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,https://ok.ru/gotovimok,None,ок video - кинокухня
163,"ок video - шоу, где звезды играют в игры со св...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,нет,нет,нет,р.1 500 000,р.1 200 000,https://ok.ru/gotovimok,None,"ок video - шоу, где звезды играют в игры со св..."


## Подготовка данных по блогерам от BlogoSphere

### Подготовка таблицы blogosphere_youtube

In [186]:
blogosphere_youtube

5,NaN,Название канала,Шоу,Подписчики,Среднее кол-во просмотров на видео,Статистика аудитории,Стандартные форматы,NaN,NaN,NaN,...,NaN,NaN,NaN,Пример рекламы 1,Пример рекламы 2,Пример рекламы 3,Среднее CPV,NaN,Ссылка,Статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,...,NaN,Формат 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,...,Цена* от,Описание,Цена* от,NaN,NaN,NaN,Стандартный Формат 1,Стандартный Формат 2,NaN,NaN
8,Э,NaN,"Осторожно, Собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,...,1 310 000 ₽,Product Placement,1 470 000 ₽,Eduson,Экоплант,Травелата,"1,31₽","0,96₽",NaN,NaN
9,NaN,Ксения Собчак,Осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,...,NaN,NaN,NaN,Astons,CGPods,SkillFactory,"1,38₽","0,93₽",https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,NaN,Красота требует КЭШ,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,...,NaN,NaN,NaN,Золотое яблоко,4FORMS,FOREO,"1,20₽","1,08₽",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,NaN,Nellifornication,Стандратный формат,NaN,NaN,NaN,1 слот,720 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,Тревел,NaN,NaN,NaN,1 слот,1 000 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,П,NaN,Shorts,NaN,1 000 000,NaN,Интеграция,720 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> Переименуем сразу имена столбцов, чтобы нам было проще ориентироваться в данных

In [187]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот']

In [188]:
bufer_columns = ['Эксклюзивность',
                 'Название канала',
                 'Шоу',
                 'количество подписчиков',
                 'среднее кол-во просмотров',
                 'Статистика аудитории',
                 'Ф1 позиция',
                 'Ф1 Цена от',
                 'Ф2 позиция',
                 'Ф2 Цена от',
                 'Ф3 позиция',
                 'Ф3 Цена от',
                 'Ф4 позиция',
                 'Ф4 Цена от',
                 'Ф5 позиция',
                 'Ф5 Цена от',                 
                 'Пример рекламы 1',
                 'Пример рекламы 2',
                 'Пример рекламы 3',
                 'Среднее CPV формат 1',
                 'Среднее CPV формат 2',
                 'ссылка',
                 'Статистика ссылка']

In [189]:
blogosphere_youtube.columns = bufer_columns

> Сделаем предобработку DataFrame

In [190]:
key_word = ['Эксклюзивность', # Список ключевых слов для поиска столбцов и их удаления
            'Пример рекламы 1',
            'Пример рекламы 2',
            'Пример рекламы 3',
            'Среднее CPV формат 1',
            'Среднее CPV формат 2'] 

drop_word = '' # Список ключевых слов для их удаления из названия столбцов

upd_columns = ['Название канала', 'Шоу'] # Список столбцов с именами блогеров для приведения к единому формату

In [191]:
blogosphere_youtube = pretraining_df(blogosphere_youtube, key_word, drop_word, upd_columns)

In [192]:
blogosphere_youtube

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,Формат 3,NaN,Формат 4,NaN,Формат 5,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,NaN,NaN
8,NaN,"осторожно, собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,3 слот,860 000 ₽,Преролл,1 310 000 ₽,Product Placement,1 470 000 ₽,NaN,NaN
9,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,Преролл,560 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,красота требует кэш,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,Преролл,660 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,nellifornication,стандратный формат,NaN,NaN,NaN,1 слот,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,тревел,NaN,NaN,NaN,1 слот,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,shorts,NaN,1 000 000,NaN,Интеграция,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
blogosphere_youtube[['название канала',
                     'шоу',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'статистика аудитории',
                     'ссылка',
                     'статистика ссылка']] = \
                    blogosphere_youtube[['название канала',
                                         'шоу',
                                         'количество подписчиков',
                                         'среднее кол-во просмотров',
                                         'статистика аудитории',
                                         'ссылка',
                                         'статистика ссылка']].fillna(method='ffill')

> Удалим первые две строки, так как в них указано неполное название столбцов и пока что в них актуальные пропуски, пока мы не запомнили "вверх"

In [194]:
blogosphere_youtube.dropna(subset=['шоу'], inplace=True)
blogosphere_youtube.reset_index(drop=True, inplace=True)

In [195]:
blogosphere_youtube[['название канала',
                     'шоу',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'статистика аудитории',
                     'ссылка',
                     'статистика ссылка']] = \
                    blogosphere_youtube[['название канала',
                                         'шоу',
                                         'количество подписчиков',
                                         'среднее кол-во просмотров',
                                         'статистика аудитории',
                                         'ссылка',
                                         'статистика ссылка']].fillna(method='backfill')

In [196]:
blogosphere_youtube.head()

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,1 слот,1 310 000 ₽,2 слот,960 000 ₽,3 слот,860 000 ₽,Преролл,1 310 000 ₽,Product Placement,1 470 000 ₽,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,Преролл,560 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,1 слот,960 000 ₽,2 Слот,860 000 ₽,Преролл,660 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
3,rogov live,обзоры,460 000,400 000,Подробнее,1 слот\nссылка после описания ролика,1 005 000 ₽,2 слот\nссылка после описания ролика,905 000 ₽,3 слот,905 000 ₽,Преролл,1 005 000 ₽,Мидролл,905 000 ₽,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...
4,без фильтров,интервью,20 000,100 000,Подробнее,1 слот для профильных клиентов,240 000 ₽,2 слот для профильных клиентов,210 000 ₽,NaN,NaN,Преролл,115 000 ₽,Мидролл,115 000 ₽,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...


> У нас есть ещё одна интересная особенность в данных: реклама в Shorts сделана как отдельная тематика видео, а не как стоимость. Сделаем из неё отдельный DataFrame, чтобы потом объединить его с основным DataFrame

In [197]:
blogosphere_youtube_shorts = blogosphere_youtube.loc[blogosphere_youtube['шоу'] == 'shorts']
blogosphere_youtube_shorts

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
18,александр пушной,shorts,1 670 000,200 000,Подробнее,Интеграция,300 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/c/PushnoyRU,https://drive.google.com/drive/folders/1-UqeBp...
41,дневник дизайнера,shorts,985 000,100 000,Подробнее,1 cлот,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCbhIVqiD-Know...,https://drive.google.com/drive/folders/1-enyaA...
69,василий уткин,shorts,615 000,7 000,Подробнее,Ролик до 1 мин,84 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCwrRor4IvrByH...,https://drive.google.com/drive/folders/11uM9Ql...
77,джарахов,shorts,3 640 000,1 000 000,Подробнее,1 слот,505 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@dzharakhov/videos,https://drive.google.com/drive/folders/1x-28-y...
79,comigration,shorts,83 500,100 000,Подробнее,1 cлот,цена обсуждаема,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@comigration,https://drive.google.com/drive/folders/13_EecF...
84,на шуме,shorts,51 000,4 000,Подробнее,1 слот,150 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...
87,nellifornication,shorts,51 000,1 000 000,Подробнее,Интеграция,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...
88,nellifornication,shorts,51 000,1 000 000,Подробнее,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...
89,nellifornication,shorts,51 000,1 000 000,Подробнее,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...


In [198]:
blogosphere_youtube_shorts.rename(columns = {'ф1 цена от':'стоимость shorts до 1 мин',
                                             'среднее кол-во просмотров':'среднее кол-во просмотров shorts'
                                            }, inplace = True)

In [199]:
blogosphere_youtube_shorts.columns

Index(['название канала', 'шоу', 'количество подписчиков',
       'среднее кол-во просмотров shorts', 'статистика аудитории',
       'ф1 позиция', 'стоимость shorts до 1 мин', 'ф2 позиция', 'ф2 цена от',
       'ф3 позиция', 'ф3 цена от', 'ф4 позиция', 'ф4 цена от', 'ф5 позиция',
       'ф5 цена от', 'ссылка', 'статистика ссылка'],
      dtype='object')

In [200]:
columns_with_shorts = ['название канала',
                       'среднее кол-во просмотров shorts',
                       'стоимость shorts до 1 мин']

# Удаление всех столбцов, кроме тех, что в списке columns_to_keep
blogosphere_youtube_shorts = blogosphere_youtube_shorts[columns_with_shorts]
blogosphere_youtube_shorts

,название канала,среднее кол-во просмотров shorts,стоимость shorts до 1 мин
18,александр пушной,200 000,300 000 ₽
41,дневник дизайнера,100 000,1 000 000 ₽
69,василий уткин,7 000,84 000 ₽
77,джарахов,1 000 000,505 000 ₽
79,comigration,100 000,цена обсуждаема
84,на шуме,4 000,150 000 ₽
87,nellifornication,1 000 000,720 000 ₽
88,nellifornication,1 000 000,NaN
89,nellifornication,1 000 000,NaN


> Удалим данные строки из основного DataFrame

In [201]:
blogosphere_youtube = blogosphere_youtube.drop(blogosphere_youtube[blogosphere_youtube['шоу'] == 'shorts'].index)

> **Перейдём к основному DataFrame**

> Сделаем новые столбцы для уникальных форматов, которые указаны в столбцах  'Ф1 позиция', 'Ф2 позиция', 'Ф3 позиция', 'Ф4 позиция',  'Ф5 позиция', указав также их цены из столбцов 'Ф1 Цена от', 'Ф2 Цена от', 'Ф3 Цена от', 'Ф4 Цена от', 'Ф5 Цена от'.   
> В некоторых новых столбцах получится по 1-5 цен, но так как это уникальный формат и он может встретиться в будущем, мы их оставим.  
> Некоторые позиции могут повторяться в разных столбцах, поэтому для разных столбцов с позициями мы будем указывать ключ с int, чтобы в дальнейшем объединить столбцы с одинакомыми позициями

> Так как у нас повторяются столцы и индексы, которые мы хотели бы использовать, то сделаем через цикл

In [202]:
def get_column_values(df: pd.DataFrame, key: int, column_with_position: str, column_with_price: str, column_list: str) -> pd.DataFrame:
    
    
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
    df - DataFrame, в котором делаем преобразования;
    key - Ключ в виде int, который нужно добавить к названию, чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
    column_with_position - Столбец, в котором указаны позиции;
    column_with_price - Столбец, в котором указаны цены на позиции;
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
    Создаём список с уникальными значениями позиций, приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
    spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                               .str.lower()\
                                               .str.replace('/\n', ', ')\
                                               .str.replace('\n', ', ')\
                                               .str.replace('*', '')\
                                               .unique()
    
    #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
    #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
    spisok = spisok[~pd.isna(spisok)]
        
        
    # Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
    
    # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
        df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
        where(df[column_with_position] == unique_position[i], other='')
    
    
    Удаляем исходные столбцы, чтобы не было повтора:
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)
    
    
    Возвращаем обновлённый DataFrame
    return df    
                
    """
#     column_list = []

    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]
    
    for i in range(len(unique_position)):
        new_column_name = f'{unique_position[i]}_{key}'
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')

            
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [203]:
column_list = []

for column_number in range(1, 6):
    position_column = f'ф{column_number} позиция'
    price_column = f'ф{column_number} цена от'
    blogosphere_youtube = get_column_values(blogosphere_youtube, column_number, position_column, price_column, column_list)

In [204]:
blogosphere_youtube

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ссылка,статистика ссылка,1 слот_1,1 слот\nссылка после описания ролика_1,1 слот для профильных клиентов_1,...,Выпуск под клиента_3,Преролл_4,Мидролл_4,Преролл \nдо 20 сек_4,Выпуск под клиента_4,Product Placement_5,Мидролл_5,Спецвыпуск_5,Спонсорство_5,Product placement_5
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,1 310 000 ₽,NaN,NaN,...,NaN,1 310 000 ₽,NaN,NaN,NaN,1 470 000 ₽,NaN,NaN,NaN,NaN
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,760 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,960 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rogov live,обзоры,460 000,400 000,Подробнее,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,1 005 000 ₽,NaN,...,NaN,1 005 000 ₽,NaN,NaN,NaN,NaN,905 000 ₽,NaN,NaN,NaN
4,без фильтров,интервью,20 000,100 000,Подробнее,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,NaN,240 000 ₽,...,NaN,115 000 ₽,NaN,NaN,NaN,NaN,115 000 ₽,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,кликклак,развлекательные шоу,6 930 000,2 000 000,Подробнее,https://www.youtube.com/@klikklak/featured,https://drive.google.com/drive/u/0/folders/198...,3 310 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,владимир яковлев,шоу яковлев по понедельникам,219 000,200 000,Подробнее,https://www.youtube.com/@YakovlevVladimir,https://drive.google.com/drive/folders/1jFhKc6...,250 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,на шуме,интервью,51 000,100 000,Подробнее,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,250 000 ₽,NaN,NaN,...,NaN,175 000 ₽,NaN,NaN,NaN,NaN,150 000 ₽,NaN,NaN,NaN
85,nellifornication,стандратный формат,51 000,4 000,Подробнее,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,720 000 ₽,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> Посмотрим на список новых столбцов, предварительно отсортировав его

In [205]:
sorted_column_names = sorted(column_list)
sorted_column_names

['1 слот\nссылка после описания ролика_1',
 '1 слот для непрофильных клиентов_1',
 '1 слот для профильных клиентов_1',
 '1 слот_1',
 '2  слот_2',
 '2 Слот_2',
 '2 слот\nссылка после описания ролика_2',
 '2 слот для непрофильных клиентов_2',
 '2 слот для профильных клиентов_2',
 '2 слот_2',
 '3 слот_3',
 'Product Placement_2',
 'Product Placement_5',
 'Product placement_5',
 'Выпуск под клиента_3',
 'Выпуск под клиента_4',
 'Интеграция в стрим_1',
 'Мидролл_3',
 'Мидролл_4',
 'Мидролл_5',
 'Мини-интеграция до 1 мин_1',
 'Нативная интеграция_1',
 'Нативная интеграция_2',
 'Преролл \nдо 20 сек_4',
 'Преролл_3',
 'Преролл_4',
 'Спецвыпуск_5',
 'Спонсорство_5']

> Объединим столбцы с общими позициями. Сделаем это через словарь

In [206]:
dict_new_columns = {'стоимость интеграция 1 слот':['1 слот\nссылка после описания ролика_1',
                                                    '1 слот_1'],
                    
                    'стоимость интеграция 2 слот':['2  слот_2',
                                                   '2 Слот_2',
                                                   '2 слот\nссылка после описания ролика_2',
                                                   '2 слот для профильных клиентов_2',
                                                   '2 слот_2'],
                                        
                    'стоимость продакт плейсмент':['Product Placement_2',
                                                   'Product Placement_5',
                                                   'Product placement_5'],
                    
                    'стоимость выпуск под клиента':['Выпуск под клиента_3',
                                                    'Выпуск под клиента_4','Спецвыпуск_5'],
                    
                    'стоимость мидролл до 30 сек':['Мидролл_3',
                                                   'Мидролл_4',
                                                   'Мидролл_5'],
                    
                    'стоимость нативная интеграция':['Нативная интеграция_1',
                                                     'Нативная интеграция_2',],
                    
                    'стоимость преролл до 30 сек':['Преролл \nдо 20 сек_4',
                                                   'Преролл_3',
                                                   'Преролл_4']
                   }

In [207]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        blogosphere_youtube[col] = blogosphere_youtube[col].fillna('').astype(str)
    
    blogosphere_youtube[new_column] = blogosphere_youtube[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    blogosphere_youtube[new_column] = blogosphere_youtube[new_column].replace('', np.nan)

In [208]:
blogosphere_youtube.drop(columns=['1 слот\nссылка после описания ролика_1',
                                  '1 слот_1',
                                  '2  слот_2',
                                  '2 Слот_2',
                                  '2 слот\nссылка после описания ролика_2',
                                  '2 слот для профильных клиентов_2',
                                  '2 слот_2',
                                  '3 слот_3',
                                  'Product Placement_2',
                                  'Product Placement_5',
                                  'Product placement_5',
                                  'Выпуск под клиента_3',
                                  'Выпуск под клиента_4',
                                  'Мидролл_3',
                                  'Мидролл_4',
                                  'Мидролл_5',
                                  'Нативная интеграция_1',
                                  'Нативная интеграция_2',
                                  'Преролл \nдо 20 сек_4',
                                  'Преролл_3',
                                  'Преролл_4',
                                  'Спецвыпуск_5'], axis=1, inplace=True)

In [209]:
blogosphere_youtube

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ссылка,статистика ссылка,1 слот для профильных клиентов_1,1 слот для непрофильных клиентов_1,Мини-интеграция до 1 мин_1,Интеграция в стрим_1,2 слот для непрофильных клиентов_2,Спонсорство_5,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,NaN,NaN,NaN,1 310 000 ₽,960 000 ₽,1 470 000 ₽,NaN,NaN,NaN,1 310 000 ₽
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,NaN,NaN,NaN,760 000 ₽,510 000 ₽,NaN,NaN,NaN,NaN,560 000 ₽
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,NaN,NaN,NaN,960 000 ₽,860 000 ₽,NaN,NaN,NaN,NaN,660 000 ₽
3,rogov live,обзоры,460 000,400 000,Подробнее,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,NaN,NaN,NaN,NaN,NaN,1 005 000 ₽,905 000 ₽,NaN,NaN,905 000 ₽,NaN,1 005 000 ₽
4,без фильтров,интервью,20 000,100 000,Подробнее,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,240 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,210 000 ₽,NaN,NaN,115 000 ₽,NaN,115 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,кликклак,развлекательные шоу,6 930 000,2 000 000,Подробнее,https://www.youtube.com/@klikklak/featured,https://drive.google.com/drive/u/0/folders/198...,NaN,NaN,NaN,NaN,NaN,NaN,3 310 000 ₽,2 210 000 ₽,NaN,NaN,NaN,NaN,1 440 000 ₽
82,владимир яковлев,шоу яковлев по понедельникам,219 000,200 000,Подробнее,https://www.youtube.com/@YakovlevVladimir,https://drive.google.com/drive/folders/1jFhKc6...,NaN,NaN,NaN,NaN,NaN,NaN,250 000 ₽,NaN,NaN,NaN,NaN,NaN,180 000 ₽
83,на шуме,интервью,51 000,100 000,Подробнее,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,250 000 ₽,150 000 ₽,NaN,NaN,150 000 ₽,NaN,175 000 ₽
85,nellifornication,стандратный формат,51 000,4 000,Подробнее,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN


> Подготовим названия столбцов к дальнейшей работе: объединим имя блогера и название шоу в один столбец "канал"

In [210]:
[col for col in blogosphere_youtube.columns]

['название канала',
 'шоу',
 'количество подписчиков',
 'среднее кол-во просмотров',
 'статистика аудитории',
 'ссылка',
 'статистика ссылка',
 '1 слот для профильных клиентов_1',
 '1 слот для непрофильных клиентов_1',
 'Мини-интеграция до 1 мин_1',
 'Интеграция в стрим_1',
 '2 слот для непрофильных клиентов_2',
 'Спонсорство_5',
 'стоимость интеграция 1 слот',
 'стоимость интеграция 2 слот',
 'стоимость продакт плейсмент',
 'стоимость выпуск под клиента',
 'стоимость мидролл до 30 сек',
 'стоимость нативная интеграция',
 'стоимость преролл до 30 сек']

In [211]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот']

In [212]:
blogosphere_youtube.rename(columns = {'1 слот для профильных клиентов_1':'стоимость интеграция для профильных клиентов 1 слот',
                                      '1 слот для непрофильных клиентов_1':'стоимость интеграция для непрофильных клиентов 1 слот',
                                      'Ролик до 1 мин_1':'стоимость отдельное видео до 1 мин',
                                      'Интеграция в стрим_1':'стоимость интеграция в стрим',
                                      '2 слот для профильных клиентов_2':'стоимость интеграция для профильных клиентов 2 слот',
                                      '2 слот для непрофильных клиентов_2':'стоимость интеграция для непрофильных клиентов 2 слот',
                                      '3 слот_3':'стоимость интеграция 3 слот',
                                      'Спонсорство_5':'стоимость спонсорство',
                                      'стоимость за интеграцию 1-ый слот':'стоимость интеграция 1 слот',
                                      '4 слот_1':'стоимость интеграция 4 слот',
                                      '5 слот_1':'стоимость интеграция 5 слот',
                                      'Мини-интеграция до 1 мин_1':'стоимость мини-интеграция до 1 мин',
                                     }, inplace = True)


> Присоединим столбец со стоимостью Shorts к единому DataFrame с помощью merge

In [213]:
blogosphere_youtube = blogosphere_youtube.merge(blogosphere_youtube_shorts, on='название канала', how='left')
blogosphere_youtube

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ссылка,статистика ссылка,стоимость интеграция для профильных клиентов 1 слот,стоимость интеграция для непрофильных клиентов 1 слот,стоимость мини-интеграция до 1 мин,...,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,среднее кол-во просмотров shorts,стоимость shorts до 1 мин
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,...,NaN,1 310 000 ₽,960 000 ₽,1 470 000 ₽,NaN,NaN,NaN,1 310 000 ₽,NaN,NaN
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,...,NaN,760 000 ₽,510 000 ₽,NaN,NaN,NaN,NaN,560 000 ₽,NaN,NaN
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,...,NaN,960 000 ₽,860 000 ₽,NaN,NaN,NaN,NaN,660 000 ₽,NaN,NaN
3,rogov live,обзоры,460 000,400 000,Подробнее,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,NaN,NaN,...,NaN,1 005 000 ₽,905 000 ₽,NaN,NaN,905 000 ₽,NaN,1 005 000 ₽,NaN,NaN
4,без фильтров,интервью,20 000,100 000,Подробнее,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,240 000 ₽,NaN,NaN,...,NaN,NaN,210 000 ₽,NaN,NaN,115 000 ₽,NaN,115 000 ₽,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,nellifornication,стандратный формат,51 000,4 000,Подробнее,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,...,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN
81,nellifornication,стандратный формат,51 000,4 000,Подробнее,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,...,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN
82,nellifornication,тревел,51 000,4 000,Подробнее,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,...,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,720 000 ₽
83,nellifornication,тревел,51 000,4 000,Подробнее,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,...,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN


> Сделаем единый столбец с именем блогера, объединив столбцы 'название канала' и 'шоу'

In [214]:
blogosphere_youtube['блогер'] = \
                blogosphere_youtube['название канала'] + ' - ' + blogosphere_youtube['шоу']

blogosphere_youtube.drop(columns=['статистика аудитории',
                                  'название канала',
                                  'шоу'], axis=1, inplace= True)

> Сделаем столбец "статистика" по примеру других DataFrame

In [215]:
blogosphere_youtube.rename(columns = {'статистика ссылка':'статистика'}, inplace = True)

> У нас есть дубликат - обработаем его

In [216]:
duplicates_blogosphere_youtube = blogosphere_youtube[blogosphere_youtube.duplicated('блогер', keep=False)]
duplicates_blogosphere_youtube

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,стоимость интеграция для профильных клиентов 1 слот,стоимость интеграция для непрофильных клиентов 1 слот,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость интеграция для непрофильных клиентов 2 слот,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,среднее кол-во просмотров shorts,стоимость shorts до 1 мин,блогер
4,20 000,100 000,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,240 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,210 000 ₽,NaN,NaN,115 000 ₽,NaN,115 000 ₽,NaN,NaN,без фильтров - интервью
5,20 000,100 000,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,160 000 ₽,NaN,NaN,140 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,без фильтров - интервью
79,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,720 000 ₽,nellifornication - стандратный формат
80,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - стандратный формат
81,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - стандратный формат
82,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,720 000 ₽,nellifornication - тревел
83,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - тревел
84,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - тревел


In [217]:
filtered_blogosphere_youtube_1 = duplicates_blogosphere_youtube[duplicates_blogosphere_youtube.index % 2 != 0]
filtered_blogosphere_youtube_2 = duplicates_blogosphere_youtube[duplicates_blogosphere_youtube.index % 2 == 0]

In [218]:
filtered_blogosphere_youtube_2 = filtered_blogosphere_youtube_2.dropna(axis=1)

In [219]:
# Создаем паттерн для поиска строк, содержащих слова стоимость и блогер
pattern = r'стоимость|блогер'

# Только их и оставить в DataFrame filtered_blogosphere_youtube_2
filtered_blogosphere_youtube_2 = filtered_blogosphere_youtube_2.filter(regex=pattern, axis=1)

# Объединим новый DataFrame с основным
blogosphere_youtube = blogosphere_youtube.merge(filtered_blogosphere_youtube_2, on='блогер', how='outer')

# Удалим дубликат из обновного DataFrame
indexes_to_duplicates_blogosphere_youtube = filtered_blogosphere_youtube_2.index.tolist()
blogosphere_youtube.drop(indexes_to_duplicates_blogosphere_youtube, inplace=True)

> Проверяем результат

In [220]:
duplicates_blogosphere_youtube = blogosphere_youtube[blogosphere_youtube.duplicated('блогер', keep=False)]
duplicates_blogosphere_youtube

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,стоимость интеграция для профильных клиентов 1 слот,стоимость интеграция для непрофильных клиентов 1 слот,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость интеграция для непрофильных клиентов 2 слот,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,среднее кол-во просмотров shorts,стоимость shorts до 1 мин,блогер
79,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,720 000 ₽,nellifornication - стандратный формат
81,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - стандратный формат
83,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,720 000 ₽,nellifornication - тревел
85,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - тревел
86,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - тревел
87,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - тревел


In [221]:
duplicates_blogosphere_youtube.columns

Index(['количество подписчиков', 'среднее кол-во просмотров', 'ссылка',
       'статистика', 'стоимость интеграция для профильных клиентов 1 слот',
       'стоимость интеграция для непрофильных клиентов 1 слот',
       'стоимость мини-интеграция до 1 мин', 'стоимость интеграция в стрим',
       'стоимость интеграция для непрофильных клиентов 2 слот',
       'стоимость спонсорство', 'стоимость интеграция 1 слот',
       'стоимость интеграция 2 слот', 'стоимость продакт плейсмент',
       'стоимость выпуск под клиента', 'стоимость мидролл до 30 сек',
       'стоимость нативная интеграция', 'стоимость преролл до 30 сек',
       'среднее кол-во просмотров shorts', 'стоимость shorts до 1 мин',
       'блогер'],
      dtype='object')

In [222]:
def rename_endswith_xy(df):
    
    """
    """
    
    columns_x = df.columns[df.columns.str.endswith("_x")]

    # Получите список столбцов, которые заканчиваются на "_y"
    columns_y = df.columns[df.columns.str.endswith("_y")]

    # Объедините оба списка в один
    base_name = np.concatenate([columns_x, columns_y])

    base_names_with_x_y = []

    # Проходим по всем столбцам DataFrame
    for column in base_name:
        # Разбиваем название столбца по "_x" и "_y"
        parts = column.split('_')

        # Если у нас есть и "_x" и "_y" и их части до "_x" и "_y" совпадают
        base_names_with_x_y.append(parts[0])  

    # base_name = pd.Series(base_name)
    unique_base_names = pd.Series(base_names_with_x_y).unique()

    # unique_base_names[1]

    for i in range(len(unique_base_names)):
        df[f'{unique_base_names[i]}_x'] = df[f'{unique_base_names[i]}_x'].fillna('').astype(str)
        df[f'{unique_base_names[i]}_y'] = df[f'{unique_base_names[i]}_y'].fillna('').astype(str)

        df[f'{unique_base_names[i]}'] = \
            df[[f'{unique_base_names[i]}_x', f'{unique_base_names[i]}_y']].agg(''.join, axis=1)

        df.drop(columns=[f'{unique_base_names[i]}_x', f'{unique_base_names[i]}_y'], axis=1, inplace=True)

        df[f'{unique_base_names[i]}'] = df[f'{unique_base_names[i]}'].replace('', np.nan)
        
    return df

In [223]:
# Проверяем
blogosphere_youtube.loc[blogosphere_youtube['блогер'] == 'без фильтров - интервью']

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,стоимость интеграция для профильных клиентов 1 слот,стоимость интеграция для непрофильных клиентов 1 слот,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость интеграция для непрофильных клиентов 2 слот,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,среднее кол-во просмотров shorts,стоимость shorts до 1 мин,блогер
5,20 000,100 000,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,160 000 ₽,NaN,NaN,140 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,без фильтров - интервью


In [224]:
blogosphere_youtube = rename_endswith_xy(blogosphere_youtube)

In [225]:
# Проверяем
blogosphere_youtube.loc[blogosphere_youtube['блогер'] == 'без фильтров - интервью']

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,стоимость интеграция для профильных клиентов 1 слот,стоимость интеграция для непрофильных клиентов 1 слот,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость интеграция для непрофильных клиентов 2 слот,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,среднее кол-во просмотров shorts,стоимость shorts до 1 мин,блогер
5,20 000,100 000,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,160 000 ₽,NaN,NaN,140 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,без фильтров - интервью


In [226]:
# try:
#     # Заполняем пропуски в столбцах списка list_merge пустыми строками
#     list_merge = ['стоимость за интеграцию для непрофильных клиентов 1 слот_x',
#                   'стоимость за интеграцию для непрофильных клиентов 2 слот_x',
#                   'стоимость за интеграцию для непрофильных клиентов 1 слот_y',
#                   'стоимость за интеграцию для непрофильных клиентов 2 слот_y']

#     blogosphere_youtube[list_merge] = blogosphere_youtube[list_merge].fillna('')

#     # Создаем новый столбец на основе объединенных значений
#     blogosphere_youtube['стоимость за интеграцию для непрофильных клиентов 1 слот'] = \
#         blogosphere_youtube[['стоимость за интеграцию для непрофильных клиентов 1 слот_x',
#                              'стоимость за интеграцию для непрофильных клиентов 1 слот_y']].agg(''.join, axis=1)

#     blogosphere_youtube['стоимость за интеграцию для непрофильных клиентов 2 слот'] = \
#         blogosphere_youtube[['стоимость за интеграцию для непрофильных клиентов 2 слот_x',
#                              'стоимость за интеграцию для непрофильных клиентов 2 слот_y']].agg(''.join, axis=1)

#     # Удалим те столбцы, которые объединяли
#     blogosphere_youtube.drop(columns=['стоимость за интеграцию для непрофильных клиентов 1 слот_x',
#                                       'стоимость за интеграцию для непрофильных клиентов 1 слот_y',
#                                       'стоимость за интеграцию для непрофильных клиентов 2 слот_x',
#                                       'стоимость за интеграцию для непрофильных клиентов 2 слот_y'], axis=1, inplace=True)

#     # Переведём пустые строки в NaN
#     blogosphere_youtube[['стоимость за интеграцию для непрофильных клиентов 1 слот',
#                         'стоимость за интеграцию для непрофильных клиентов 2 слот']] = \
#                                         blogosphere_youtube[['стоимость за интеграцию для непрофильных клиентов 1 слот',
#                                                             'стоимость за интеграцию для непрофильных клиентов 2 слот']].replace('', np.nan)

# except:
#             # Заполняем пропуски в столбцах списка list_merge пустыми строками
#     list_merge = ['стоимость за интеграцию для профильных клиентов 1 слот_x',
#                   'стоимость за интеграцию для профильных клиентов 2 слот_x',
#                   'стоимость за интеграцию для профильных клиентов 1 слот_y',
#                   'стоимость за интеграцию для профильных клиентов 2 слот_y']

#     blogosphere_youtube[list_merge] = blogosphere_youtube[list_merge].fillna('')

#     # Создаем новый столбец на основе объединенных значений
#     blogosphere_youtube['стоимость за интеграцию для профильных клиентов 1 слот'] = \
#         blogosphere_youtube[['стоимость за интеграцию для профильных клиентов 1 слот_x',
#                              'стоимость за интеграцию для профильных клиентов 1 слот_y']].agg(''.join, axis=1)

#     blogosphere_youtube['стоимость за интеграцию для профильных клиентов 2 слот'] = \
#         blogosphere_youtube[['стоимость за интеграцию для профильных клиентов 2 слот_x',
#                              'стоимость за интеграцию для профильных клиентов 2 слот_y']].agg(''.join, axis=1)

#     # Удалим те столбцы, которые объединяли
#     blogosphere_youtube.drop(columns=['стоимость за интеграцию для профильных клиентов 1 слот_x',
#                                       'стоимость за интеграцию для профильных клиентов 1 слот_y',
#                                       'стоимость за интеграцию для профильных клиентов 2 слот_x',
#                                       'стоимость за интеграцию для профильных клиентов 2 слот_y'], axis=1, inplace=True)

#     # Переведём пустые строки в NaN
#     blogosphere_youtube[['стоимость за интеграцию для профильных клиентов 1 слот',
#                         'стоимость за интеграцию для профильных клиентов 2 слот']] = \
#                                         blogosphere_youtube[['стоимость за интеграцию для профильных клиентов 1 слот',
#                                                             'стоимость за интеграцию для профильных клиентов 2 слот']].replace('', np.nan)


In [227]:
# # Заполняем пропуски в столбцах списка list_merge пустыми строками
# list_merge = ['стоимость за преролл до 30 сек_x',
#               'стоимость за преролл до 30 сек_y']

# blogosphere_youtube[list_merge] = blogosphere_youtube[list_merge].fillna('')

# # Создаем новый столбец на основе объединенных значений
# blogosphere_youtube['стоимость за преролл до 30 сек'] = \
#     blogosphere_youtube[['стоимость за преролл до 30 сек_x',
#                          'стоимость за преролл до 30 сек_y']].agg(''.join, axis=1)

# # Удалим те столбцы, которые объединяли
# blogosphere_youtube.drop(columns=['стоимость за преролл до 30 сек_x',
#                                   'стоимость за преролл до 30 сек_y'], axis=1, inplace=True)

# # Переведём пустые строки в NaN
# blogosphere_youtube[['стоимость за преролл до 30 сек']] = \
#                                     blogosphere_youtube[['стоимость за преролл до 30 сек']].replace('', np.nan)


In [228]:
blogosphere_youtube

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,стоимость интеграция для профильных клиентов 1 слот,стоимость интеграция для непрофильных клиентов 1 слот,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость интеграция для непрофильных клиентов 2 слот,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,среднее кол-во просмотров shorts,стоимость shorts до 1 мин,блогер
0,3 430 000,1 000 000,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,NaN,NaN,NaN,1 310 000 ₽,960 000 ₽,1 470 000 ₽,NaN,NaN,NaN,1 310 000 ₽,NaN,NaN,"ксения собчак - осторожно, собчак"
1,3 430 000,550 000,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,NaN,NaN,NaN,760 000 ₽,510 000 ₽,NaN,NaN,NaN,NaN,560 000 ₽,NaN,NaN,ксения собчак - осторожно: новости!
2,3 430 000,800 000,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,NaN,NaN,NaN,960 000 ₽,860 000 ₽,NaN,NaN,NaN,NaN,660 000 ₽,NaN,NaN,ксения собчак - красота требует кэш
3,460 000,400 000,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,NaN,NaN,NaN,NaN,NaN,1 005 000 ₽,905 000 ₽,NaN,NaN,905 000 ₽,NaN,1 005 000 ₽,NaN,NaN,rogov live - обзоры
5,20 000,100 000,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,160 000 ₽,NaN,NaN,140 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,без фильтров - интервью
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - стандратный формат
83,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,720 000 ₽,nellifornication - тревел
85,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - тревел
86,51 000,4 000,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,NaN,nellifornication - тревел


### Десятое объединение

In [229]:
bufer = merge_bufer_df(bufer, blogosphere_youtube, 'YouTube')

Количество дубликатов: 6


In [230]:
def show_duplicates(df: pd.DataFrame) -> str:
    
    """
    Удаляет дубликаты из DataFrame и выводит информацию о количестве дубликатов в столбце 'блогер'.
    
    ПАРАМЕТРЫ:
    df (pd.DataFrame): Исходный DataFrame.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    str: Строка с информацией о дубликатах, отсортированных по столбцу 'блогер'.

    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    duplicates_info = show_duplicates(my_dataframe)
    print(duplicates_info)
    """
    
    df = df.drop_duplicates()

    duplicates_df = df[df.duplicated('блогер', keep=False)]

    print("Количество дубликатов:", len(duplicates_df))

    return duplicates_df.sort_values(by='блогер')

In [231]:
show_duplicates(bufer)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,OK Шоу блогер,YouTube среднее кол-во просмотров,YouTube стоимость интеграция для профильных клиентов 1 слот,YouTube стоимость интеграция для непрофильных клиентов 1 слот,YouTube стоимость мини-интеграция до 1 мин,YouTube стоимость интеграция в стрим,YouTube стоимость интеграция для непрофильных клиентов 2 слот,YouTube стоимость спонсорство,YouTube стоимость выпуск под клиента,YouTube стоимость нативная интеграция
243,nellifornication - стандратный формат,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4 000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,nellifornication - стандратный формат,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4 000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,nellifornication - тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4 000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,nellifornication - тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4 000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [232]:
indices_to_remove = [244, 246]  # Индексы строк для удаления

bufer = bufer.drop(indices_to_remove)

### Подготовка таблицы blogosphere_tg_vk_zen

In [233]:
blogosphere_tg_vk_zen

4,NaN,Имя блогера,Telegram,NaN,NaN,NaN,NaN,TikTok,NaN,NaN,...,NaN,NaN,ссылка,Статистика ссылка,ссылка,Статистика ссылка,ссылка,Статистика ссылка,ссылка,Статистика ссылка
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,URL,Статистика,Подписчики,Средний охват поста,Цена* от...,URL,Статистика,Подписчики,...,Средний охват поста,Цена* от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Э,Редакция. Наука,Редакция. Наука,Подробнее,30 000,10 000,Серия 3 поста\n35 000 ₽,NaN,NaN,NaN,...,NaN,NaN,https://t.me/redakciya_nauka,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN
8,Э,Тамара Эйдельман,Уроки истории с Тамарой Эйдельман,Подробнее,58 000,30 000,135 000 ₽,NaN,NaN,NaN,...,NaN,NaN,https://t.me/eidelman,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN
9,Э,Михаил Шапошников,Mikhail Shaposhnikov LIVE,Подробнее,25 000,20 000,220 000 ₽,misha_des,NaN,268 600,...,800,50 000 ₽,https://t.me/misha_des,https://drive.google.com/drive/folders/14K8YiX...,https://www.tiktok.com/@misha_des,NaN,NaN,https://drive.google.com/drive/folders/14K8YiX...,https://zen.yandex.ru/mishades?lang=ru,https://zen.yandex.ru/mishades?lang=ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,freekicksrus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
[col for col in blogosphere_tg_vk_zen.columns]

[nan,
 'Имя блогера',
 'Telegram',
 nan,
 nan,
 nan,
 nan,
 'TikTok',
 nan,
 nan,
 nan,
 nan,
 'VK',
 nan,
 nan,
 nan,
 nan,
 'Яндекс.Дзен',
 nan,
 nan,
 nan,
 nan,
 'ссылка',
 'Статистика ссылка',
 'ссылка',
 'Статистика ссылка',
 'ссылка',
 'Статистика ссылка',
 'ссылка',
 'Статистика ссылка']

In [235]:
bufer_columns_blogosphere_tg_vk_zen = ['Эксклюзивность',
                                       'Имя блогера',
                                       'Telegram блогер',
                                       'Telegram статистика',
                                       'Telegram количество подписчиков',
                                       'Telegram среднее кол-во просмотров поста',
                                       'Telegram стоимость фотопост',
                                       'TikTok блогер',
                                       'TikTok статистика',
                                       'TikTok количество подписчиков',
                                       'TikTok среднее кол-во просмотров',
                                       'TikTok стоимость видеопост',
                                       'VK группа блогер',
                                       'VK группа блогер статистика',
                                       'VK группа количество подписчиков',
                                       'VK группа среднее кол-во просмотров поста',
                                       'VK группа стоимость фотопост',
                                       'Дзен блогер',
                                       'Дзен статистика',
                                       'Дзен количество подписчиков',
                                       'Дзен среднее кол-во просмотров',
                                       'Дзен стоимость пост',
                                       'Telegram ссылка',
                                       'Telegram cтатистика',
                                       'TikTok ссылка',
                                       'TikTok статистика',
                                       'VK группа ссылка',
                                       'VK группа статистика',
                                       'Дзен ссылка',
                                       'Дзен статистика']

In [236]:
blogosphere_tg_vk_zen.columns = bufer_columns_blogosphere_tg_vk_zen

In [237]:
blogosphere_tg_vk_zen

,Эксклюзивность,Имя блогера,Telegram блогер,Telegram статистика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram стоимость фотопост,TikTok блогер,TikTok статистика,TikTok количество подписчиков,...,Дзен среднее кол-во просмотров,Дзен стоимость пост,Telegram ссылка,Telegram cтатистика,TikTok ссылка,TikTok статистика,VK группа ссылка,VK группа статистика,Дзен ссылка,Дзен статистика
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,URL,Статистика,Подписчики,Средний охват поста,Цена* от...,URL,Статистика,Подписчики,...,Средний охват поста,Цена* от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Э,Редакция. Наука,Редакция. Наука,Подробнее,30 000,10 000,Серия 3 поста\n35 000 ₽,NaN,NaN,NaN,...,NaN,NaN,https://t.me/redakciya_nauka,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN
8,Э,Тамара Эйдельман,Уроки истории с Тамарой Эйдельман,Подробнее,58 000,30 000,135 000 ₽,NaN,NaN,NaN,...,NaN,NaN,https://t.me/eidelman,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN
9,Э,Михаил Шапошников,Mikhail Shaposhnikov LIVE,Подробнее,25 000,20 000,220 000 ₽,misha_des,NaN,268 600,...,800,50 000 ₽,https://t.me/misha_des,https://drive.google.com/drive/folders/14K8YiX...,https://www.tiktok.com/@misha_des,NaN,NaN,https://drive.google.com/drive/folders/14K8YiX...,https://zen.yandex.ru/mishades?lang=ru,https://zen.yandex.ru/mishades?lang=ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,freekicksrus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
key_word = ['Эксклюзивность']# Список ключевых слов для поиска столбцов и их удаления
             
drop_word = '' # Список ключевых слов для их удаления из названия столбцов

upd_columns = ['Имя блогера',
               'Telegram блогер',
               'TikTok блогер',
               'VK группа блогер',
               'Дзен блогер'] # Список столбцов с именами блогеров для приведения к единому формату

In [239]:
blogosphere_tg_vk_zen = pretraining_df(blogosphere_tg_vk_zen, key_word, drop_word, upd_columns)

> Назовём сразу столбцы как в едином DataFrame bufer, чтобы потом не делить данный DataFrame на части и не объединять их через функцию merge_bufer_df

In [240]:
bufer_columns_blogosphere_tg_vk_zen = ['блогер',
                                       'Telegram блогер',
                                       'Telegram статистика',
                                       'Telegram количество подписчиков',
                                       'Telegram среднее кол-во просмотров поста',
                                       'Telegram стоимость фотопост',
                                       'TikTok блогер',
                                       'TikTok статистика',
                                       'TikTok количество подписчиков',
                                       'TikTok среднее кол-во просмотров',
                                       'TikTok стоимость видеопост',
                                       'VK группа блогер',
                                       'VK группа статистика',
                                       'VK группа количество подписчиков',
                                       'VK группа среднее кол-во просмотров поста',
                                       'VK группа стоимость фотопост',
                                       'Дзен блогер',
                                       'Дзен статистика',
                                       'Дзен количество подписчиков',
                                       'Дзен среднее кол-во просмотров',
                                       'Дзен стоимость пост',
                                       'Telegram ссылка',
                                       'Telegram cтатистика',
                                       'TikTok ссылка',
                                       'TikTok статистика',
                                       'VK группа ссылка',
                                       'VK группа статистика',
                                       'Дзен ссылка',
                                       'Дзен статистика']

In [241]:
blogosphere_tg_vk_zen.columns = bufer_columns_blogosphere_tg_vk_zen

In [242]:
blogosphere_tg_vk_zen

,блогер,Telegram блогер,Telegram статистика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram стоимость фотопост,TikTok блогер,TikTok статистика,TikTok количество подписчиков,TikTok среднее кол-во просмотров,...,Дзен среднее кол-во просмотров,Дзен стоимость пост,Telegram ссылка,Telegram cтатистика,TikTok ссылка,TikTok статистика,VK группа ссылка,VK группа статистика,Дзен ссылка,Дзен статистика
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,url,Статистика,Подписчики,Средний охват поста,Цена* от...,url,Статистика,Подписчики,Средний охват поста,...,Средний охват поста,Цена* от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,редакция. наука,редакция. наука,Подробнее,30 000,10 000,Серия 3 поста\n35 000 ₽,NaN,NaN,NaN,NaN,...,NaN,NaN,https://t.me/redakciya_nauka,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN
8,тамара эйдельман,уроки истории с тамарой эйдельман,Подробнее,58 000,30 000,135 000 ₽,NaN,NaN,NaN,NaN,...,NaN,NaN,https://t.me/eidelman,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN
9,михаил шапошников,mikhail shaposhnikov live,Подробнее,25 000,20 000,220 000 ₽,misha_des,NaN,268 600,NaN,...,800,50 000 ₽,https://t.me/misha_des,https://drive.google.com/drive/folders/14K8YiX...,https://www.tiktok.com/@misha_des,NaN,NaN,https://drive.google.com/drive/folders/14K8YiX...,https://zen.yandex.ru/mishades?lang=ru,https://zen.yandex.ru/mishades?lang=ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,freekicksrus,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> Удалим лишние пустые строки, в которых у нас нет информации из-за особенности DataFrame, в котором под таблицей указали запасные данные: удалить строки, в которых значения равны NaN как в столбце 'блогер', так и в столбце 'telegram блогер' как в самых заполненных столбцах

In [243]:
blogosphere_tg_vk_zen = blogosphere_tg_vk_zen.dropna(subset=['блогер', 'Telegram блогер'], how='all')

# Отдельно удалим первую строку с NaN в столбец 'блогер', которая представляем собой название столбцов в виде строки
blogosphere_tg_vk_zen = blogosphere_tg_vk_zen.dropna(subset=['блогер'])

> Заполним значение столбца "имя блогера" с помощью метода .fillna(method='ffill')

In [244]:
blogosphere_tg_vk_zen[['блогер']] = blogosphere_tg_vk_zen[['блогер']].fillna(method='ffill')

> Удалим лишние столбцы, где нет ссылки на статистику

In [245]:
blogosphere_tg_vk_zen.drop(columns=['Telegram статистика',
                                    'TikTok статистика',
                                    'VK группа статистика',
                                    'Дзен статистика',
                                    'Telegram cтатистика'], axis=1, inplace=True)

In [246]:
blogosphere_tg_vk_zen

,блогер,Telegram блогер,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram стоимость фотопост,TikTok блогер,TikTok количество подписчиков,TikTok среднее кол-во просмотров,TikTok стоимость видеопост,VK группа блогер,...,VK группа среднее кол-во просмотров поста,VK группа стоимость фотопост,Дзен блогер,Дзен количество подписчиков,Дзен среднее кол-во просмотров,Дзен стоимость пост,Telegram ссылка,TikTok ссылка,VK группа ссылка,Дзен ссылка
7,редакция. наука,редакция. наука,30 000,10 000,Серия 3 поста\n35 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/redakciya_nauka,NaN,NaN,NaN
8,тамара эйдельман,уроки истории с тамарой эйдельман,58 000,30 000,135 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/eidelman,NaN,NaN,NaN
9,михаил шапошников,mikhail shaposhnikov live,25 000,20 000,220 000 ₽,misha_des,268 600,NaN,по запросу,NaN,...,NaN,NaN,дневник дизайнера,3 100,800,50 000 ₽,https://t.me/misha_des,https://www.tiktok.com/@misha_des,NaN,https://zen.yandex.ru/mishades?lang=ru
10,аня консервы,фасоликус томатикус,38 000,30 000,90 000 ₽,a_conservi,99 200,40 000,80 000 ₽,conservi,...,25 000,65 000 ₽,NaN,NaN,NaN,NaN,https://t.me/fasolicustomaticus,https://www.tiktok.com/@a_conservi,https://vk.com/conservishit,NaN
11,солодников,солодников,32 000,30 000,80 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/s/solodnikov,NaN,NaN,NaN
12,юлия меньшова,юлия меньшова,267 000,160 000,305 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/JuliaMenshovaJulia,NaN,NaN,NaN
13,the люди. лядов,the люди. лядов,145 000,40 000,90 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/Theludi1,NaN,NaN,NaN
14,полинэ кудрявцева,полинэ и ее мандеж,37 000,25 000,90 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/polinekudriavtseva,NaN,NaN,NaN
15,ксения собчак,кровавая барыня,1 245 000,750 000,710 000 ₽,xenia_sobchak,680 500,NaN,по запросу,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/bloodysx,https://www.tiktok.com/@xenia_sobchak,NaN,NaN
19,надежда стрелец,стрелец-молодец,325 000,130 000,545 000 ₽,NaN,NaN,NaN,NaN,стрелец-молодец,...,100 000,235 000 ₽,NaN,NaN,NaN,NaN,https://t.me/strelets_molodec,NaN,https://vk.com/strelets_molodec,NaN


### Одинадцатое объединение

> У нас есть общие столбцы - на основе их и присоединим blogosphere_tg_vk_zen к bufer

In [247]:
common_columns_blogosphere_tg_vk_zen = bufer.columns.intersection(blogosphere_tg_vk_zen.columns)
print(common_columns_blogosphere_tg_vk_zen)

Index(['блогер', 'Telegram блогер', 'Telegram ссылка',
       'Telegram количество подписчиков',
       'Telegram среднее кол-во просмотров поста',
       'Telegram стоимость фотопост', 'VK группа блогер', 'VK группа ссылка',
       'VK группа количество подписчиков',
       'VK группа среднее кол-во просмотров поста',
       'VK группа стоимость фотопост'],
      dtype='object')


> Так как мы сразу указали название соц. сети в названии столбцов, чтобы не делить на несколько DataFrame, то объединим не с помощью функции, а вручную с помощью merge

In [248]:
bufer = bufer.merge(blogosphere_tg_vk_zen, \
                    left_on=common_columns_blogosphere_tg_vk_zen.tolist(), \
                    right_on=common_columns_blogosphere_tg_vk_zen.tolist(), \
                    how='outer')

In [249]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,TikTok блогер,TikTok количество подписчиков,TikTok среднее кол-во просмотров,TikTok стоимость видеопост,Дзен блогер,Дзен количество подписчиков,Дзен среднее кол-во просмотров,Дзен стоимость пост,TikTok ссылка,Дзен ссылка
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,ömankö,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,василий уткин,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,джарахов,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,владимир яковлев,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [250]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,TikTok блогер,TikTok количество подписчиков,TikTok среднее кол-во просмотров,TikTok стоимость видеопост,Дзен блогер,Дзен количество подписчиков,Дзен среднее кол-во просмотров,Дзен стоимость пост,TikTok ссылка,Дзен ссылка
244,nellifornication - тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,nellifornication - тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,nellifornication - тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('sales@blogo-sphere.com', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

## Подготовка данных по блогерам от Insight People

**Имеющиеся DataFrame:**  

* insight_instagram
* insight_threads
* insight_telegram
* insight_vk_ls
* insight_vk_group
* insight_youtube
* insight_zen
* insight_likee
* insight_yappy
* insight_rutube
<!-- * insight_ok -->

> Предварительно сделаем предобработку DataFrame от Insight People

In [252]:
insight_df = [insight_instagram,
              insight_threads,
              insight_telegram,
              insight_vk_ls,
              insight_vk_group,
              insight_youtube,
              insight_zen,
              insight_likee,
              insight_yappy,
              insight_rutube,]
#               insight_ok]

In [253]:
key_word = ['CPU', 'ER', 'CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов

In [254]:
for i in range(len(insight_df)):
    df = insight_df[i]  # Получаем текущий DataFrame по индексу
    
    upd_columns = []  # Пустой список столбцов с именами блогеров для приведения к единому формату

    if 'Фамилия' in df.columns:
        upd_columns.append('Фамилия')
    if 'Имя' in df.columns:
        upd_columns.append('Имя')
    if 'Имя блогера' in df.columns:
        upd_columns.append('Имя блогера')
    if 'Название канала' in df.columns:
        upd_columns.append('Название канала')
    if 'ФИО' in df.columns:
        upd_columns.append('ФИО')
    if 'Фамилия имя' in df.columns:
        upd_columns.append('Фамилия имя')

    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы insight_instagram

In [255]:
insight_instagram

3,город,фамилия имя,ссылка,тематика,количество подписчиков,охват пост,охват сторис,охват серии сторис,стоимость за фотопост,стоимость за сторис за 15 секунд,стоимость за серию сторис до 60 секунд,стоимость за видеопост / reels
4,Москва,манукян давид,https://instagram.com/dava_m,"Артист, стример, лайфстайл, развлечения, кино,...",13 500 000,1 900 000,700 000,1 750 000,"р.1 244 445,00","р.497 778,00","р.835 555,00","р.1 866 667,00"
5,Москва,лазарьянц карина,https://instagram.com/karinakross,"Развлечения, юмор, лайфстайл",10 400 000,950 000,500 000,1 250 000,"р.832 500,00",под запрос,"р.832 500,00","р.1 165 500,00"
6,Москва,хилькевич анна,https://instagram.com/annakhilkevich,"Лайфстайл, мама, актриса",9 900 000,410 000,120 000,300 000,"р.510 000,00","р.170 000,00","р.425 000,00","р.600 000,00"
7,Москва,левда алина,https://www.instagram.com/botanovna_/,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",7 600 000,950 000,420 000,1 050 000,"р.1 322 222,00",под запрос,"р.622 222,00","р.1 555 555,00"
8,Москва,ершов евгений,https://instagram.com/ershovich,"Лайфстайл, контент-мейкер, развлечения",5 300 000,450 000,400 000,1 000 000,"р.422 222,00","р.116 111,00","р.306 111,00","р.570 000,00"
...,...,...,...,...,...,...,...,...,...,...,...,...
86,Тюмень,буркова мария,https://www.instagram.com/sinitsynamv/,"Развлечения, семья, дети",98 000,30 000,13 000,32 500,"р.60 000,00","р.30 000,00","р.75 000,00","р.120 000,00"
87,Москва (Саратов),рюмшина алина,https://instagram.com/wow_rumsha,"Лайфстайл, юмор, бьюти",91 300,60 000,20 000,50 000,"р.37 500,00",по запросу,"р.28 000,00","р.55 500,00"
88,Тверь,вишневская алиса,https://instagram.com/alice.vishneva,"Бьюти, фешн",53 900,30 000,8 700,21 750,"р.75 000,00","р.75 000,00","р.165 000,00","р.195 000,00"
89,Екатеринбург,грицай елена,https://www.instagram.com/elena_gricai/,"Лайфстайл, семья",52 700,5 000,10 000,25 000,"р.12 648,00","р.6 324,00","р.10 118,00","р.22 766,00"


> Сделаем общий столбец с именем и фамилией блогера, и так как данное действие мы будем делать практически у всех DataFrame от Insight, то сделаем это функцией, а так как мы будем делать это для большинства DataFrame от Insight People, то сделаем это функцией

In [256]:
[col for col in insight_instagram.columns]

['город',
 'фамилия имя',
 'ссылка',
 'тематика',
 'количество подписчиков',
 'охват пост',
 'охват сторис',
 'охват серии сторис',
 'стоимость за фотопост',
 'стоимость за сторис за 15 секунд',
 'стоимость за серию сторис до 60 секунд',
 'стоимость за видеопост / reels']

In [257]:
[col for col in bufer.columns if 'Instagram' in col]

['Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика']

In [258]:
insight_instagram.rename(columns =  {'фамилия имя':'блогер',
                                     'стоимость за фотопост':'стоимость фотопост',
                                     'охват пост':'среднее кол-во просмотров фотопоста',
                                     'охват сторис':'среднее кол-во просмотров 1 сторис',
                                     'охват серии сторис':'среднее кол-во просмотров серии сторис',
                                     'стоимость за сторис за 15 секунд':'стоимость 1 сторис до 15 сек',
                                     'стоимость за серию сторис до 60 секунд':'стоимость серия сторис до 60 сек',
                                     'стоимость за видеопост / reels':'стоимость reels / видеопост'}, inplace=True)

#### Проверка на дубликаты

> **ДОПИСАТЬ**

In [259]:
def show_duplicates(df):
    
    """
    """
    
    df = df.drop_duplicates()

    duplicates_df = df[df.duplicated('блогер', keep=False)]

    print("Количество дубликатов:", len(duplicates_df))

    return duplicates_df.sort_values(by='блогер')

In [260]:
show_duplicates(insight_instagram)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров серии сторис,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия сторис до 60 сек,стоимость reels / видеопост


> В исходном DataFrame у нас есть дубликат, который отличается только значением в столбце "Instagram стоимость за серию сторис до 60 сек". Удалим один из них по индексу

In [261]:
duplicates_insight_instagram = insight_instagram[insight_instagram.duplicated('блогер', keep=False)]
duplicates_insight_instagram

3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров серии сторис,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия сторис до 60 сек,стоимость reels / видеопост


In [262]:
indexes_to_duplicates_insight_instagram = duplicates_insight_instagram[duplicates_insight_instagram.index % 2 == 0]
indexes_to_duplicates_insight_instagram = indexes_to_duplicates_insight_instagram.index.tolist()
indexes_to_duplicates_insight_instagram

[]

In [263]:
insight_instagram.drop(indexes_to_duplicates_insight_instagram, inplace=True)

In [264]:
show_duplicates(insight_instagram)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров серии сторис,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия сторис до 60 сек,стоимость reels / видеопост


### Двенадцатое объединение

In [265]:
bufer = merge_bufer_df(bufer, insight_instagram, 'Instagram')

Количество дубликатов: 3


In [266]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,TikTok стоимость видеопост,Дзен блогер,Дзен количество подписчиков,Дзен среднее кол-во просмотров,Дзен стоимость пост,TikTok ссылка,Дзен ссылка,Instagram город,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vikishow,лайфстайл/дети/подростки,vikishow,https://www.instagram.com/vikishow_official/,лайфстайл/дети/подростки,2 500 000,70 000,30 000,80 000,15 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,буркова мария,"Развлечения, семья, дети",буркова мария,https://www.instagram.com/sinitsynamv/,"Развлечения, семья, дети",98 000,30 000,13 000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тюмень,32 500,"р.75 000,00"
350,рюмшина алина,"Лайфстайл, юмор, бьюти",рюмшина алина,https://instagram.com/wow_rumsha,"Лайфстайл, юмор, бьюти",91 300,60 000,20 000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Москва (Саратов),50 000,"р.28 000,00"
351,вишневская алиса,"Бьюти, фешн",вишневская алиса,https://instagram.com/alice.vishneva,"Бьюти, фешн",53 900,30 000,8 700,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тверь,21 750,"р.165 000,00"
352,грицай елена,"Лайфстайл, семья",грицай елена,https://www.instagram.com/elena_gricai/,"Лайфстайл, семья",52 700,5 000,10 000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Екатеринбург,25 000,"р.10 118,00"


### Подготовка таблицы insight_threads

In [267]:
insight_threads.head()

3,город,фамилия,имя,ссылка,тематика,количество подписчиков,стоимость за фотопубликацию,стоимость за видеопубликацию
4,Москва,манукян,давид,https://www.threads.net/@dava_m,"Артист, стример, лайфстайл, развлечения, кино,...",70 800,под запрос,под запрос
5,Москва,акилова,алина,https://www.threads.net/@alina_akilova,"Лайфстайл, бьюти, мама",50 900,под запрос,под запрос
6,Москва,серов,никита,https://www.threads.net/@serovski,"Лайфстайл, семья, путешествия, авто",43 700,под запрос,под запрос
7,Москва,левда,алина,https://www.threads.net/@botanovna_,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",42 400,под запрос,под запрос
8,Москва,нигай,карина,https://www.threads.net/@karina_nigay,"Лайфстайл, фешн, мода, бьюти, путешествия, семья",32 500,под запрос,под запрос


In [268]:
def name_familia(df: pd.DataFrame) -> pd.DataFrame:
    
    """
    Функция объединяет два столбца 'имя' и 'фамилия' нашего DataFrame df в единый столбец 'блогер'.
    Далее мы удаляем столбцы 'имя' и 'фамилия' из DataFrame.
    
    
    ПАРАМЕТРЫ:
    
    df (DataFrame) - DataFrame, в котором мы обновляем столбцы
    
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    
    df (DataFrame) - обновлённый DataFrame
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    insight_instagram = name_familia(insight_instagram)
    
    """
    
    df['блогер'] = df['фамилия'] + ' ' + df['имя']

    df.drop(columns=['имя', 'фамилия'], axis= 1, inplace=True)
    
    return df

In [269]:
insight_threads = name_familia(insight_threads)

In [270]:
insight_threads.head()

3,город,ссылка,тематика,количество подписчиков,стоимость за фотопубликацию,стоимость за видеопубликацию,блогер
4,Москва,https://www.threads.net/@dava_m,"Артист, стример, лайфстайл, развлечения, кино,...",70 800,под запрос,под запрос,манукян давид
5,Москва,https://www.threads.net/@alina_akilova,"Лайфстайл, бьюти, мама",50 900,под запрос,под запрос,акилова алина
6,Москва,https://www.threads.net/@serovski,"Лайфстайл, семья, путешествия, авто",43 700,под запрос,под запрос,серов никита
7,Москва,https://www.threads.net/@botanovna_,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",42 400,под запрос,под запрос,левда алина
8,Москва,https://www.threads.net/@karina_nigay,"Лайфстайл, фешн, мода, бьюти, путешествия, семья",32 500,под запрос,под запрос,нигай карина


In [271]:
insight_threads.rename(columns = {'стоимость за фотопубликацию':'стоимость фотопост',
                                  'стоимость за видеопубликацию':'стоимость видеопост'}, inplace=True)

#### Проверка на дубликаты

In [272]:
show_duplicates(insight_threads)

Количество дубликатов: 2


3,город,ссылка,тематика,количество подписчиков,стоимость фотопост,стоимость видеопост,блогер
15,Москва,https://www.threads.net/@leratumanova,Лайфстайл,13 200,под запрос,под запрос,туманова валерия
16,Москва,https://www.threads.net/@leratumanova,"Лайфстайл, артист, семья, путешествия, мода, б...",13 100,под запрос,под запрос,туманова валерия


> Опять есть дубликат - удалим тот, где меньше всего данных в столбце "тематика", так как в единственном столбце "количество подписчиков", где есть изменения, цифры практически не отличаются

In [273]:
duplicates_insight_threads = insight_threads[insight_threads.duplicated('блогер', keep=False)]
duplicates_insight_threads

3,город,ссылка,тематика,количество подписчиков,стоимость фотопост,стоимость видеопост,блогер
15,Москва,https://www.threads.net/@leratumanova,Лайфстайл,13 200,под запрос,под запрос,туманова валерия
16,Москва,https://www.threads.net/@leratumanova,"Лайфстайл, артист, семья, путешествия, мода, б...",13 100,под запрос,под запрос,туманова валерия


In [274]:
indexes_to_duplicates_insight_threads = duplicates_insight_threads[duplicates_insight_threads.index % 2 != 0]
indexes_to_duplicates_insight_threads = indexes_to_duplicates_insight_threads.index.tolist()
indexes_to_duplicates_insight_threads

[15]

In [275]:
insight_threads.drop(indexes_to_duplicates_insight_threads, inplace=True)

In [276]:
show_duplicates(insight_threads)

Количество дубликатов: 0


3,город,ссылка,тематика,количество подписчиков,стоимость фотопост,стоимость видеопост,блогер


### Тринадцатое объединение

> У нас нет общих столбцов, поэтому объединим по имени блогера

In [277]:
bufer = merge_bufer_df(bufer, insight_threads, 'Threads')

Количество дубликатов: 3


### Подготовка таблицы insight_telegram

In [278]:
insight_telegram

3,город,фамилия имя,ссылка,тематика,количество подписчиков,средний охват публикации,кружок,стоимость за фотопост,стоимость за видеопост
4,Барнаул,горбунова надежда,https://t.me/brai_sa,"Лайфстайл, Сериальные квесты",276 663,154 000,"р.448 000,00","р.448 000,00","р.512 000,00"
5,Нижний Новгород,шишкова оксана,https://t.me/oxgerman,"Лайфстайл, семья, путешествия, еда",216 000,120 000,"р.180 000,00","р.300 000,00","р.360 000,00"
6,Москва,левда алина,https://t.me/+-wxY1yGLRhU0NzMy,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",200 000,35 000,"р.260 000,00","р.292 500,00","р.364 000,00"
7,Краснодар,морозова алина,https://t.me/Alinamorozovaofficial,"Нутрициолог, ЗОЖ",184 403,84 000,"р.144 000,00","р.144 000,00","р.144 000,00"
8,Кемерово,гаврилова анжелика,https://t.me/anj0731,"Семья, лайфстайл, бьюти-обзоры",155 000,130 000,"р.150 000,00","р.210 000,00","р.300 000,00"
9,Москва,лазарьянц карина,https://t.me/KR0SSNEWS,"Развлечение, юмор, лайфстайл",133 000,20 000,"р.750 000,00","р.575 000,00","р.750 000,00"
10,Москва,манукян давид,https://t.me/davatelega,"Артист, стример, лайфстайл, развлечения, кино,...",117 000,35 000,"р.168 000,00","р.342 222,00","р.466 666,00"
11,Иваново,аверкин максим,https://t.me/+sgfBETmdbBg1NDFi,"Новости, треш , 18+",115 556,17 000,"р.73 955,00","р.110 934,00","р.166 403,00"
12,Москва,зарубин александр,https://t.me/sasha_stone13,"Лайфстайл, контент-мейкер, развлечения",114 000,60 000,"р.228 000,00","р.342 000,00","р.380 000,00"
13,Воронеж,енина анна,https://t.me/ane4ka24,Лайфстайл,104 489,40 000,"р.62 694,00","р.100 311,00","р.160 497,00"


> Переименуем столбцы под общий формат

In [279]:
[col for col in insight_telegram.columns]

['город',
 'фамилия имя',
 'ссылка',
 'тематика',
 'количество подписчиков',
 'средний охват публикации',
 'кружок',
 'стоимость за фотопост',
 'стоимость за видеопост']

In [280]:
[col for col in bufer.columns if 'Telegram' in col]

['Telegram блогер',
 'Telegram ссылка',
 'Telegram тематика',
 'Telegram количество подписчиков',
 'Telegram среднее кол-во просмотров поста',
 'Telegram средняя вовлечённость',
 'Telegram стоимость фотопост',
 'Telegram стоимость видеопост',
 'Telegram стоимость кружочек до 60 сек',
 'Telegram стоимость текстовое сообщение без фото',
 'Telegram статистика']

In [281]:
insight_telegram.rename(columns = {'фамилия имя':'блогер',
                                   'средний охват публикации':'среднее кол-во просмотров поста',
                                   'кружок':'стоимость кружочек до 60 сек',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост'}, inplace = True)

#### Проверка на дубликаты

In [282]:
show_duplicates(insight_telegram)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров поста,стоимость кружочек до 60 сек,стоимость фотопост,стоимость видеопост


> Дубликатов нет - может объединять

### Четырнадцатое объединение

> Исправим одну неточность в имени блогера

In [283]:
insight_telegram.loc\
    [insight_telegram['блогер'] == 'сивцев станислав', 'блогер']  = 'станислав сивцев'

In [284]:
bufer = merge_bufer_df(bufer, insight_telegram, 'Telegram')

Количество дубликатов: 49


In [285]:
show_duplicates(bufer).head(6)

Количество дубликатов: 48


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек,Threads город,Threads ссылка,Threads тематика,Threads количество подписчиков,Threads стоимость фотопост,Threads стоимость видеопост,Threads блогер,Telegram город
209,nellifornication - тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,nellifornication - тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,акилова алина,"Лайфстайл, бьюти",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Москва
223,акилова алина,"Лайфстайл, бьюти, мама",акилова алина,https://instagram.com/alina_akilova,"Лайфстайл, бьюти, мама",3 900 000,350 000,250 000,NaN,NaN,...,625 000,"р.422 222,00",Москва,https://www.threads.net/@alina_akilova,"Лайфстайл, бьюти, мама",50 900,под запрос,под запрос,акилова алина,NaN
436,ахметзянов айрат,"Food / Еда, рецепты",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Казань
282,ахметзянов айрат,"Фудблогер, рецепты,готовка",ахметзянов айрат,https://instagram.com/bistrie.recepty2.0,"Фудблогер, рецепты,готовка",256 000,120 000,48 000,NaN,NaN,...,120 000,"р.30 720,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> У нас есть определённая проблема с DataFrame insight_telegram - почти половина каналов не заполнились с помощью merge, а сделались новыми строками - скорее всего это произошло из-за особого форматирования данных столбца "блогер", так как проверка ниже показывает, что данные равны по значению, и нам надо объединить данные строки в единую строку.

In [286]:
# Проверка
test_dub = bufer.loc[bufer['блогер'] == 'ахметзянов айрат']
test_dub.reset_index()

,index,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,...,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек,Threads город,Threads ссылка,Threads тематика,Threads количество подписчиков,Threads стоимость фотопост,Threads стоимость видеопост,Threads блогер,Telegram город
0,282,ахметзянов айрат,"Фудблогер, рецепты,готовка",ахметзянов айрат,https://instagram.com/bistrie.recepty2.0,"Фудблогер, рецепты,готовка",256 000,120 000,48 000,NaN,...,120 000,"р.30 720,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,436,ахметзянов айрат,"Food / Еда, рецепты",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Казань


In [287]:
string1 = test_dub.iloc[0]['блогер']
string2 = test_dub.iloc[1]['блогер']

print(repr(string1))
print(repr(string2))

'ахметзянов айрат'
'ахметзянов айрат'


In [288]:
# Сравним их
if repr(string1) == repr(string2):
    print("Строки идентичны")
else:
    print("Строки различаются")

Строки идентичны


> Отдельно стоит отметить, что даже если совпадают имена блогера, то бывают случаи, когда другие значения, в том числе по ценам, но чаще всего по столбцу "тематика", не совпадают, из-за чего мы не можем удалить дубликаты - учтём это в функции ниже

In [289]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
duplicates_bufer_sample = duplicates_bufer.loc[duplicates_bufer['блогер'] == 'бурмистрова татьяна']
duplicates_bufer_sample[['блогер', 'тематика']]

,блогер,тематика
245,бурмистрова татьяна,"Мама, лайфстайл"
410,бурмистрова татьяна,"Лайфстайл, семья, путешествия, развлечения, об..."


> Напишем функцию, которая будет заполненять пропущенные значения в DataFrame, объединенять уникальные значений в столбцах, а также удалять дубликаты строк и выполняет обработку столбцов на основе заданных правил, включая разделение значений и обновление 'тематики' для каждого блогера.

In [290]:
def duplicates_stroka(df: pd.DataFrame, col_to_check: str, columns_to_upd: str) -> pd.DataFrame:
    
    """
    Эта функция выполняет заполнение пропущенных значений в датафрейме, 
    объединение уникальных значений в столбцах, а также удаляет дубликаты строк и
    выполняет обработку столбцов на основе заданных правил, включая разделение значений и обновление 'тематики' для каждого блогера.
    
    
    ПАРАМЕТРЫ:
    df (DataFrame) - DataFrame, в котором мы будет делать преобразования
    col_to_check (str) - столбец, по которому мы будет смотреть дубликаты (чаще всего имя блогера)
    
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (DataFrame) - обновлённый DataFrame  
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    col_to_check = 'блогер'
    bufer = duplicates_bufer(bufer, col_to_check)
    """

    for col in df.columns:
        if col != col_to_check: # Пропускаем столбец, по которому выполняется проверка
            
            df['temp_fill'] = np.nan

# Создается временный столбец с именем 'temp_fill', который содержит последнее (по индексу)
# непустое значение из столбца col для каждой группы, сгруппированной по значениям в столбце col_to_check.
# Это выполняется с помощью bufer.groupby(col_to_check)[col].transform('last').
        
            df['temp_fill'] = df.groupby(col_to_check)[col].transform('last')  # Создание временного столбца

# Пропущенные значения в столбце col заполняются соответствующими значениями
# из временного столбца 'temp_fill' с использованием bufer[col].fillna(bufer['temp_fill']).
        
            df[col] = df[col].fillna(df['temp_fill'])  # Заполнение пропущенных значений

# Сбрасываются индексы в DataFrame bufer с помощью bufer.reset_index(drop=True). 
# Это делается для того, чтобы индексы начались с нуля после заполнения пропущенных значений.
            df = df.reset_index(drop=True)  # Сброс индексов

# Удаляется временный столбец 'temp_fill' с помощью bufer = bufer.drop(columns=['temp_fill']).    
            df = df.drop(columns=['temp_fill'])  # Удаление временного столбца

# # Удаляются дубликаты строк в DataFrame bufer с помощью bufer = bufer.drop_duplicates().    
#             df = df.drop_duplicates()
   
#     df = df.drop_duplicates()
    
    for column in df.columns:
        
        if 'тематика' in column:
            df[column] = df[column].astype(str)
            df[column] = df[column].fillna('')
            
            df[column] = df[column].str.strip()\
                                    .str.lower()\
                                    .str.replace('/', ', ')\
                                    .str.replace(r'\n', ', ')\
                                    .str.replace('*', '')\
                                    .str.replace(', ', ',')\
                                    .str.replace(',', ', ')
            df[column] = df[column].str.strip()
#             df[column] = df[column].str.split(', ')    
    
    for blogger in df[col_to_check].unique():
        blogger_rows = df[df[col_to_check] == blogger]
        unique_thematics = set()
        for column in blogger_rows.columns[blogger_rows.columns.str.contains(columns_to_upd)]:
            for value in blogger_rows[column].str.split(', ').dropna():
                unique_thematics.update(value)
        unique_thematics = ', '.join(sorted(unique_thematics))
        df.loc[df[col_to_check] == blogger, blogger_rows.columns[blogger_rows.columns.str.contains(columns_to_upd)]] = unique_thematics
    
    
    for column in df.columns:
        if columns_to_upd in column:    
            df[column] = df[column].str.lstrip('none, ')
#             df[column] = df[column].str.lstrip(', ')
            

    
    df = df.drop_duplicates()


    return df

In [291]:
# До 
bufer.shape

(444, 137)

> Сделаем преобразование

In [292]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [293]:
# После

bufer.shape

(419, 137)

> Мы убрали почти 40 дубликатов, кроме тех, где есть отличия в ценах

> Проверим результат:

> Ниже видно, что заполнились все столбцы и объединились правильно по блогеру, а также заполнились все столбцы, где есть слово "тематика" суммарным списком тематик

In [294]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек,Threads город,Threads ссылка,Threads тематика,Threads количество подписчиков,Threads стоимость фотопост,Threads стоимость видеопост,Threads блогер,Telegram город


In [295]:
bufer.loc[bufer['блогер'] == 'бурмистрова татьяна']

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек,Threads город,Threads ссылка,Threads тематика,Threads количество подписчиков,Threads стоимость фотопост,Threads стоимость видеопост,Threads блогер,Telegram город
245,бурмистрова татьяна,"лайфстайл, мама, обзоры, путешествия, развлече...",бурмистрова татьяна,https://instagram.com/tassi_stories,"лайфстайл, мама, обзоры, путешествия, развлече...",1 000 000,400 000,250 000,None,None,...,625 000,"р.600 000,00",None,None,"лайфстайл, мама, обзоры, путешествия, развлече...",None,None,None,None,Владивосток


### Подготовка таблицы insight_vk_ls

In [296]:
insight_vk_ls

3,город,фамилия имя,ссылка,тематика,количество подписчиков,средние просмотры поста,средие просмотры клипа вк,стоимость за vk clip,стоимость за фотопост,стоимость за видеопост,"передача прав на paid social , ( 1 месяц)"
4,Казань,шарипов руслан,https://vk.com/sharipoffru,"Лайфстайл, юмор, развлечения, папа",357 000,15 000,15 000,"р.66 000,00","р.44 000,00","р.66 000,00","р.44 000,00"
5,Москва,данилова арина,https://vk.com/id230302965,"Тиктокер, артист, лайфстайл",251 000,10 000,20 000,"р.56 000,00","р.40 000,00","р.56 000,00",под запрос
6,Москва,курчанов евгений,https://vk.com/kuuurchanov,"Лайфстайл, контент-мейкер, развлечения",192 500,20 000,20 000,"р.50 000,00","р.45 000,00","р.50 000,00","р.30 000,00"
7,Нижний Новгород,федулова елизавета,https://vk.com/indigo_liza,"Лайфстайл, контент-мейкер, отношения",120 000,40 000,15 000,"р.38 400,00","р.80 000,00","р.38 400,00","р.53 333,00"
8,Москва,пиндюра артем,https://vk.com/kid_was_here,"Артист, музыка, лайфстайл",118 400,2 000,-,под запрос,под запрос,под запрос,под запрос
9,Тула,бубнов денис,https://vk.com/bubnov_denis,"Лайфстайл, развлечения, контент-мейкер",116 000,10 000,30 000,"р.46 771,00","р.25 984,00","р.46 771,00","р.17 323,00"
10,Москва,левда алина,https://vk.com/botanovna,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",106 200,50 000,70 000,"р.385 000,00","р.280 000,00","р.385 000,00","р.50 000,00"
11,Москва,страуфман данил,https://vk.com/lucikok,"Артист, контент-мейкер",99 800,1 000,3 000,"р.127 744,00",под запрос,"р.127 744,00","р.19 162,00"
12,Волгоград,рагимов мурад,http://vk.com/muragi,"Лайфстайл, развлечения, контент-мейкер, отношения",85 600,6 500,18 900,"р.34 514,00","р.19 174,00","р.34 514,00","р.12 783,00"
13,Воронеж,французов григорий,https://vk.com/grigamoscow,"Лайфстайл, юмор, развлечения",84 500,30 000,80 000,"р.128 000,00","р.48 000,00","р.128 000,00","р.32 000,00"


In [297]:
insight_vk_ls.rename(columns = {'фамилия имя':'блогер',
                                'средние просмотры поста':'среднее кол-во просмотров поста',
                                'средие просмотры клипа вк':'среднее кол-во просмотров вк клипа',
                                'стоимость за vk clip':'стоимость вк клип',
                                'стоимость за фотопост':'стоимость фотопост',
                                'стоимость за видеопост':'стоимость видеопост',
                                'передача прав на paid social , ( 1 месяц)':'стоимость передачи прав на 1 месяц'
                               }, inplace = True)

#### Проверка на дубликаты

In [298]:
show_duplicates(insight_vk_ls)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,стоимость вк клип,стоимость фотопост,стоимость видеопост,стоимость передачи прав на 1 месяц


### Пятнадцатое объединение

In [299]:
bufer = merge_bufer_df(bufer, insight_vk_ls, 'VK личная страница')

Количество дубликатов: 0


In [300]:
# До

bufer.shape

(451, 148)

In [301]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [302]:
# После

bufer.shape

(451, 148)

In [303]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница блогер,VK личная страница ссылка,VK личная страница тематика,VK личная страница количество подписчиков,VK личная страница среднее кол-во просмотров поста,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц


In [304]:
bufer[bufer['блогер'] == 'юлия ланге']

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница блогер,VK личная страница ссылка,VK личная страница тематика,VK личная страница количество подписчиков,VK личная страница среднее кол-во просмотров поста,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц


### Подготовка таблицы insight_vk_group

In [305]:
insight_vk_group

3,город,фамилия имя,ссылка,тематика,количество подписчиков,средние просмотры публикации,средние просмотры клипа вк,стоимость за vk clip,стоимость за фотопост,стоимость за видеопост,"передача прав на paid social , ( 1 месяц)"
4,Москва,лазарьянц карина,https://vk.com/krosskarina,"Развлечение, юмор, лайфстайл",1 400 000,230 000,500 000,"р.650 000,00","р.410 000,00","р.650 000,00",под запрос
5,Москва,манукян давид,https://vk.com/dava,"Артист, стример, лайфстайл, развлечения, кино,...",1 200 000,500 000,500 000,"р.850 000,00","р.680 000,00","р.850 000,00",под запрос
6,Тюмень,самчук елена,https://vk.com/samchuk_studio,"Развлечения, юмор, лайфстайл",801 000,800 000,600 000,"р.1 125 000,00","р.550 000,00",под запрос,"р.150 000,00"
7,Москва,ершов женя,https://vk.com/ershov_officiall,"Лайфстайл, контент-мейкер, развлечения",611 000,350 000,400 000,"р.340 000,00","р.240 000,00","р.340 000,00",под запрос
8,Москва,хилькевич анна,https://vk.com/khilkevich.public,"Актриса, лайфстайл, развлечения, юмор, семья",399 000,150 000,200 000,"р.500 000,00","р.350 000,00","р.500 000,00","р.130 000,00"
9,Москва,селиванов владимир,https://vk.com/vavanmusic,"Артист, юмор, развлечения",255 000,40 000,60 000,"р.338 000,00","р.135 000,00","р.338 000,00","р.41 000,00"
10,Москва,сарычев кирилл,https://vk.com/sarychev_kirill,"Спорт, борьба, развлечения, юмор, лайфстайл",253 000,60 000,80 000,"р.320 000,00","р.250 000,00","р.320 000,00","р.100 000,00"
11,Москва,зарубин александр,https://vk.com/sashka_stone,"Лайфстайл, контент-мейкер, развлечения",235 000,200 000,300 000,"р.340 000,00","р.510 000,00","р.340 000,00","р.94 445,00"
12,Москва,гончарова наталья,https://vk.com/bigkitty_b,"Лайфстайл, юмор, развлечения",190 000,200 000,400 000,"р.400 000,00","р.300 000,00","р.400 000,00","р.200 000,00"
13,Москва,байдавлетова анна,https://vk.com/nutaloveis,"Артист, певица, лайфстайл, мода, животные, бью...",189 000,50 000,150 000,"р.154 800,00","р.63 000,00","р.154 800,00","р.39 600,00"


In [306]:
insight_vk_group.rename(columns = {'фамилия имя':'блогер',
                                   'средние просмотры публикации':'среднее кол-во просмотров поста',
                                   'средние просмотры клипа вк':'среднее кол-во просмотров вк клипа',
                                   'стоимость за vk clip':'стоимость вк клип',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост',
                                   'передача прав на paid social , ( 1 месяц)':'стоимость передачи прав на 1 месяц'
                                  }, inplace = True)

#### Проверка на дубликаты

In [307]:
show_duplicates(insight_vk_group)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,стоимость вк клип,стоимость фотопост,стоимость видеопост,стоимость передачи прав на 1 месяц


### Шестнадцатое объединение

In [308]:
bufer = merge_bufer_df(bufer, insight_vk_group, 'VK группа')

Количество дубликатов: 56


In [309]:
show_duplicates(bufer).head(6)

Количество дубликатов: 56


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница тематика,VK личная страница количество подписчиков,VK личная страница среднее кол-во просмотров поста,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц
460,байдавлетова анна,"Артист, певица, лайфстайл, мода, животные, бью...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Москва,"р.39 600,00"
369,байдавлетова анна,"артист, бьюти, животные, контент-мейкер, лайфс...",None,None,"артист, бьюти, животные, контент-мейкер, лайфс...",None,None,None,None,None,...,"артист, бьюти, животные, контент-мейкер, лайфс...",None,None,None,None,None,None,None,NaN,NaN
446,брыкун анна,дизайн,None,None,дизайн,None,None,None,None,None,...,дизайн,12 000,2 000,20 000,"р.39 690,00","р.11 340,00","р.39 690,00","р.7 560,00",NaN,NaN
489,брыкун анна,Дизайн,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тверь,"р.5 248,00"
487,буркова мария,"Семья, отношения, путешествия, лайфстайл",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тюмень,"р.32 000,00"
362,буркова мария,"дети, развлечения, семья",буркова мария,https://www.instagram.com/sinitsynamv/,"дети, развлечения, семья",98 000,30 000,13 000,None,None,...,"дети, развлечения, семья",None,None,None,None,None,None,None,NaN,NaN


In [310]:
# До

bufer.shape

(495, 150)

In [311]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [312]:
# После

bufer.shape

(467, 150)

In [313]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates.sort_values(by='блогер')

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница тематика,VK личная страница количество подписчиков,VK личная страница среднее кол-во просмотров поста,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц


### Подготовка таблицы insight_youtube

> Название канала

In [314]:
insight_youtube

3,город,название канала,фамилия имя,ссылка,тематика,количество подписчиков,просмотры видео среднее кол-во,просмотры shorts среднее кол-во,стоимость преролла,стоимость интеграции (до 1 минуты),стоимость shorts,стоимость эксклюзивного ролика
4,Москва,dava,манукян давид,https://youtube.com/channel/UCbh7BxJWCMnWLwJ-t...,"Артист, стример, лайфстайл, развлечения, кино,...",5 440 000,450 000,-,р.700 000,"р.1 000 000,00",-,"р.1 360 000,00"
5,Санкт-Петербург,mamasobolina,соболева мария,https://www.youtube.com/channel/UCv6aQfBFfbepf...,"Семья, лайфстайл, дети, шортс-формат",5 200 000,60 000,50 000,-,-,"р.60 000,00","р.135 000,00"
6,Тверь,виола,кузьмина варвара,https://youtube.com/channel/UCyCKV6zM59D8yT24e...,"Юмор, развлечения (Шортс)",3 820 000,332 000,600 000,-,-,"р.90 000,00",-
7,Тверь,druemae,параманов андрей,https://youtube.com/channel/UC4WC9TkTTtnsVz6Ay...,"Шортс, развлечения, лайфстайл, юмор",3 740 000,110 000,120 000,"р.135 000,00","р.150 000,00","р.135 000,00","р.225 000,00"
8,Москва,dinara salimova,файзрахманова динара,https://www.youtube.com/c/DinaraSalimova/videos,"Развлечения, юмор, лайфстайл, шортс, обзоры",3 560 000,60 000,70 000,"р.45 000,00","р.75 000,00","р.67 500,00","р.135 000,00"
9,Пенза,красавица и чудовище,молоканов семен,https://www.youtube.com/c/%D0%9A%D1%80%D0%B0%D...,Развлечения,3 140 000,350 000,200 000,"р.300 000,00","р.600 000,00","р.450 000,00",под запрос
10,Казань,аяз насибуллин,насибуллин аяз,https://www.youtube.com/channel/UCK1EBno24N3SC...,"Шортс, развлечения, лайфстайл, юмор",3 090 000,200 000,110 000,-,-,"р.195 000,00","р.360 000,00"
11,Москва,milana star,гогунская милана,https://youtube.com/c/MILANASTARR,"Лайфстайл, развлечения, юмор, артист, музыка, ...",3 002 000,300 000,230 000,"р.300 000,00","р.520 000,00","р.468 000,00","р.560 000,00"
12,Барнаул,ная,горбунова надежда,https://youtube.com/@brai_sa,Сериальные квесты,2 610 000,400 000,300 000,"р.360 000,00","р.435 000,00","р.391 500,00","р.600 000,00"
13,Нижний Новгород,hanna11-тиктокер с 4 мультами,суворова анна,https://www.youtube.com/channel/UC-rntvBxAngmY...,"Юмор,развлечения, шортс формат",2 180 000,500 000,500 000,-,-,"р.450 000,00","р.900 000,00"


In [315]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость интеграция для профильных клиентов 1 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 1 слот',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость интеграция для непрофильных клиентов 2 слот',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция']

In [316]:
insight_youtube.rename(columns = {'фамилия имя':'блогер',
                                  'просмотры видео среднее кол-во':'среднее кол-во просмотров видео',
                                  'просмотры shorts среднее кол-во':'среднее кол-во просмотров shorts',
                                  'стоимость преролла':'стоимость преролл до 30 сек',
                                  'стоимость интеграции (до 1 минуты)':'стоимость интеграция 1 слот',
                                  'стоимость shorts':'стоимость за shorts до 1 мин',
                                  'стоимость эксклюзивного ролика':'стоимость выпуск под клиента'
                                 }, inplace = True)

#### Проверка на дубликаты

In [317]:
show_duplicates(insight_youtube)

Количество дубликатов: 2


3,город,название канала,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,стоимость преролл до 30 сек,стоимость интеграция 1 слот,стоимость за shorts до 1 мин,стоимость выпуск под клиента
19,Москва,kirill sarychev,сарычев кирилл,https://www.youtube.com/c/KIRILLSARYCHEV,"Спорт, борьба, развлечения, юмор, лайфстайл",1 390 000,400 000,-,"р.220 000,00","р.300 000,00",-,"р.1 200 000,00"
32,Москва,биполярка,сарычев кирилл,https://www.youtube.com/channel/UCmCyOwGrgqAPH...,"Спорт, борьба, развлечения, юмор, лайфстайл",269 000,120 000,-,"р.150 000,00","р.280 000,00",-,"р.500 000,00"


> Два канала сделаны как разные строки по одному блогеру - исправим это: переименуем вторую строчку на название канала - 'биполярка сарычева'

In [318]:
insight_youtube.loc\
    [insight_youtube['название канала'] == 'биполярка', 'блогер']  = 'биполярка сарычева'

In [319]:
# duplicates_insight_youtube = insight_youtube[insight_youtube.duplicated('блогер', keep=False)]
# duplicates_insight_youtube

# indexes_insight_youtube_sarychev = duplicates_insight_youtube_sarychev.index.tolist()

# indexes_insight_youtube_sarychev

# insight_youtube.drop(indexes_insight_youtube_sarychev, inplace=True)

# columns_with_sarychev = ['количество подписчиков',
#                          'среднее кол-во просмотров',
#                          'среднее кол-во просмотров shorts',
#                          'стоимость за преролл до 30 сек',
#                          'стоимость за интеграцию до 1 мин',
#                          'стоимость за shorts до 1 мин',
#                          'стоимость за выпуск под клиента',
#                          'блогер']

# # Удаление всех столбцов, кроме тех, что в списке columns_with_sarychev
# duplicates_insight_youtube_sarychev = duplicates_insight_youtube_sarychev[columns_with_sarychev]
# duplicates_insight_youtube_sarychev

In [320]:
show_duplicates(insight_youtube)

Количество дубликатов: 0


3,город,название канала,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,стоимость преролл до 30 сек,стоимость интеграция 1 слот,стоимость за shorts до 1 мин,стоимость выпуск под клиента


### Семнадцатое объединение

In [321]:
bufer = merge_bufer_df(bufer, insight_youtube, 'YouTube')

Количество дубликатов: 36


In [322]:
show_duplicates(bufer).head(6)

Количество дубликатов: 36


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц,YouTube город,YouTube название канала,YouTube стоимость за shorts до 1 мин
267,бубнов денис,"контент-мейкер, лайфстайл, развлечения",None,None,"контент-мейкер, лайфстайл, развлечения",None,None,None,None,None,...,30 000,"р.46 771,00","р.25 984,00","р.46 771,00","р.17 323,00",None,None,NaN,NaN,NaN
479,бубнов денис,"Развлечения, юмор, лайфхаки, обзоры",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тула,корявый vlog,-
474,гогунская милана,"Лайфстайл, развлечения, юмор, артист, музыка, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Москва,milana star,"р.468 000,00"
303,гогунская милана,"артист, детство, лайфстайл, музыка, развлечени...",гогунская милана,https://instagram.com/milana__star,"артист, детство, лайфстайл, музыка, развлечени...",993 000,75 000,40 000,None,None,...,None,None,None,None,None,Москва,"р.51 000,00",NaN,NaN,NaN
475,горбунова надежда,Сериальные квесты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Барнаул,ная,"р.391 500,00"
368,горбунова надежда,"pov, лайфстайл, сериальные квесты, стиль",горбунова надежда,https://instagram.com/braiisa,"pov, лайфстайл, сериальные квесты, стиль",173 000,75 200,15 900,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN


In [323]:
# До

bufer.shape

(504, 153)

In [324]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [325]:
# После

bufer.shape

(486, 153)

In [326]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц,YouTube город,YouTube название канала,YouTube стоимость за shorts до 1 мин


### Подготовка таблицы insight_zen

In [327]:
insight_zen

3,город,фамилия,название канала,ссылка,категория,количество подписчиков,средний охват публикации,стоимость размещения за фотопост,стоимость размещения за видеопост
4,Калининград,самоха антон,антон самоха,https://dzen.ru/anton_samoha,"Юрист, адвокат",80 300,800,"р.4 950,00","р.9 240,00"
5,Казань,ахметзянов айрат,bistrie.recepty2.0,https://zen.yandex.ru/bistrierecepty,"Кулинария, еда, рецепты",67 200,3 000,"р.17 500,00","р.37 500,00"
6,Смоленск,бондарев алексей,леша аргумент,http://zen.yandex.ru/bitovuxa,"Развлечения, юмор",47 400,9 000,"р.24 000,00","р.36 000,00"
7,Санкт-Петербург,фокин валентин,валентин фокин,https://zen.yandex.ru/profile/editor/id/5ef6d4...,"Развлечения, пранки",44 300,500,"р.8 000,00","р.15 000,00"
8,Москва,беляничева анастасия,анабель шпилька,https://zen.yandex.ru/anabel_shpilka,Юмор,30 300,9 000,"р.24 000,00","р.36 000,00"
...,...,...,...,...,...,...,...,...,...
87,Чебоксары,илларионова,anawesome98,https://dzen.ru/anawesome98,Лайфстайл,4,5,р.1 650,р.3 300
88,Калининград,кубашин,толяныч,https://dzen.ru/tolyanich39rus,юмор,2,50,р.1 500,р.3 000
89,Саратов,арутюнян,official_anson,https://dzen.ru/id/634687a4d8d55824c4427416,"лайфстайл, музыка",1,80,р.1 500,р.3 000
90,Воронеж,жаворонко,викторина,https://dzen.ru/id/635b0565cd00240d96daf44f,-,1,0,р.1 500,р.3 000


#### Обрежем DataFrame по высоте

> Так как мы загружаем данные с CSV, то мы видим не только данные, которые видят все, но и скрытые данные - обрежем данные, чтобы они соответствовали действительности

In [328]:
def trim_df(df: pd.DataFrame, columns: str, number: int) -> pd.DataFrame:
    
    """
    Данная функция trim_df предназначена для обработки и фильтрации данных в DataFrame,
    в которой мы обрезаем DataFrame по определённому размеру числа number в столбце columns

    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - DataFrame, который нужно обработать и фильтровать..
    columns (str) - строка, представляющая название столбца в DataFrame, который должен быть числовой или можно его перевести
    который будет обрабатываться, то есть порог, по которому мы обрезаем DataFrame.
    number (int) - целое число, которое используется для фильтрации данных в указанном столбце.
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - обновлённый DataFrame.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    insight_zen = trim_df(insight_zen, 'количество подписчиков', 999)
        
    """
    
    # Уберем неразрывные пробелы из значений столбца 'количество подписчиков'
    df[columns] = df[columns].str.replace('\xa0', '')
    df[columns] = df[columns].str.replace(' ', '')

    # Теперь преобразуем значения в числа с плавающей точкой
    df[columns] = df[columns].astype(float)

    #Обрежем DataFrame по-определённому размеру столбца columns.
    
    df = df[df[columns] >= number]
    
    #Переведём обратно в str, чтобы у нас не было проблем с объединением.

    df[columns] = df[columns].astype(str)
    
    return df

In [329]:
insight_zen = trim_df(insight_zen, 'количество подписчиков', 999)

In [330]:
[col for col in bufer.columns if 'Дзен' in col]

['Дзен блогер',
 'Дзен количество подписчиков',
 'Дзен среднее кол-во просмотров',
 'Дзен стоимость пост',
 'Дзен ссылка']

In [331]:
[col for col in insight_zen.columns]

['город',
 'фамилия',
 'название канала',
 'ссылка',
 'категория',
 'количество подписчиков',
 'средний охват публикации',
 'стоимость размещения за фотопост',
 'стоимость размещения за видеопост']

In [332]:
insight_zen.rename(columns = {'фамилия':'блогер',
                              'категория':'тематика',
                              'средний охват публикации':'среднее кол-во просмотров',
                              'стоимость размещения за фотопост':'стоимость пост',
                              'стоимость размещения за видеопост':'стоимость видеопост'}, inplace = True)

#### Проверка на дубликаты

> У нас есть дубликаты, но помимо столбца "тематика", также отличаются  столбцы с количествами, а также с городом

In [333]:
show_duplicates(insight_zen)

Количество дубликатов: 6


3,город,блогер,название канала,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость пост,стоимость видеопост
51,Москва,гущина,алена гущина.,https://zen.yandex.ru/id/6132025e71c8776747711662,"Развлечения, юмор",3600.0,5 000,р.9 000,р.20 000
52,Владимир,гущина,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,Развлечения,3600.0,3 000,р.3 750,р.10 000
29,Москва,гущина алена,viviannru,https://zen.yandex.ru/id/6132025e71c8776747711662,Развлечения,6600.0,3 000,"р.3 750,00","р.10 000,00"
37,Владимир,гущина алена,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"Развлечения, юмор",3500.0,5 000,"р.9 000,00","р.20 000,00"
27,Якутск,лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,DIY,7600.0,400,"р.2 500,00","р.5 000,00"
28,Москва,лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,Творчество,7300.0,300,"р.2 500,00","р.5 000,00"


> Обновим столбец "Тематика"

In [334]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
insight_zen = duplicates_stroka(insight_zen, col_to_check, columns_to_upd)

In [335]:
show_duplicates(insight_zen)

Количество дубликатов: 6


3,город,блогер,название канала,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость пост,стоимость видеопост
47,Москва,гущина,алена гущина.,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3600.0,5 000,р.9 000,р.20 000
48,Владимир,гущина,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3600.0,3 000,р.3 750,р.10 000
25,Москва,гущина алена,viviannru,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",6600.0,3 000,"р.3 750,00","р.10 000,00"
33,Владимир,гущина алена,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3500.0,5 000,"р.9 000,00","р.20 000,00"
23,Якутск,лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,"diy, творчество",7600.0,400,"р.2 500,00","р.5 000,00"
24,Москва,лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,"diy, творчество",7300.0,300,"р.2 500,00","р.5 000,00"


> Обновим столбец "Город"

In [336]:
col_to_check = 'блогер'
columns_to_upd = 'город'
insight_zen = duplicates_stroka(insight_zen, col_to_check, columns_to_upd)

In [337]:
show_duplicates(insight_zen)

Количество дубликатов: 6


3,город,блогер,название канала,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость пост,стоимость видеопост
47,"Владимир, Москва",гущина,алена гущина.,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3600.0,5 000,р.9 000,р.20 000
48,"Владимир, Москва",гущина,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3600.0,3 000,р.3 750,р.10 000
25,"Владимир, Москва",гущина алена,viviannru,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",6600.0,3 000,"р.3 750,00","р.10 000,00"
33,"Владимир, Москва",гущина алена,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3500.0,5 000,"р.9 000,00","р.20 000,00"
23,"Москва, Якутск",лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,"diy, творчество",7600.0,400,"р.2 500,00","р.5 000,00"
24,"Москва, Якутск",лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,"diy, творчество",7300.0,300,"р.2 500,00","р.5 000,00"


> Удалим дубликаты, оставив только первое вхождение, так как чаще всего именно в них самые большие и видимо правильные значения

In [338]:
insight_zen = insight_zen.drop_duplicates(subset='блогер', keep='first')

In [339]:
show_duplicates(insight_zen)

Количество дубликатов: 0


3,город,блогер,название канала,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость пост,стоимость видеопост


### Восемнадцатое объединение

In [340]:
bufer = merge_bufer_df(bufer, insight_zen, 'Дзен')

Количество дубликатов: 22


In [341]:
show_duplicates(bufer).head(6)

Количество дубликатов: 22


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц,YouTube город,YouTube название канала,YouTube стоимость за shorts до 1 мин,Дзен город,Дзен название канала,Дзен тематика,Дзен стоимость видеопост
360,ахметзянов айрат,"food , none, готовка, еда, рецепты, фудблогер",ахметзянов айрат,https://instagram.com/bistrie.recepty2.0,"food , none, готовка, еда, рецепты, фудблогер",256 000,120 000,48 000,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
487,ахметзянов айрат,"еда, кулинария, рецепты",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Казань,bistrie.recepty2.0,"еда, кулинария, рецепты","р.37 500,00"
477,бондаренко ирина,"лайфстайл, развелечения, семья, юмор",None,None,"лайфстайл, развелечения, семья, юмор",None,None,None,None,None,...,None,None,None,Новосибирск,bondarinki,"р.30 000,00",NaN,NaN,NaN,NaN
495,бондаренко ирина,"лайфстайл, развелечения, семья",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Новосибирск,бондаринки,"лайфстайл, развелечения, семья","р.9 625,00"
441,горшков алексей,"лайфстайл, лайфхаки, обзоры, помощь людям, раз...",None,None,"лайфстайл, лайфхаки, обзоры, помощь людям, раз...",None,None,None,None,None,...,"р.14 001,00",None,None,Москва,леха медведь,-,NaN,NaN,NaN,NaN
514,горшков алексей,"лайфстайл, развлечения",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Москва,алексей медведь,"лайфстайл, развлечения","р.22 500,00"


In [342]:
# До

bufer.shape

(556, 157)

In [343]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [344]:
# После

bufer.shape

(545, 157)

In [345]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц,YouTube город,YouTube название канала,YouTube стоимость за shorts до 1 мин,Дзен город,Дзен название канала,Дзен тематика,Дзен стоимость видеопост


### Подготовка таблицы insight_likee

In [346]:
insight_likee

3,город,фамилия,имя,ссылка,тематика,количество подписчиков,средний просмотры публикации,стоимость за видеопост (60 сек.)
4,Нижний Новгород,федулова,елизавета,https://likee.video/@Lizzka_Strizzzz,"Лайфстайл, контент-мейкер, отношения",8 500 000,15 000,"р.13 500,00"
5,Москва,гогунская,милана,https://likee.video/@milana_star_offi,"Лайфстайл, развлечения, юмор, артист, музыка, ...",4 200 000,250 000,"р.200 000,00"
6,Уфа,мазитова,регина,https://l.likee.video/p/NWuFzx,"Неформалка, Юмор",3 400 000,50 000,"р.49 500,00"
7,Уфа,кашаева,динара,https://mobile.likeevideo.ru/p/jDTQ9q,"Лайфстайл, бьюти, POV",2 600 000,15 000,"р.13 500,00"
8,Тверь,кузьмина,варвара,https://l.likee.video/p/fSuROB,"Лайфстайл,юмор",1 100 000,120 000,"р.108 000,00"
9,Ярославль,селянкина,оксана,https://mobile.likeevideo.ru/p/du7PBO,"Семья, дети, фудблогер, POV, лайфстайл",968 000,80 000,"р.120 000,00"
10,Москва,розанова,екатерина,https://mobile.likeevideo.ru/p/u104SV,"Лайфстайл, бьюти, POV",923 000,80 000,"р.72 000,00"
11,Москва,мирасова,наталья,https://l.likee.video/p/nolcRs,"POV, юмор, развлечения",900 000,100 000,"р.90 000,00"
12,Смоленск,бондарев,алексей,https://l.likee.video/p/KgvilX,"Юмор, развлечения, лайфхаки",742 000,28 000,"р.25 200,00"
13,Барнаул,горбунова,надежда,https://l.likee.video/p/GXYCCU,"Лайфстайл, Сериалы",570 300,350 000,"р.228 000,00"


In [347]:
insight_likee = name_familia(insight_likee)

In [348]:
[col for col in bufer.columns if 'TikTok' in col]

['TikTok блогер',
 'TikTok количество подписчиков',
 'TikTok среднее кол-во просмотров',
 'TikTok стоимость видеопост',
 'TikTok ссылка']

In [349]:
[col for col in insight_likee.columns]

['город',
 'ссылка',
 'тематика',
 'количество подписчиков',
 'средний просмотры публикации',
 'стоимость за видеопост (60 сек.)',
 'блогер']

In [350]:
insight_likee.rename(columns = {'средний просмотры публикации':'среднее кол-во просмотров',
                                'стоимость за видеопост (60 сек.)':'стоимость видеопост'}, inplace = True)

#### Обрежем DataFrame по высоте

In [351]:
insight_likee = trim_df(insight_likee, 'количество подписчиков', 27000)

In [352]:
insight_likee.tail()

3,город,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость видеопост,блогер
39,Великий Новгород,https://l.likee.video/p/8Gth4,POV,38800.0,15 000,"р.13 500,00",юрахно евгений
40,Саранск,https://l.likee.video/p/WDUdAG,"Лайфстайл,юмор",35000.0,20 000,"р.18 000,00",валеев артур
41,Тула,https://l.likee.video/p/rwpo7V,"POV, юмор, развлечения",34100.0,100 000,"р.90 000,00",шакиров эмиль
42,Махачкала,https://l.likee.video/p/svYssZ,"Развлечение,ПОВ",27400.0,10 000,"р.9 000,00",мирзаев сулейман
43,Тверь,https://l.likee.video/p/HVfP6y,"Лайфстайл, развлечения, юмор",27000.0,50 000,"р.45 000,00",самодуров андрей


#### Проверка на дубликаты

In [353]:
show_duplicates(insight_likee)

Количество дубликатов: 0


3,город,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость видеопост,блогер


### Девятнадцатое объединение

In [354]:
bufer = merge_bufer_df(bufer, insight_likee, 'Likee')

Количество дубликатов: 0


In [355]:
# Посмотрим на размер DataFrame

bufer.shape

(567, 164)

### Подготовка таблицы insight_yappy

In [356]:
insight_yappy

3,город,фамилия,имя,ссылка на профиль в yappy,тематика,количество подписчиков,среднее кол-во просмотров,стоимость за видеопост (60 сек.)
4,Москва,лазарьянц,карина,https://yappy.media/profile/25517079000644b88e...,"Развлечения, юмор, лайфстайл",308 600,5 000,"р.935 000,00"
5,Москва,хилькевич,анна,https://yappy.media/profile/eedaea1dc6f948ebb0...,"Лайфстайл, мама, актриса",243 300,60 000,"р.510 000,00"
6,Ростов-на-Дону,кобзарева,лаура,https://yappy.media/s/p_6nnXi2kGciIS2awRqfvKuY,"Танцы, развлечения, лайфстайл",239 600,10 000,"р.12 500,00"
7,Москва,акилова,алина,https://yappy.media/profile/70134d60962b485c90...,"Лайфстайл, семья, бьюти",123 500,11 000,"р.99 000,00"
8,Самара,графинина,елизавета,https://yappy.media/s/p_1UqVXvm0yaJqruortHOtba,"Животные, лайфстайл",82 400,8 000,"р.10 000,00"
...,...,...,...,...,...,...,...,...
294,Чебоксары,харитонова,ксения,https://yappy.media/s/p_33xK3d9Y4ITJWXSJDDq1cT,Лайфстайл,0,15,под запрос
295,Сочи,воронина,екатерина,https://yappy.media/s/p_3JxvBSU4dGwB6MhVT8slVd,Лайфстайл,2,30,под запрос
296,Астрахань,лукьянова,тамара,https://yappy.media/profile/36e92ceff2614c53ab...,Лайфстайл,1,3,под запрос
297,Владивосток,учанина,елизавета,https://yappy.media/profile/9ea1a2c3-1038-401e...,Лайфстайл,1,17,под запрос


In [357]:
insight_yappy = name_familia(insight_yappy)

In [358]:
[col for col in bufer.columns if 'TikTok' in col]

['TikTok блогер',
 'TikTok количество подписчиков',
 'TikTok среднее кол-во просмотров',
 'TikTok стоимость видеопост',
 'TikTok ссылка']

In [359]:
[col for col in insight_yappy.columns]

['город',
 'ссылка на профиль в yappy',
 'тематика',
 'количество подписчиков',
 'среднее кол-во просмотров',
 'стоимость за видеопост (60 сек.)',
 'блогер']

In [360]:
insight_yappy.rename(columns = {'ссылка на профиль в yappy':'ссылка',
                                'стоимость за видеопост (60 сек.)':'стоимость видеопост',
                               }, inplace = True)

#### Обрежем DataFrame по высоте

In [361]:
insight_yappy = trim_df(insight_yappy, 'количество подписчиков', 3000)

In [362]:
insight_yappy.tail()

3,город,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость видеопост,блогер
74,Брянск,https://yappy.media/s/p_55liVoClPxs7XJBQo2JO8c,"Лайфстайл, юмор, отношения",4400.0,10 000,"р.12 500,00",абрамочкина дарья
75,Омск,https://yappy.media/profile/2284c0fbef8345deba...,"Юмор, развлечения, POV",4200.0,10 000,"р.12 500,00",сурыкин сергей
76,Москва,https://yappy.media/profile/d888416bb4404b01b6...,"Лайфстайл, POV, юмор, развлечения",4200.0,6 000,"р.7 500,00",габай янина
77,Владимир,https://yappy.media/profile/1abc9f5e7d4c444987...,"Лайфстайл, бьюти, юмор",4000.0,5 000,"р.6 250,00",бебчук екатерина
78,Уфа,https://yappy.media/profile/539f87e6fa6c4afda4...,Астролог,3000.0,5 000,"р.6 250,00",ахунова гульназ


#### Проверка на дубликаты

In [363]:
show_duplicates(insight_yappy)

Количество дубликатов: 0


3,город,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость видеопост,блогер


### Двадцатое объединение

In [364]:
bufer = merge_bufer_df(bufer, insight_yappy, 'Yappy')

Количество дубликатов: 0


### Подготовка таблицы insight_rutube

> У нас есть проблема с именем блогера: оно сделано как **ФИО**, а нам нужно **имя и фамилия**, чтобы правильно объединить по строкам - сделаем преобразования, чтобы подвести к правильному формату

In [365]:
insight_rutube.head()

3,город,фио,ссылка,количество подписчиков,средние просмотры за месяц,стоимость преролла,стоимость интеграции (до 1 минуты)
4,Тверь,аркатов станислав александрович,https://rutube.ru/channel/23684374/,Фудблогер,118 300,25 030,"р.50 060,00"
5,Краснодар,солодов юрий юрьевич,https://rutube.ru/channel/23722477/,"Лайфстайл, влог",24 500,47 000,"р.82 250,00"
6,Казань,ахмадуллин наиль радикович,https://rutube.ru/channel/23694775/,"Путешествия, лайфстайл",18 500,27 524,под запрос
7,Воронеж,рязанова татьяна алексеевна,https://rutube.ru/channel/19363597/,"Бьюти, лайфстайл",16 500,50 066,"р.87 615,00"
8,Воронеж,лаврова валерия владимировна,https://rutube.ru/channel/26070201/,Художник,15 600,40 063,"р.70 115,00"


In [366]:
def manipulate_text(text: str) -> str:
    words = text.split()  # Разделение строки на слова
    if len(words) >= 3:
        words.pop(2)  # Удаление третьего слова
    return ' '.join(words)  # Объединение слов обратно в строку

In [367]:
# def manipulate_text(text: str) -> str:
    
#     """
#     Выполняет редактирование входного текста, удаляя третье слово (если оно существует)
#     и меняя местами первое и второе слово (если в тексте есть как минимум два слова).


#     ПАРАМЕТРЫ:
#         text (str) - Входной текст, который требуется преобразовать.


#     ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
#         str: Преобразованный текст с указанными изменениями.


#     ПРИМЕР ИСПОЛЬЗОВАНИЯ:
#     insight_rutube['фио'] = insight_rutube['фио'].apply(manipulate_text).str.strip()
    
#     """
    
#     words = text.split()  # Разделение строки на слова
    
#     if len(words) >= 3:
#         words.pop(2)  # Удаление третьего слова
    
#     if len(words) >= 2:
#         words[0], words[1] = words[1], words[0]  # Меняем местами первое и второе слово
    
#     return ' '.join(words)  # Объединение слов обратно в строку

In [368]:
insight_rutube['фио'] = insight_rutube['фио'].apply(manipulate_text).str.strip()

In [369]:
[col for col in insight_rutube.columns]

['город',
 'фио',
 'ссылка',
 'количество подписчиков',
 'средние просмотры за месяц',
 'стоимость преролла',
 'стоимость интеграции (до 1 минуты)']

In [370]:
[col for col in insight_youtube.columns]

['YouTube город',
 'YouTube название канала',
 'YouTube блогер',
 'YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость за shorts до 1 мин',
 'YouTube стоимость выпуск под клиента']

In [371]:
insight_rutube.rename(columns = {'фио':'блогер',
                                 'средние просмотры за месяц':'среднее кол-во просмотров видео',
                                 'стоимость преролла':'стоимость преролл до 30 сек',
                                 'стоимость интеграции (до 1 минуты)':'стоимость интеграция 1 слот',
                                 'количество подписчиков':'тематика'}, inplace = True)

#### Обрежем DataFrame по высоте

In [372]:
insight_rutube

3,город,блогер,ссылка,тематика,среднее кол-во просмотров видео,стоимость преролл до 30 сек,стоимость интеграция 1 слот
4,Тверь,аркатов станислав,https://rutube.ru/channel/23684374/,Фудблогер,118 300,25 030,"р.50 060,00"
5,Краснодар,солодов юрий,https://rutube.ru/channel/23722477/,"Лайфстайл, влог",24 500,47 000,"р.82 250,00"
6,Казань,ахмадуллин наиль,https://rutube.ru/channel/23694775/,"Путешествия, лайфстайл",18 500,27 524,под запрос
7,Воронеж,рязанова татьяна,https://rutube.ru/channel/19363597/,"Бьюти, лайфстайл",16 500,50 066,"р.87 615,00"
8,Воронеж,лаврова валерия,https://rutube.ru/channel/26070201/,Художник,15 600,40 063,"р.70 115,00"
...,...,...,...,...,...,...,...
217,Уфа,субханкулов артем,https://rutube.ru/channel/16976000/,1,0,NaN,NaN
218,Сочи,сомкин демьян,https://rutube.ru/channel/23665327/,1,0,NaN,NaN
219,Самара,боярцева анна,https://rutube.ru/channel/23786070/,1,0,NaN,NaN
220,Тверь,фрикбард алексей,https://rutube.ru/channel/25668794/,1,0,NaN,NaN


In [373]:
insight_rutube = trim_df(insight_rutube, 'среднее кол-во просмотров видео', 3299)
insight_rutube.tail()

3,город,блогер,ссылка,тематика,среднее кол-во просмотров видео,стоимость преролл до 30 сек,стоимость интеграция 1 слот
96,Москва,лавренюк евгений,https://rutube.ru/channel/23662042/,5 348,41352.0,NaN,NaN
97,Москва,гуслистов михаил,https://rutube.ru/channel/23685098/,5 346,43213.0,NaN,NaN
98,Краснодар,колодник дарья,https://rutube.ru/channel/23706945/,5 327,30000.0,NaN,NaN
99,Владимир,шуклецова анастасия,https://rutube.ru/channel/9570650/,5 199,65385.0,NaN,NaN
116,Тверь,садикова заира,https://rutube.ru/channel/23712253/,22,4000.0,NaN,NaN


In [374]:
insight_rutube = insight_rutube.loc[0: 70]

In [375]:
insight_rutube.tail(15)

3,город,блогер,ссылка,тематика,среднее кол-во просмотров видео,стоимость преролл до 30 сек,стоимость интеграция 1 слот
56,Воронеж,бабарин алексей,https://rutube.ru/channel/23705479/,"Лайфстайл, влог",4800.0,2 563,"р.4 490,00"
57,Москва,заричная анастасия,https://rutube.ru/channel/7456516/,Лайфстайл,4600.0,7 890,под запрос
58,Краснодар,бойко мария,https://rutube.ru/channel/23724373/,"Лайфстайл, путешествия",4600.0,41 000,"р.71 750,00"
59,Кемерово,наумов данил,https://rutube.ru/channel/23663217/,Врач,4300.0,10 988,"р.19 233,00"
60,Ярославль,якушевич вячеслав,https://rutube.ru/channel/23467085/,"Лайфстайл, фудблогер",4265.0,38 444,"р.67 280,00"
61,Ростов-на-Дону,курдус дмитрий,https://rutube.ru/channel/23684825/,Геймер,4200.0,2 440,"р.4 270,00"
62,Волгоград,хамидова мария,https://rutube.ru/channel/6882983/,Фудблогер,3900.0,37 829,"р.87 500,00"
63,Воронеж,омаров гаджимурад,https://rutube.ru/channel/12999482/,"Лайфстайл, влог",3800.0,10 022,"р.17 535,00"
64,Волгоград,трубчанинова раиса,https://rutube.ru/channel/23706142/,"Лайфстайл, семья",3700.0,15 095,под запрос
65,Пенза,захватова людмила,https://rutube.ru/channel/23650043/,Спорт,3600.0,1 942,"р.3 395,00"


#### Проверка на дубликаты

In [376]:
show_duplicates(insight_rutube)

Количество дубликатов: 2


3,город,блогер,ссылка,тематика,среднее кол-во просмотров видео,стоимость преролл до 30 сек,стоимость интеграция 1 слот
52,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,Фудблогер,5200.0,7 389,под запрос
70,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,6 278,14235.0,под запрос,под запрос


### Двадцать первое объединение

In [377]:
bufer = merge_bufer_df(bufer, insight_rutube, 'RuTube')

Количество дубликатов: 2


### Подготовка таблицы insight_ok

In [378]:
# insight_ok.head()

In [379]:
# insight_ok = name_familia(insight_ok)

In [380]:
# [col for col in bufer.columns if 'OK' in col]

In [381]:
# [col for col in insight_ok.columns]

#### Обрежем DataFrame по высоте

In [382]:
# insight_ok = trim_df(insight_ok, 'количество подписчиков', 53)

#### Проверка на дубликаты

In [383]:
# show_duplicates(insight_ok)

### Двадцать второе объединение

In [384]:
# bufer = merge_bufer_df(bufer, insight_ok, 'Одноклассники')

> Запустим функцию duplicates_stroka, чтобы обновить столбец bufer

In [385]:
# col_to_check = 'блогер'
# columns_to_upd = 'тематика'
# bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

> Посмотрим на размер DataFrame bufer на данный момент

In [386]:
# bufer.shape

> Заполним столбец контакты и налог

In [387]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('info@insightpeople.ru', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

## Подготовка данных по блогерам от NextUp

In [388]:
nextup_df = [nextup_instagram,
             nextup_telegram,
             nextup_youtube,
             nextup_twitch,
             nextup_vk]

In [389]:
key_word = ['CPV', ]  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Имя блогера']

In [390]:
for df in nextup_df:
    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы nextup_instagram

In [391]:
nextup_instagram

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,тематика,ца по городам,ца по возрасту,ца по полу,охват фотопостов за 30 дней,охват reels за 30 дней,охват stories за 14 дней,стоимость фотопост,стоимость reels,стоимость серия stories (3 шт. по 15 сек.),статистика ссылка
7,,Э,алла брулетова,https://instagram.com/bruletova_,1 960 000,статистика,"Фешн, Лайфстайл, Бьюти","Москва – 4,5%\nСПб – 1,2%","13–17 лет – 4,4%\n18–24 года – 29,7%\n25–34 го...","Мужской – 77,3%\nЖенский – 22,6%",750 000,1 200 000,110 000,430 000 ₽,500 000 ₽,150 000 ₽,https://disk.yandex.ru/d/Os8m8Vm5zfygPg
8,NaN,П,настя badbaarbie,https://instagram.com/badbaarbie,1 850 000,статистика,"Фешн, Лайфстайл, Бьюти","Москва – 8,2%\nСПб – 2,6%","13–17 лет – 19,5%\n18–24 года – 49,5%\n25–34 г...","Мужской – 31,4%\nЖенский – 68,5%",200 000,300 000,75 000,360 000 ₽,286 000 ₽,145 000 ₽,https://disk.yandex.ru/d/EDbrK6por7PaNA
9,NaN,Э,эвелина лунаком,https://instagram.com/lyna_com2000,1 700 000,статистика,"Лайфстайл, Бьюти","Москва – 16,9%\nСПб – 5%","13–17 лет – 14,6%\n18–24 года – 62,2%\n25–34 г...","Мужской – 24,4%\nЖенский – 75,5%",500 000,1 000 000,250 000,572 000 ₽,930 000 ₽,500 000 ₽,https://disk.yandex.ru/d/Ex59HV0Fg9zFjg
10,NaN,П,владос мирос,https://www.instagram.com/vladosmiros/,1 670 000,статистика,"Бьюти, Юмор, Лайфстайл","Москва – 16,6% СПб – 7,3%","13-17 лет - 15% \n18-24 года - 60,5% \n25-34 г...",Мужской – 9% Женский – 91%,150 000,400 000,180 000,300 000 ₽,400 000 ₽,100 000 ₽,https://disk.yandex.ru/d/aBHRwykqUtllIQ
11,NaN,П,саша трусова,https://www.instagram.com/avtrusova,1 250 000,статистика,"Спорт, Бьюти","Москва – 9%\nСПб – 3,4%","13–17 лет – 14,3%\n18–24 года – 39,2%\n25–34 г...","Мужской – 23%\nЖенский – 76,9%",650 000,400 000,130 000,430 000 ₽,572 000 ₽,358 000 ₽,https://drive.google.com/drive/folders/1BdjD3x...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,NaN,П,андрей (mister kokoshka),https://instagram.com/misterkokoshka,31 200,статистика,"Лайфстайл, Юмор","Москва - 22,4% СПб - 8,8%","13-17 лет - 16,5% 18-24 года - 58,7% 25-34 год...","Мужской - 26,9% Женский -73%",20 000,300 000,15 000,75 000 ₽,150 000 ₽,36 000 ₽,https://drive.google.com/drive/u/0/folders/1Dp...
65,NaN,П,арина lumitalle,https://www.instagram.com/lumitalle/,27 800,статистика,"Лайфстайл, Бьюти","Москва - 7,6%\nСПб - 3,6%","13–17 лет – 14% 18–24 года – 57,5% 25–34 года...","Мужской – 39,5% Женский – 60,4%",30 000,–,4 500,45 000 ₽,–,10 000 ₽,https://disk.yandex.ru/d/9kpXqT0ruvP1XA
66,NaN,П,павел довольнов,https://www.instagram.com/pashadizel,22 000,статистика,"Лайфстайл, гейминг","Москва - 25,3%\nСПб - 9,6%",13–17 лет – 1% 18–24 года – 50% 25–34 года – ...,Мужской – 89% Женский – 11%,7 000,NaN,3 500,25 000 ₽,–,15 000 ₽,NaN
67,NaN,Э,марина ясно,https://www.instagram.com/yyyyasno/,19 600,статистика,"Лайфстайл, Бьюти","Москва – 14,1% СПб – 6,3%","13–17 лет – 17,2% 18–24 года – 59,1% 25–34 го...","Мужской – 62,4% Женский – 37,5%",7 000,–,6 000,20 000 ₽,27 000 ₽,6 500 ₽,NaN


In [392]:
def nextup_col_upd(df: pd.DataFrame, nextup_columns_upd: list) -> pd.DataFrame:
    
    """
    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - .
    nextup_columns_upd (list) - .
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - DataFrame с обновлёнными столбцами nextup_columns_upd.
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    """
    

#     for col in df.columns:
#         df[nextup_columns_upd] = df[nextup_columns_upd].apply(lambda x: str(x).lower())\
#                                                         .str.replace('/\n', ', ')\
#                                                         .str.replace('\n', ', ')\
#                                                         .str.replace('*', '')\
#                                                         .str.strip()
        
    for col in nextup_columns_upd:
        df[col] = df[col].apply(lambda x: str(x).lower())\
                        .str.replace('\n', ', ')\
                        .str.replace('*', '')\
                        .str.strip()
        
    return df

In [393]:
nextup_instagram_columns_upd = ['тематика', 'ца по городам', 'ца по возрасту', 'ца по полу']
nextup_instagram = nextup_col_upd(nextup_instagram, nextup_instagram_columns_upd)

In [394]:
nextup_instagram.head()

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,тематика,ца по городам,ца по возрасту,ца по полу,охват фотопостов за 30 дней,охват reels за 30 дней,охват stories за 14 дней,стоимость фотопост,стоимость reels,стоимость серия stories (3 шт. по 15 сек.),статистика ссылка
7,,Э,алла брулетова,https://instagram.com/bruletova_,1 960 000,статистика,"фешн, лайфстайл, бьюти","москва – 4,5%, спб – 1,2%","13–17 лет – 4,4%, 18–24 года – 29,7%, 25–34 го...","мужской – 77,3%, женский – 22,6%",750 000,1 200 000,110 000,430 000 ₽,500 000 ₽,150 000 ₽,https://disk.yandex.ru/d/Os8m8Vm5zfygPg
8,NaN,П,настя badbaarbie,https://instagram.com/badbaarbie,1 850 000,статистика,"фешн, лайфстайл, бьюти","москва – 8,2%, спб – 2,6%","13–17 лет – 19,5%, 18–24 года – 49,5%, 25–34 г...","мужской – 31,4%, женский – 68,5%",200 000,300 000,75 000,360 000 ₽,286 000 ₽,145 000 ₽,https://disk.yandex.ru/d/EDbrK6por7PaNA
9,NaN,Э,эвелина лунаком,https://instagram.com/lyna_com2000,1 700 000,статистика,"лайфстайл, бьюти","москва – 16,9%, спб – 5%","13–17 лет – 14,6%, 18–24 года – 62,2%, 25–34 г...","мужской – 24,4%, женский – 75,5%",500 000,1 000 000,250 000,572 000 ₽,930 000 ₽,500 000 ₽,https://disk.yandex.ru/d/Ex59HV0Fg9zFjg
10,NaN,П,владос мирос,https://www.instagram.com/vladosmiros/,1 670 000,статистика,"бьюти, юмор, лайфстайл","москва – 16,6% спб – 7,3%","13-17 лет - 15% , 18-24 года - 60,5% , 25-34 г...",мужской – 9% женский – 91%,150 000,400 000,180 000,300 000 ₽,400 000 ₽,100 000 ₽,https://disk.yandex.ru/d/aBHRwykqUtllIQ
11,NaN,П,саша трусова,https://www.instagram.com/avtrusova,1 250 000,статистика,"спорт, бьюти","москва – 9%, спб – 3,4%","13–17 лет – 14,3%, 18–24 года – 39,2%, 25–34 г...","мужской – 23%, женский – 76,9%",650 000,400 000,130 000,430 000 ₽,572 000 ₽,358 000 ₽,https://drive.google.com/drive/folders/1BdjD3x...


> Удалим первые два столбца, которые не несут в себе полезной информации, а так как нам это надо будет делать ещё у нескольких DataFrame, то сделаем функцию

In [395]:
def nextup_columns_to_drop(df: pd.DataFrame, columns_to_drop: list, statistika: bool) -> pd.DataFrame:
    
    """
    Функция удаляет определённые столбцы по индексу, которые мы передаём в листе columns_to_drop.
    Также у нас есть флаг statistika, и если стоит True, то также удаляется столбец "статистика"
    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - DataFrame, который мы редактируем.
    columns_to_drop: (list) - список с индексами столбцов для удаления.
    statistika (bool) - флаг для удаления столбца "Статистика".
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - обновлённый DataFrame.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    columns_to_drop = [0, 1]
    nextup_instagram = nextup_columns_to_drop(nextup_instagram, columns_to_drop, True)
    
    
    """
    df = df.drop(df.columns[columns_to_drop], axis=1)
    
    if statistika:
        df.drop(columns=['статистика'], axis=1, inplace=True)
    
    return df  # Вы должны вернуть измененный DataFrame

In [396]:
columns_to_drop = [0, 1]
nextup_instagram = nextup_columns_to_drop(nextup_instagram, columns_to_drop, True)

In [397]:
nextup_instagram

6,имя блогера,ссылка,кол-во подписчиков,тематика,ца по городам,ца по возрасту,ца по полу,охват фотопостов за 30 дней,охват reels за 30 дней,охват stories за 14 дней,стоимость фотопост,стоимость reels,стоимость серия stories (3 шт. по 15 сек.),статистика ссылка
7,алла брулетова,https://instagram.com/bruletova_,1 960 000,"фешн, лайфстайл, бьюти","москва – 4,5%, спб – 1,2%","13–17 лет – 4,4%, 18–24 года – 29,7%, 25–34 го...","мужской – 77,3%, женский – 22,6%",750 000,1 200 000,110 000,430 000 ₽,500 000 ₽,150 000 ₽,https://disk.yandex.ru/d/Os8m8Vm5zfygPg
8,настя badbaarbie,https://instagram.com/badbaarbie,1 850 000,"фешн, лайфстайл, бьюти","москва – 8,2%, спб – 2,6%","13–17 лет – 19,5%, 18–24 года – 49,5%, 25–34 г...","мужской – 31,4%, женский – 68,5%",200 000,300 000,75 000,360 000 ₽,286 000 ₽,145 000 ₽,https://disk.yandex.ru/d/EDbrK6por7PaNA
9,эвелина лунаком,https://instagram.com/lyna_com2000,1 700 000,"лайфстайл, бьюти","москва – 16,9%, спб – 5%","13–17 лет – 14,6%, 18–24 года – 62,2%, 25–34 г...","мужской – 24,4%, женский – 75,5%",500 000,1 000 000,250 000,572 000 ₽,930 000 ₽,500 000 ₽,https://disk.yandex.ru/d/Ex59HV0Fg9zFjg
10,владос мирос,https://www.instagram.com/vladosmiros/,1 670 000,"бьюти, юмор, лайфстайл","москва – 16,6% спб – 7,3%","13-17 лет - 15% , 18-24 года - 60,5% , 25-34 г...",мужской – 9% женский – 91%,150 000,400 000,180 000,300 000 ₽,400 000 ₽,100 000 ₽,https://disk.yandex.ru/d/aBHRwykqUtllIQ
11,саша трусова,https://www.instagram.com/avtrusova,1 250 000,"спорт, бьюти","москва – 9%, спб – 3,4%","13–17 лет – 14,3%, 18–24 года – 39,2%, 25–34 г...","мужской – 23%, женский – 76,9%",650 000,400 000,130 000,430 000 ₽,572 000 ₽,358 000 ₽,https://drive.google.com/drive/folders/1BdjD3x...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,андрей (mister kokoshka),https://instagram.com/misterkokoshka,31 200,"лайфстайл, юмор","москва - 22,4% спб - 8,8%","13-17 лет - 16,5% 18-24 года - 58,7% 25-34 год...","мужской - 26,9% женский -73%",20 000,300 000,15 000,75 000 ₽,150 000 ₽,36 000 ₽,https://drive.google.com/drive/u/0/folders/1Dp...
65,арина lumitalle,https://www.instagram.com/lumitalle/,27 800,"лайфстайл, бьюти","москва - 7,6%, спб - 3,6%","13–17 лет – 14% 18–24 года – 57,5% 25–34 года...","мужской – 39,5% женский – 60,4%",30 000,–,4 500,45 000 ₽,–,10 000 ₽,https://disk.yandex.ru/d/9kpXqT0ruvP1XA
66,павел довольнов,https://www.instagram.com/pashadizel,22 000,"лайфстайл, гейминг","москва - 25,3%, спб - 9,6%",13–17 лет – 1% 18–24 года – 50% 25–34 года – 44%,мужской – 89% женский – 11%,7 000,NaN,3 500,25 000 ₽,–,15 000 ₽,NaN
67,марина ясно,https://www.instagram.com/yyyyasno/,19 600,"лайфстайл, бьюти","москва – 14,1% спб – 6,3%","13–17 лет – 17,2% 18–24 года – 59,1% 25–34 го...","мужской – 62,4% женский – 37,5%",7 000,–,6 000,20 000 ₽,27 000 ₽,6 500 ₽,NaN


In [398]:
[col for col in nextup_instagram.columns]

['имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'тематика',
 'ца по городам',
 'ца по возрасту',
 'ца по полу',
 'охват фотопостов за 30 дней',
 'охват reels за 30 дней',
 'охват stories за 14 дней',
 'стоимость фотопост',
 'стоимость reels',
 'стоимость серия stories (3 шт. по 15 сек.)',
 'статистика ссылка']

In [399]:
[col for col in bufer.columns if 'Instagram' in col]

['Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика',
 'Instagram город',
 'Instagram среднее кол-во просмотров серии сторис',
 'Instagram стоимость серия сторис до 60 сек']

In [400]:
nextup_instagram.rename(columns = {'имя блогера':'блогер',
                                   'кол-во подписчиков':'количество подписчиков',
                                   'охват фотопостов за 30 дней':'среднее кол-во просмотров фотопоста',
                                   'охват reels за 30 дней':'среднее кол-во просмотров reels / видеопоста',
                                   'охват stories за 14 дней':'среднее кол-во просмотров 1 сторис',
                                   'стоимость reels':'стоимость reels / видеопост',
                                   'стоимость серия stories (3 шт. по 15 сек.)':'стоимость серия из 3х сторис до 45 сек',
                                   'статистика ссылка':'статистика'
                                  }, inplace = True)

#### Проверка на дубликаты

In [401]:
show_duplicates(nextup_instagram)

Количество дубликатов: 0


6,блогер,ссылка,количество подписчиков,тематика,ца по городам,ца по возрасту,ца по полу,среднее кол-во просмотров фотопоста,среднее кол-во просмотров reels / видеопоста,среднее кол-во просмотров 1 сторис,стоимость фотопост,стоимость reels / видеопост,стоимость серия из 3х сторис до 45 сек,статистика


### Двадцать третье объединение

In [402]:
bufer = merge_bufer_df(bufer, nextup_instagram, 'Instagram')

Количество дубликатов: 4


In [403]:
show_duplicates(bufer).head(6)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
436,лиза барашик,"бьюти, влоги, лайфстайл",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл",373 000,100 000,55 000,100 000,20 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
691,лиза барашик,"лайфстайл, бьюти, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"лайфстайл, бьюти, путешествия",376 000,100 000,55 000,200 000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
347,ознобишин дмитрий,"рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,Фудблогер,5200.0,7 389,под запрос,NaN,NaN,NaN
348,ознобишин дмитрий,"рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,6 278,14235.0,под запрос,под запрос,NaN,NaN,NaN


> Обновим столбец "статистика"

In [404]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [405]:
show_duplicates(bufer).head(6)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
436,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
691,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",376 000,100 000,55 000,200 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
347,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",5200.0,7 389,под запрос,None,None,None
348,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",14235.0,под запрос,под запрос,None,None,None


> Лиза Барашик на эксклюзиве у агенства DIDENOK, поэтому мы должны оставить именно строчку с ними, то есть первую

In [406]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
duplicates_bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
347,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",5200.0,7 389,под запрос,None,None,None
348,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",14235.0,под запрос,под запрос,None,None,None
436,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
691,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",376 000,100 000,55 000,200 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"


In [407]:
max_index = duplicates_bufer.index.max()
bufer.drop(max_index, inplace=True)

In [408]:
bufer[bufer['блогер'] == 'лиза барашик']

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
436,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"


### Подготовка таблицы nextup_telegram

In [409]:
nextup_telegram

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,охват поста за 30 дней,стоимость текстовый пост (24 часа),стоимость фотопост (24 часа),стоимость видеокружок (24 часа),статистика ссылка
7,NaN,П,илья exile,https://t.me/exilemusic13,744 113,статистика,600 000,500 000 ₽,600 000 ₽,500 000 ₽,https://disk.yandex.ru/d/ajGlE9Fx7SrxGw
8,NaN,П,саша парадеевич,https://t.me/paradeevich,554 269,статистика,400 000,1 200 000 ₽,1 218 000 ₽,1 200 000 ₽,https://disk.yandex.ru/d/FiihzOmrftH8WQ
9,NaN,П,леша кореш,https://t.me/koreshzy,500 842,статистика,230 000,360 000 ₽,400 000 ₽,360 000 ₽,https://disk.yandex.ru/d/eJbfo3TwOAKkHw
10,NaN,П,костя плохой парень,https://t.me/plohoypacan,402 901,статистика,205 000,220 000 ₽,—,170 000 ₽,https://disk.yandex.ru/d/dIhdb9_BG6ldmw
11,NaN,П,саша frame tamer,https://t.me/oieiei,344 359,статистика,141 000,260 000 ₽,300 000 ₽,260 000 ₽,https://disk.yandex.ru/d/ug17XSRvF-uynw
12,NaN,П,данила горилла,https://t.me/danila_gorilla,245 415,статистика,127 000,200 000 ₽,230 000 ₽,178 500 ₽,https://disk.yandex.ru/d/V4S_i8fk31Ss6g
13,NaN,П,саша трусова,https://t.me/avtrusovaofficial,214 219,статистика,190 000,386 000 ₽,429 000 ₽,572 000 ₽,https://disk.yandex.ru/d/5AuO4P6oP0vxSQ
14,NaN,П,эвелина лунаком,https://t.me/lynacom200,176 306,статистика,108 000,165 000 ₽,186 000 ₽,143 000 ₽,https://disk.yandex.ru/d/Ex59HV0Fg9zFjg
15,NaN,Э,аня акулич,https://t.me/akyulenokk,184 181,статистика,102 000,213 000 ₽,245 000 ₽,213 000 ₽,https://disk.yandex.ru/d/AVUsAQCIrgrjbw
16,NaN,П,факел (рустик),https://t.me/syetarustamafackela,161 456,статистика,43 000,45 000 ₽,50 000 ₽,35 000 ₽,https://disk.yandex.ru/d/TmAFXNsxwZPzYQ


In [410]:
nextup_telegram = nextup_telegram.dropna(subset=['имя блогера'])

In [411]:
nextup_telegram.tail()

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,охват поста за 30 дней,стоимость текстовый пост (24 часа),стоимость фотопост (24 часа),стоимость видеокружок (24 часа),статистика ссылка
57,NaN,Э,даша (polovinkago),https://t.me/polovinkago,8 459,статистика,3 000,7 000 ₽,10 000 ₽,6 000 ₽,https://disk.yandex.ru/d/Ugl6qcfYDvCGFA
58,NaN,П,саша маслова,https://t.me/sashamaslova,6 812,статистика,3 000,15 000 ₽,19 000 ₽,15 000 ₽,https://disk.yandex.ru/d/cMifIiN2pWcSAQ
59,NaN,П,катя бомонд,https://t.me/keyt_tut,7 355,статистика,4 000,6 000 ₽,7 500 ₽,7 500 ₽,https://drive.google.com/drive/u/0/folders/1Wt...
60,NaN,Э,марина ясно,https://t.me/yyyyyasno,4 867,по запросу,5 000,7 000 ₽,10 000 ₽,6 000 ₽,NaN
61,NaN,П,даша (lovesomemama),https://t.me/lovesomechannel,2 747,статистика,1 500,10 000 ₽,12 000 ₽,17 000 ₽,https://disk.yandex.ru/d/MOEcnIl-sKYc2w


In [412]:
columns_to_drop = [0, 1]
nextup_telegram = nextup_columns_to_drop(nextup_telegram, columns_to_drop, True)

In [413]:
[col for col in nextup_telegram.columns]

['имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'охват поста за 30 дней',
 'стоимость текстовый пост (24 часа)',
 'стоимость фотопост (24 часа)',
 'стоимость видеокружок (24 часа)',
 'статистика ссылка']

In [414]:
[col for col in bufer.columns if 'Telegram' in col]

['Telegram блогер',
 'Telegram ссылка',
 'Telegram тематика',
 'Telegram количество подписчиков',
 'Telegram среднее кол-во просмотров поста',
 'Telegram средняя вовлечённость',
 'Telegram стоимость фотопост',
 'Telegram стоимость видеопост',
 'Telegram стоимость кружочек до 60 сек',
 'Telegram стоимость текстовое сообщение без фото',
 'Telegram статистика',
 'Telegram город']

In [415]:
nextup_telegram.rename(columns = {'имя блогера':'блогер',
                                  'кол-во подписчиков':'количество подписчиков',
                                  'охват поста за 30 дней':'среднее кол-во просмотров поста',
                                  'стоимость текстовый пост (24 часа)':'стоимость текстовое сообщение без фото',
                                  'стоимость фотопост (24 часа)':'стоимость фотопост',
                                  'стоимость видеокружок (24 часа)':'стоимость кружочек до 60 сек',
                                  'статистика ссылка':'статистика'
                                 }, inplace = True)

#### Проверка на дубликаты

In [416]:
show_duplicates(nextup_telegram)

Количество дубликатов: 0


6,блогер,ссылка,количество подписчиков,среднее кол-во просмотров поста,стоимость текстовое сообщение без фото,стоимость фотопост,стоимость кружочек до 60 сек,статистика


### Двадцать четвёртое объединение

In [417]:
bufer = merge_bufer_df(bufer, nextup_telegram, 'Telegram')

Количество дубликатов: 88


In [418]:
show_duplicates(bufer).head(6)

Количество дубликатов: 88


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
754,алина олешева,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,алина олешева,"лайфстайл, музыка",алина олешева,https://www.instagram.com/oleshev,"лайфстайл, музыка",625 000,300 000,110 000,250 000,None,...,None,None,None,"лайфстайл, музыка",None,None,None,"москва - 14,6% спб - 6,6%","13-17 лет - 10,3% 18-24 года - 60,6% 25-34 год...","мужской - 34,5% женский - 65,4%"
757,алла брулетова,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,алла брулетова,"бьюти, лайфстайл, фешн",алла брулетова,https://instagram.com/bruletova_,"бьюти, лайфстайл, фешн",1 960 000,750 000,110 000,1 200 000,None,...,None,None,None,"бьюти, лайфстайл, фешн",None,None,None,"москва – 4,5%, спб – 1,2%","13–17 лет – 4,4%, 18–24 года – 29,7%, 25–34 го...","мужской – 77,3%, женский – 22,6%"
756,алёна эксекверт,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,алёна эксекверт,"бьюти, лайфстайл",алёна эксекверт,https://instagram.com/exeqwert,"бьюти, лайфстайл",245 000,90 000,35 000,200 000,None,...,None,None,None,"бьюти, лайфстайл",None,None,None,"москва – 15,6% спб – 6,1%","13–17 лет – 23% 18–24 года – 63,4% 25–34 года ...","мужской – 9,6% женский – 90,3%"


In [419]:
# До

bufer.shape

(784, 181)

In [420]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [421]:
# После

bufer.shape

(742, 181)

In [422]:
show_duplicates(bufer)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
713,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
748,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
291,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",5200.0,7 389,под запрос,None,None,None
292,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",14235.0,под запрос,под запрос,None,None,None


In [423]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
291,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",5200.0,7 389,под запрос,None,None,None
292,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",14235.0,под запрос,под запрос,None,None,None
713,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
748,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"


In [424]:
telegram_columns = duplicates.filter(like='Telegram')
telegram_columns

,Telegram блогер,Telegram ссылка,Telegram тематика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram средняя вовлечённость,Telegram стоимость фотопост,Telegram стоимость видеопост,Telegram стоимость кружочек до 60 сек,Telegram стоимость текстовое сообщение без фото,Telegram статистика,Telegram город
291,None,None,"6 278, none, рецепты, фудблогер",None,None,None,None,None,None,None,None,None
292,None,None,"6 278, none, рецепты, фудблогер",None,None,None,None,None,None,None,None,None
713,лиза барашик,https://t.me/barrashikliza,"бьюти, влоги, лайфстайл, путешествия",82 000,17 000,-,р.60 000,р.105 000,р.75 000,р.40 000,https://drive.google.com/drive/folders/1MoeDga...,None
748,лиза барашик,https://t.me/barrashikliza,"бьюти, влоги, лайфстайл, путешествия",82 239,40 000,-,72 000 ₽,р.105 000,75 000 ₽,60 000 ₽,https://disk.yandex.ru/d/eAxssjK0YAkrgg,None


> Оставим только первое значение данного блогера, так как блогер "лиза барашик" на эксклюзиве у DidenokTeam

In [425]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
max_index = duplicates.index.max()
bufer.drop(max_index, inplace=True)

In [426]:
bufer[bufer['блогер'] == 'лиза барашик']

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
713,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"


### Подготовка таблицы nextup_youtube

In [427]:
nextup_youtube.head()

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,тематика,ца по возрасту,ца по полу,ца по странам,охват видео за 30 дней,стоимость интеграция / 1 слот,стоимость интеграция / 2 слот,стоимость преролл,NaN
7,NaN,П,илья exile,https://www.youtube.com/@ExileShow,4 280 000,статистика,Развлекательное,13–17 лет – 6%;\n18–24 года – 61%;\n25–34 года...,Мужской – 76%\nЖенский – 23%,"Россия – 63,1%.",3 500 000,1 572 000 ₽,1 300 000 ₽,1 250 000 ₽,https://disk.yandex.ru/d/ajGlE9Fx7SrxGw
8,NaN,П,настя badbaarbie,https://www.youtube.com/@badbarbara,4 240 000,статистика,"Развлекательное, Лайфстайл","13–17 лет – 14,4%;\n18–24 года – 58,8%;\n25–34...",Мужской – 7%\nЖенский – 93%,"Россия – 65,5%.",100 000,200 000 ₽,175 000 ₽,100 000 ₽,https://disk.yandex.ru/d/EDbrK6por7PaNA
9,NaN,П,приятный ильдар,https://www.youtube.com/@pleasantildar,3 770 000,статистика,"Развлекательное, Комментаторство","13–17 лет– 3,7%;\n18–24 года – 57,6%;\n25–34 г...",Мужской – 19%\nЖенский – 81%,Россия – 75%,1 000 000,715 000 ₽,600 000 ₽,600 000 ₽,https://disk.yandex.ru/d/TSHuC9PsJrkVnQ
10,NaN,П,"алексей hardplay , (только спецпроекты)",https://www.youtube.com/@HardPlayYT/,2 800 000,Статистика,"Развлекательное, Реакции","13–17 лет – 3,6%;\n18–24 года – 54,2%;\n25–34 ...",Мужской – 90%\nЖенский – 10%,Россия – 75%.,170 000,от 750000 Р,под запрос,под запрос,https://disk.yandex.ru/d/EkPhZdYvUZUUfg
11,NaN,П,даша боровик,https://www.youtube.com/@borovikdasha,2 600 000,статистика,NaN,NaN,NaN,NaN,150 000,NaN,NaN,NaN,https://drive.google.com/drive/folders/1gNch8O...


In [428]:
columns_to_drop = [0, 1]
nextup_youtube = nextup_columns_to_drop(nextup_youtube, columns_to_drop, True)

In [429]:
nextup_youtube_columns_upd = ['тематика', 'ца по возрасту', 'ца по полу', 'ца по странам']
nextup_youtube = nextup_col_upd(nextup_youtube, nextup_youtube_columns_upd)

In [430]:
[col for col in nextup_youtube.columns]

[nan,
 'имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'тематика',
 'ца по возрасту',
 'ца по полу',
 'ца по странам',
 'охват видео за 30 дней',
 'стоимость интеграция / 1 слот',
 'стоимость интеграция / 2 слот',
 'стоимость преролл']

In [431]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость интеграция для профильных клиентов 1 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 1 слот',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость интеграция для непрофильных клиентов 2 слот',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'YouTube город',
 'YouTube название канала

In [432]:
nextup_youtube.rename(columns = {'имя блогера':'блогер',
                                 'кол-во подписчиков':'количество подписчиков',
                                 'охват видео за 30 дней':'среднее кол-во просмотров видео',
                                 'стоимость интеграция / 1 слот':'стоимость интеграция 1 слот',
                                 'стоимость интеграция / 2 слот':'стоимость интеграция 2 слот',
                                 'стоимость преролл':'стоимость преролл до 30 сек',
                                 'статистика ссылка':'статистика'
                                }, inplace = True)

In [433]:
nextup_youtube

6,NaN,блогер,ссылка,количество подписчиков,тематика,ца по возрасту,ца по полу,ца по странам,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость преролл до 30 сек
7,NaN,илья exile,https://www.youtube.com/@ExileShow,4 280 000,развлекательное,"13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%.",3 500 000,1 572 000 ₽,1 300 000 ₽,1 250 000 ₽
8,NaN,настя badbaarbie,https://www.youtube.com/@badbarbara,4 240 000,"развлекательное, лайфстайл","13–17 лет – 14,4%;, 18–24 года – 58,8%;, 25–34...","мужской – 7%, женский – 93%","россия – 65,5%.",100 000,200 000 ₽,175 000 ₽,100 000 ₽
9,NaN,приятный ильдар,https://www.youtube.com/@pleasantildar,3 770 000,"развлекательное, комментаторство","13–17 лет– 3,7%;, 18–24 года – 57,6%;, 25–34 г...","мужской – 19%, женский – 81%",россия – 75%,1 000 000,715 000 ₽,600 000 ₽,600 000 ₽
10,NaN,"алексей hardplay , (только спецпроекты)",https://www.youtube.com/@HardPlayYT/,2 800 000,"развлекательное, реакции","13–17 лет – 3,6%;, 18–24 года – 54,2%;, 25–34 ...","мужской – 90%, женский – 10%",россия – 75%.,170 000,от 750000 Р,под запрос,под запрос
11,NaN,даша боровик,https://www.youtube.com/@borovikdasha,2 600 000,nan,nan,nan,nan,150 000,NaN,NaN,NaN
12,NaN,сабина хайрова,https://www.youtube.com/@Sabinakhayrova_,1 210 000,"развлекательное, лайфстайл","13–17 лет – 6%;, 18–24 года – 37%;, 25–34 года...","мужской – 57%, женский – 41%",россия – 66%.,1 000 000,—,—,—
13,NaN,paradeev1ch,https://www.youtube.com/@paradeevich,932 000,развлекательное,"13–17 лет – 4,7%;, 18–24 года – 64,3%;, 25–34 ...","мужской – 73,3%, женский – 26,1%","россия – 74,9%.",2 200 000,1 155 000 ₽,1 000 000 ₽,715 000 ₽
14,NaN,маш милаш,https://www.youtube.com/@mash_milash,807 000,nan,nan,nan,nan,65 000,230 000 ₽,NaN,NaN
15,NaN,ярослав дилблин,https://www.youtube.com/channel/UCcQZMSBg_nbOd...,463 000,развлекательное,"13–17 лет – 3,8%;, 18–24 года – 65,40%;, 25–34...","мужской – 75,9%, женский – 23,9%","россия – 68,1%.",700 000,900 000 ₽,715 000 ₽,600 000 ₽
16,NaN,саша frame tamer,https://www.youtube.com/@frametamer6,452 000,подкасты,"13–17 лет – 3,3%;, 18–24 года – 62,5%;, 25–34 ...","мужской – 80,2%, женский – 19,3%","россия – 62,7%.",1 000 000,860 000 ₽,800 000 ₽,715 000 ₽


In [434]:
columns_to_drop = [0]
nextup_youtube = nextup_columns_to_drop(nextup_youtube, columns_to_drop, False)

In [435]:
nextup_youtube

6,блогер,ссылка,количество подписчиков,тематика,ца по возрасту,ца по полу,ца по странам,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость преролл до 30 сек
7,илья exile,https://www.youtube.com/@ExileShow,4 280 000,развлекательное,"13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%.",3 500 000,1 572 000 ₽,1 300 000 ₽,1 250 000 ₽
8,настя badbaarbie,https://www.youtube.com/@badbarbara,4 240 000,"развлекательное, лайфстайл","13–17 лет – 14,4%;, 18–24 года – 58,8%;, 25–34...","мужской – 7%, женский – 93%","россия – 65,5%.",100 000,200 000 ₽,175 000 ₽,100 000 ₽
9,приятный ильдар,https://www.youtube.com/@pleasantildar,3 770 000,"развлекательное, комментаторство","13–17 лет– 3,7%;, 18–24 года – 57,6%;, 25–34 г...","мужской – 19%, женский – 81%",россия – 75%,1 000 000,715 000 ₽,600 000 ₽,600 000 ₽
10,"алексей hardplay , (только спецпроекты)",https://www.youtube.com/@HardPlayYT/,2 800 000,"развлекательное, реакции","13–17 лет – 3,6%;, 18–24 года – 54,2%;, 25–34 ...","мужской – 90%, женский – 10%",россия – 75%.,170 000,от 750000 Р,под запрос,под запрос
11,даша боровик,https://www.youtube.com/@borovikdasha,2 600 000,nan,nan,nan,nan,150 000,NaN,NaN,NaN
12,сабина хайрова,https://www.youtube.com/@Sabinakhayrova_,1 210 000,"развлекательное, лайфстайл","13–17 лет – 6%;, 18–24 года – 37%;, 25–34 года...","мужской – 57%, женский – 41%",россия – 66%.,1 000 000,—,—,—
13,paradeev1ch,https://www.youtube.com/@paradeevich,932 000,развлекательное,"13–17 лет – 4,7%;, 18–24 года – 64,3%;, 25–34 ...","мужской – 73,3%, женский – 26,1%","россия – 74,9%.",2 200 000,1 155 000 ₽,1 000 000 ₽,715 000 ₽
14,маш милаш,https://www.youtube.com/@mash_milash,807 000,nan,nan,nan,nan,65 000,230 000 ₽,NaN,NaN
15,ярослав дилблин,https://www.youtube.com/channel/UCcQZMSBg_nbOd...,463 000,развлекательное,"13–17 лет – 3,8%;, 18–24 года – 65,40%;, 25–34...","мужской – 75,9%, женский – 23,9%","россия – 68,1%.",700 000,900 000 ₽,715 000 ₽,600 000 ₽
16,саша frame tamer,https://www.youtube.com/@frametamer6,452 000,подкасты,"13–17 лет – 3,3%;, 18–24 года – 62,5%;, 25–34 ...","мужской – 80,2%, женский – 19,3%","россия – 62,7%.",1 000 000,860 000 ₽,800 000 ₽,715 000 ₽


#### Проверка на дубликаты

In [436]:
show_duplicates(nextup_youtube)

Количество дубликатов: 0


6,блогер,ссылка,количество подписчиков,тематика,ца по возрасту,ца по полу,ца по странам,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость преролл до 30 сек


### Двадцать пятое объединение

In [437]:
bufer = bufer.astype(str)

In [438]:
bufer = merge_bufer_df(bufer, nextup_youtube, 'YouTube')

Количество дубликатов: 36


In [439]:
show_duplicates(bufer).head(6)

Количество дубликатов: 36


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам
759,алина олешева,лайфстайл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"13–17 лет– 2,8%;, 18–24 года – 56,1%;, 25–34 г...","мужской – 35,8%, женский – 64,1%","россия – 80,5%."
668,алина олешева,"лайфстайл, музыка",алина олешева,https://www.instagram.com/oleshev,"лайфстайл, музыка",625 000,300 000,110 000,250 000,None,...,"лайфстайл, музыка",None,None,None,"москва - 14,6% спб - 6,6%","13-17 лет - 10,3% 18-24 года - 60,6% 25-34 год...","мужской - 34,5% женский - 65,4%",NaN,NaN,NaN
14,алёна эксекверт,"бьюти, лайфстайл",алёна эксекверт,https://instagram.com/exeqwert,"бьюти, лайфстайл",245 000,90 000,35 000,200 000,None,...,"бьюти, лайфстайл",None,None,None,"москва – 15,6% спб – 6,1%","13–17 лет – 23% 18–24 года – 63,4% 25–34 года ...","мужской – 9,6% женский – 90,3%",NaN,NaN,NaN
760,алёна эксекверт,"развлекательное, лайфстайл",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"13–17 лет– 12,9%;, 18–24 года – 66%;, 25–34 го...","мужской – 8,1%, женский – 91,7%","россия – 80,6%."
10,аня акулич,"бьюти, лайфстайл",аня акулич,https://instagram.com/anna.utka22,"бьюти, лайфстайл",613 000,350 000,120 000,290 000,None,...,"бьюти, лайфстайл",None,None,None,"москва – 13,5% спб – 4,7%","13–17 лет – 20,1%, 18–24 года – 62,7%, 25–34 г...","мужской – 48,2% женский – 51,7%",NaN,NaN,NaN
762,аня акулич,лайфстайл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"13–17 лет – 8%;, 18–24 года – 68%;, 25–34 года...","мужской – 19%, женский – 80%",россия – 74%.


In [440]:
# До

bufer.shape

(769, 184)

In [441]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [442]:
# После

bufer.shape

(769, 184)

In [443]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates.sort_values(by='блогер')

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам
759,алина олешева,"лайфстайл, музыка",алина олешева,https://www.instagram.com/oleshev,"лайфстайл, музыка",625 000,300 000,110 000,250 000,None,...,"лайфстайл, музыка",None,None,None,"москва - 14,6% спб - 6,6%","13-17 лет - 10,3% 18-24 года - 60,6% 25-34 год...","мужской - 34,5% женский - 65,4%","13–17 лет– 2,8%;, 18–24 года – 56,1%;, 25–34 г...","мужской – 35,8%, женский – 64,1%","россия – 80,5%."
668,алина олешева,"лайфстайл, музыка",алина олешева,https://www.instagram.com/oleshev,"лайфстайл, музыка",625 000,300 000,110 000,250 000,None,...,"лайфстайл, музыка",None,None,None,"москва - 14,6% спб - 6,6%","13-17 лет - 10,3% 18-24 года - 60,6% 25-34 год...","мужской - 34,5% женский - 65,4%","13–17 лет– 2,8%;, 18–24 года – 56,1%;, 25–34 г...","мужской – 35,8%, женский – 64,1%","россия – 80,5%."
14,алёна эксекверт,"бьюти, лайфстайл, развлекательное",алёна эксекверт,https://instagram.com/exeqwert,"бьюти, лайфстайл, развлекательное",245 000,90 000,35 000,200 000,None,...,"бьюти, лайфстайл, развлекательное",None,None,None,"москва – 15,6% спб – 6,1%","13–17 лет – 23% 18–24 года – 63,4% 25–34 года ...","мужской – 9,6% женский – 90,3%","13–17 лет– 12,9%;, 18–24 года – 66%;, 25–34 го...","мужской – 8,1%, женский – 91,7%","россия – 80,6%."
760,алёна эксекверт,"бьюти, лайфстайл, развлекательное",алёна эксекверт,https://instagram.com/exeqwert,"бьюти, лайфстайл, развлекательное",245 000,90 000,35 000,200 000,None,...,"бьюти, лайфстайл, развлекательное",None,None,None,"москва – 15,6% спб – 6,1%","13–17 лет – 23% 18–24 года – 63,4% 25–34 года ...","мужской – 9,6% женский – 90,3%","13–17 лет– 12,9%;, 18–24 года – 66%;, 25–34 го...","мужской – 8,1%, женский – 91,7%","россия – 80,6%."
10,аня акулич,"бьюти, лайфстайл",аня акулич,https://instagram.com/anna.utka22,"бьюти, лайфстайл",613 000,350 000,120 000,290 000,None,...,"бьюти, лайфстайл",None,None,None,"москва – 13,5% спб – 4,7%","13–17 лет – 20,1%, 18–24 года – 62,7%, 25–34 г...","мужской – 48,2% женский – 51,7%","13–17 лет – 8%;, 18–24 года – 68%;, 25–34 года...","мужской – 19%, женский – 80%",россия – 74%.
762,аня акулич,"бьюти, лайфстайл",аня акулич,https://instagram.com/anna.utka22,"бьюти, лайфстайл",613 000,350 000,120 000,290 000,None,...,"бьюти, лайфстайл",None,None,None,"москва – 13,5% спб – 4,7%","13–17 лет – 20,1%, 18–24 года – 62,7%, 25–34 г...","мужской – 48,2% женский – 51,7%","13–17 лет – 8%;, 18–24 года – 68%;, 25–34 года...","мужской – 19%, женский – 80%",россия – 74%.
741,илья exile,"музыка, развлекательное",илья exile,https://instagram.com/exile_music,"музыка, развлекательное",1 020 000,95 000,170 000,100 000,None,...,"музыка, развлекательное",None,None,None,"москва - 14,5% спб - 3,8%","13-17 лет - 21,8% 18-24 года - 61% 25-34 года ...","мужской - 60,1% женский - 39,2%","13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%."
660,илья exile,"музыка, развлекательное",илья exile,https://instagram.com/exile_music,"музыка, развлекательное",1 020 000,95 000,170 000,100 000,None,...,"музыка, развлекательное",None,None,None,"москва - 14,5% спб - 3,8%","13-17 лет - 21,8% 18-24 года - 61% 25-34 года ...","мужской - 60,1% женский - 39,2%","13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%."
766,катя бомонд,"бьюти, лайфстайл",катя бомонд,https://instagram.com/rul.bomond,"бьюти, лайфстайл",12 400,8 000,7 000,15 000,None,...,"бьюти, лайфстайл",None,None,None,"москва 

> Удалим дубликаты, оставив только первое вхождение, так как чаще всего именно в них самые большие и видимо правильные значения

In [444]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

In [445]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам


### Подготовка таблицы nextup_twitch

In [446]:
nextup_twitch.head()

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,средний онлайн на стриме,охват стрима за 14 дней,баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим,баннер + чат-бот + эксклюзивный стрим до 1 часа,баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц),статистика ссылка
7,NaN,П,paradeev1ch,https://twitch.tv/paradeev1ch,1 688 761,статистика,15 747,200 000,360 000 ₽,860 000 ₽,1 715 000 ₽,https://disk.yandex.ru/d/1qH15yQqi30A6g
8,NaN,П,koreshzy,https://twitch.tv/koreshzy,1 623 587,статистика,9 135,170 000,215 000 ₽,430 000 ₽,1 115 000 ₽,https://disk.yandex.ru/d/L5_h5VTloJAJ4w
9,NaN,П,stray228,https://www.twitch.tv/stray228,1 508 278,статистика,8 130,115 000,-,-,715 000 ₽,https://disk.yandex.ru/d/A6ST21aj67tKQw
10,NaN,П,jojohf,https://twitch.tv/jojohf,1 136 747,статистика,5 525,75 000,120 000 ₽,240 000 ₽,230 000 ₽,https://disk.yandex.ru/d/Wf1dKH0EO4noqw
11,NaN,Э,gensyxa,https://www.twitch.tv/gensyxa,1 106 288,статистика,2 456,80 000,150 000 ₽,279 000 ₽,500 000 ₽,https://disk.yandex.ru/d/OBbr9aLYNJTpZQ


In [447]:
columns_to_drop = [0, 1]
nextup_twitch = nextup_columns_to_drop(nextup_twitch, columns_to_drop, True)

In [448]:
nextup_twitch.head(1)

6,имя блогера,ссылка,кол-во подписчиков,средний онлайн на стриме,охват стрима за 14 дней,баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим,баннер + чат-бот + эксклюзивный стрим до 1 часа,баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц),статистика ссылка
7,paradeev1ch,https://twitch.tv/paradeev1ch,1 688 761,15 747,200 000,360 000 ₽,860 000 ₽,1 715 000 ₽,https://disk.yandex.ru/d/1qH15yQqi30A6g


In [449]:
[col for col in nextup_twitch.columns]

['имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'средний онлайн на стриме',
 'охват стрима за 14 дней',
 'баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим',
 'баннер + чат-бот + эксклюзивный стрим до 1 часа',
 'баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц)',
 'статистика ссылка']

In [450]:
[col for col in bufer.columns if 'Twitch' in col]

['Twitch блогер',
 'Twitch сcылка',
 'Twitch тематика',
 'Twitch количество подписчиков',
 'Twitch среднее кол-во зрителей',
 'Twitch уникальные зрители (30 дней)',
 'Twitch стоимость за баннер + ссылка в описании 1 мес',
 'Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции',
 'Twitch стоимость нативное голосовое упоминание товара в трансляции',
 'Twitch стоимость ссылка в чат-боте на 1 стрим',
 'Twitch стоимость демонстрация/обзор товара на трансляции',
 'Twitch статистика']

In [451]:
nextup_twitch.rename(columns = {'имя блогера':'блогер',
                                'кол-во подписчиков':'количество подписчиков',
                                'средний онлайн на стриме':'среднее кол-во зрителей',
                                'охват стрима за 14 дней':'уникальные зрители (14 дней)',
                                'баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим':'стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим',
                                'баннер + чат-бот + эксклюзивный стрим до 1 часа':'стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа',
                                'баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц)':'стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)',
                                'статистика ссылка':'статистика'
                               }, inplace = True)

#### Проверка на дубликаты

In [452]:
show_duplicates(nextup_twitch)

Количество дубликатов: 0


6,блогер,ссылка,количество подписчиков,среднее кол-во зрителей,уникальные зрители (14 дней),"стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)",статистика


### Двадцать шестое объединение

In [453]:
bufer = merge_bufer_df(bufer, nextup_twitch, 'Twitch')

Количество дубликатов: 4


In [454]:
show_duplicates(bufer)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram ца по возрасту,Instagram ца по полу,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам,Twitch ссылка,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)"
177,finargot,,None,None,,None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
766,finargot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://www.twitch.tv/finargot,20 000,27 000 ₽,30 000 ₽,75 000 ₽
742,paradeev1ch,развлекательное,None,None,развлекательное,None,None,None,None,None,...,None,None,"13–17 лет – 4,7%;, 18–24 года – 64,3%;, 25–34 ...","мужской – 73,3%, женский – 26,1%","россия – 74,9%.",NaN,NaN,NaN,NaN,NaN
751,paradeev1ch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://twitch.tv/paradeev1ch,200 000,360 000 ₽,860 000 ₽,1 715 000 ₽


In [455]:
# До

bufer.shape

(777, 189)

In [456]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [457]:
# После

bufer.shape

(776, 189)

In [458]:
show_duplicates(bufer)

Количество дубликатов: 2


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram ца по возрасту,Instagram ца по полу,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам,Twitch ссылка,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)"
177,finargot,,None,None,,None,None,None,None,None,...,None,None,None,None,None,https://www.twitch.tv/finargot,20 000,27 000 ₽,30 000 ₽,75 000 ₽
766,finargot,,None,None,,None,None,None,None,None,...,None,None,None,None,None,https://www.twitch.tv/finargot,20 000,27 000 ₽,30 000 ₽,75 000 ₽


> Удалим дубликаты, оставив только первое вхождение

In [459]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

### Подготовка таблицы nextup_vk

In [460]:
nextup_vk.head()

5,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,ца по городам,ца по возрасту,ца по полу,охват поста за 30 дней,охват vk клип за 30 дней,стоимость поста,стоимость vk клип,статистика ссылка
6,NaN,П,hardplay,https://vk.com/hp_legion,407 000,NaN,NaN,NaN,NaN,10 000,—,72 000 ₽,—,NaN
7,NaN,П,paradeev1ch,https://vk.com/paradeev1ch,414 000,NaN,NaN,NaN,NaN,45 000,-,78 500 ₽,71 500 ₽,NaN
8,NaN,П,маш милаш,https://vk.com/milash,92 000,NaN,NaN,NaN,NaN,NaN,NaN,79 000 ₽,71 500 ₽,NaN
9,NaN,Э,ира (inst_rinna),https://vk.com/inst_rinnaaaa,113 000,статистика,Москва – 26%\nСПб – 11%\nКраснодар – 4%,"13–17 лет – 17,5%\n18–24 года – 15%\n25–34 год...",Мужской – 68%\nЖенский – 32%,60 000,40 000,72 000 ₽,72 000 ₽,https://drive.google.com/drive/folders/1-3rUVS...
10,NaN,П,ярослав (дилблин),https://vk.com/dilblinnn,87 000,статистика,Москва – 21%\nСПб – 11%\nУфа – 4%,13–17 лет – 35%\n18–24 года – 17%\n25–34 года ...,Мужской – 34%\nЖенский – 66%,100 000,–,80 000 ₽,–,https://drive.google.com/drive/folders/15UUvEu...


In [461]:
columns_to_drop = [0, 1]
nextup_vk = nextup_columns_to_drop(nextup_vk, columns_to_drop, True)

In [462]:
nextup_vk_columns_upd = ['ца по городам', 'ца по возрасту', 'ца по полу']
nextup_vk = nextup_col_upd(nextup_vk, nextup_vk_columns_upd)

In [463]:
[col for col in nextup_vk.columns]

['имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'ца по городам',
 'ца по возрасту',
 'ца по полу',
 'охват поста за 30 дней',
 'охват vk клип за 30 дней',
 'стоимость поста',
 'стоимость vk клип',
 'статистика ссылка']

In [464]:
[col for col in bufer.columns if 'VK группа' in col]

['VK группа блогер',
 'VK группа ссылка',
 'VK группа тематика',
 'VK группа количество подписчиков',
 'VK группа среднее кол-во просмотров поста',
 'VK группа среднее кол-во просмотров вк клипа',
 'VK группа среднее кол-во лайков',
 'VK группа среднее кол-во комментариев',
 'VK группа стоимость фотопост',
 'VK группа стоимость видеопост',
 'VK группа стоимость вк клип',
 'VK группа статистика',
 'VK группа город',
 'VK группа стоимость передачи прав на 1 месяц']

In [465]:
nextup_vk.rename(columns = {'имя блогера':'блогер',
                            'кол-во подписчиков':'количество подписчиков',
                            'охват поста за 30 дней':'среднее кол-во просмотров поста',
                            'охват vk клип за 30 дней':'среднее кол-во просмотров вк клипа',
                            'стоимость поста':'стоимость фотопост',
                            'стоимость vk клип':'стоимость вк клип',
                            'статистика ссылка':'статистика'
                           }, inplace = True)

In [466]:
nextup_vk.head(1)

5,блогер,ссылка,количество подписчиков,ца по городам,ца по возрасту,ца по полу,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,стоимость фотопост,стоимость вк клип,статистика
6,hardplay,https://vk.com/hp_legion,407 000,nan,nan,nan,10 000,—,72 000 ₽,—,NaN


#### Проверка на дубликаты

In [467]:
show_duplicates(nextup_vk)

Количество дубликатов: 0


5,блогер,ссылка,количество подписчиков,ца по городам,ца по возрасту,ца по полу,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,стоимость фотопост,стоимость вк клип,статистика


### Двадцать седьмое объединение

In [468]:
bufer = merge_bufer_df(bufer, nextup_vk, 'VK группа')

Количество дубликатов: 16


In [469]:
show_duplicates(bufer).head(6)

Количество дубликатов: 16


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube ца по полу,YouTube ца по странам,Twitch ссылка,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)",VK группа ца по городам,VK группа ца по возрасту,VK группа ца по полу
742,paradeev1ch,развлекательное,None,None,развлекательное,None,None,None,None,None,...,"мужской – 73,3%, женский – 26,1%","россия – 74,9%.",https://twitch.tv/paradeev1ch,200 000,360 000 ₽,860 000 ₽,1 715 000 ₽,NaN,NaN,NaN
776,paradeev1ch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan
745,арина arlovedbyyou,"лайфстайл, развлекательное",None,None,"лайфстайл, развлекательное",None,None,None,None,None,...,"мужской – 5,9%, женский – 93,9%","россия – 85,3%.",None,None,None,None,None,NaN,NaN,NaN
783,арина arlovedbyyou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan
330,данила горилла,лайфстайл,данила горилла,https://instagram.com/danila.gorilla,лайфстайл,155 000,56 000,49 000,–,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
782,данила горилла,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan


In [470]:
# До

bufer.shape

(788, 192)

In [471]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [472]:
# После

bufer.shape

(786, 192)

In [473]:
show_duplicates(bufer)

Количество дубликатов: 12


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube ца по полу,YouTube ца по странам,Twitch ссылка,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)",VK группа ца по городам,VK группа ца по возрасту,VK группа ца по полу
330,данила горилла,лайфстайл,данила горилла,https://instagram.com/danila.gorilla,лайфстайл,155 000,56 000,49 000,–,None,...,None,None,None,None,None,None,None,nan,nan,nan
782,данила горилла,лайфстайл,данила горилла,https://instagram.com/danila.gorilla,лайфстайл,155 000,56 000,49 000,–,None,...,None,None,None,None,None,None,None,nan,nan,nan
607,илья exile,"музыка, развлекательное",илья exile,https://instagram.com/exile_music,"музыка, развлекательное",1 020 000,95 000,170 000,100 000,None,...,"мужской – 76%, женский – 23%","россия – 63,1%.",None,None,None,None,None,nan,nan,nan
780,илья exile,"музыка, развлекательное",илья exile,https://instagram.com/exile_music,"музыка, развлекательное",1 020 000,95 000,170 000,100 000,None,...,"мужской – 76%, женский – 23%","россия – 63,1%.",None,None,None,None,None,nan,nan,nan
606,ира (inst_rinna),"бьюти, лайфстайл, танцы",ира (inst_rinna),https://instagram.com/inst_rinna,"бьюти, лайфстайл, танцы",1 170 000,650 000,230 000,210 000,None,...,None,None,None,None,None,None,None,"москва – 26%, спб – 11%, краснодар – 4%","13–17 лет – 17,5%, 18–24 года – 15%, 25–34 год...","мужской – 68%, женский – 32%"
778,ира (inst_rinna),"бьюти, лайфстайл, танцы",ира (inst_rinna),https://instagram.com/inst_rinna,"бьюти, лайфстайл, танцы",1 170 000,650 000,230 000,210 000,None,...,None,None,None,None,None,None,None,"москва – 26%, спб – 11%, краснодар – 4%","13–17 лет – 17,5%, 18–24 года – 15%, 25–34 год...","мужской – 68%, женский – 32%"
620,маш милаш,"an, лайвстайл",маш милаш,https://www.instagram.com/mash_milash_hub,"an, лайвстайл",568 000,-,150 000,250 000,None,...,nan,nan,None,None,None,None,None,nan,nan,nan
777,маш милаш,"an, лайвстайл",маш милаш,https://www.instagram.com/mash_milash_hub,"an, лайвстайл",568 000,-,150 000,250 000,None,...,nan,nan,None,None,None,None,None,nan,nan,nan
621,саша теслонд,"лайфстайл, развлекательное, фешн",саша теслонд,https://instagram.com/teslond,"лайфстайл, развлекательное, фешн",538 000,200 000,90 000,350 000,None,...,"мужской – 14%, женский – 93,9%",россия – 84%.,None,None,None,None,None,nan,nan,nan
786,саша теслонд,"лайфстайл, развлекательное, фешн",саша теслонд,https://instagram.com/teslond,"лайфстайл, развлекательное, фешн",538 000,200 000,90 000,350 000,None,...,"мужской – 14%, женский – 93,9%",россия – 84%.,None,None,None,None,None,nan,nan,nan


> Удалим дубликаты, оставив только первое вхождение

In [474]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

In [475]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('https://t.me/Oliva_Nick', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (6%)', inplace=True)

> Посмотрим на нашу таблицу bufer

In [476]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube ца по полу,YouTube ца по странам,Twitch ссылка,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)",VK группа ца по городам,VK группа ца по возрасту,VK группа ца по полу
0,катя клэп,"бьюти, лайфстайл",катя клэп,https://www.instagram.com/kateclapp/,"бьюти, лайфстайл",10 100 000,250 000,150 000,200 000,50 000,...,None,None,None,None,None,None,None,None,None,None
1,катя мотовилова,"бьюти, лайфстайл",катя мотовилова,https://www.instagram.com/kate_motovilova/,"бьюти, лайфстайл",276 000,30 000,15 000,70 000,5 000,...,None,None,None,None,None,None,None,None,None,None
2,маша нарциссизм,"бьюти, лайфстайл",None,None,"бьюти, лайфстайл",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,строкова татьяна,"бьюти, лайфстайл",None,None,"бьюти, лайфстайл",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,рязанова татьяна,"бьюти, лайфстайл",None,None,"бьюти, лайфстайл",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,hardplay,,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,nan,nan,nan
781,frametamer,,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,nan,nan,nan
784,лунаком,,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,nan,nan,nan
785,bad barbie,,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,nan,nan,nan


## Подготовка данных по блогерам от AvtorMedia

> Предварительно посмотрим на содержание таблицы

In [477]:
avtormedia_telegram.head(1)

,Блогер,Тематика,Подписчики,Материалы\nи статистика,Средние просмотры,Короткое видео,Пост,Спешл,Цена,Спешл,...,Пакетное предложение,Цена,Охват,CPV,Пакетное предложение,Цена,Охват,CPV,Ссылка,Статистика ссылка
1,Илья Варламов @varlamov_news,новости,1 200 000,Ссылка,470 000,833 333 ₽,450 000 ₽,Сторис,500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov_news,https://drive.google.com/drive/u/1/folders/165...


In [478]:
avtormedia_df = [avtormedia_youtube,
                 avtormedia_instagram,
                 avtormedia_telegram,
                 avtormedia_vk,
                 avtormedia_zen,
                 avtormedia_podcast]

In [479]:
key_word = ['CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Блогер']

In [480]:
# for df in avtormedia_df:
#     pretraining_df(df, key_word, drop_word, upd_columns)
    
    
for i in range(len(avtormedia_df)):
    df = avtormedia_df[i]  # Получаем текущий DataFrame по индексу
    
    upd_columns = []  # Пустой список столбцов с именами блогеров для приведения к единому формату

    if 'Блогер' in df.columns:
        upd_columns.append('Блогер')
    if 'Канал' in df.columns:
        upd_columns.append('Канал')

    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы avtormedia_youtube

In [481]:
avtormedia_youtube.head()

,блогер,тематика,подписчики,"материалы, и статистика",мидролл в основные форматы,охват видео,интеграция,пре-ролл,интеграция (2 слот),спецпроект,...,цена,охват,пакетное предложение,цена,охват,пакетное предложение,цена,охват,ссылка,статистика
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@HiBestMan/featured,https://drive.google.com/drive/folders/1AG9et9...
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@ItsMamix/featured,https://drive.google.com/drive/folders/12S_GJx...
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@VilyamBruno/featured,https://drive.google.com/drive/u/1/folders/1pS...
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@LUANAAA/featured,https://drive.google.com/drive/u/1/folders/1Cx...
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@OneTwoOneTwoRS/featured,https://drive.google.com/drive/folders/1HCe1B_...


In [482]:
[col for col in avtormedia_youtube.columns]

['блогер',
 'тематика',
 'подписчики',
 'материалы, и статистика',
 'мидролл в основные форматы',
 'охват видео',
 'интеграция',
 'пре-ролл',
 'интеграция (2 слот)',
 'спецпроект',
 'охват shorts',
 'shorts',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'ссылка',
 'статистика']

In [483]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'стоимость мидролл до 30 сек',
                 'среднее кол-во просмотров видео',
                 'стоимость интеграция 1 слот',
                 'стоимость преролл до 30 сек',
                 'стоимость интеграция 2 слот',
                 'стоимость спецпроект',
                 'среднее кол-во просмотров shorts',
                 'стоимость за shorts до 1 мин',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'спешл 3',
                 'цена 3',
                 'спешл 4',
                 'цена 4',
                 'спешл 5',
                 'цена 5',
                 'спешл 6',
                 'цена 6',
                 'спешл 7',
                 'цена 7',
                 'пакетное предложение 8',
                 'цена 8',
                 'охват 8',
                 'пакетное предложение 9',
                 'цена 9',
                 'охват 9',
                 'пакетное предложение 10',
                 'цена 10',
                 'охват 10',
                 'пакетное предложение 11',
                 'цена 11',
                 'охват 11',
                 'пакетное предложение 12',
                 'цена 12',
                 'охват 12',
                 'ссылка',
                 'статистика']

In [484]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость интеграция для профильных клиентов 1 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 1 слот',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость интеграция для непрофильных клиентов 2 слот',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'YouTube город',
 'YouTube название канала

In [485]:
avtormedia_youtube.columns = bufer_columns

In [486]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,материалы и статистика,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,...,цена 10,охват 10,пакетное предложение 11,цена 11,охват 11,пакетное предложение 12,цена 12,охват 12,ссылка,статистика
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@HiBestMan/featured,https://drive.google.com/drive/folders/1AG9et9...
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@ItsMamix/featured,https://drive.google.com/drive/folders/12S_GJx...
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@VilyamBruno/featured,https://drive.google.com/drive/u/1/folders/1pS...
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@LUANAAA/featured,https://drive.google.com/drive/u/1/folders/1Cx...
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@OneTwoOneTwoRS/featured,https://drive.google.com/drive/folders/1HCe1B_...


In [487]:
column_list_1 = []

for column_number in range(1, 8):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_youtube = get_column_values(avtormedia_youtube, column_number, position_column, price_column, column_list_1)

In [488]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,материалы и статистика,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,...,Интеграция в формат «Книга жалоб» (1 слот)_4,Продакт-плейсмент_4,"Выездная интеграция в воскресный выпуск новостей ""Чё Происходит""_5",Преролл в реакцию на трейлер или разборы_5,Пост в сообществе_5,Интеграция в обзор_5,Интеграция в формат «Спешл»_5,Возможность выбрать тему ролика_6,Интеграция в подкаст «Сладкая жизнь» (2 слот)_6,Пре-ролл в подкаст «Сладкая жизнь»_7
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [489]:
sorted_column_names_1 = sorted(column_list_1)
sorted_column_names_1

['Бегущая строка_1',
 'Возможность выбрать тему ролика_6',
 'Выездная интеграция в воскресный выпуск новостей "Чё Происходит"_5',
 'Выездная интеграция_2',
 'Выездная интеграция_3',
 'Выездной спецпроект_1',
 'Интеграция (3 слот)_2',
 'Интеграция (4 слот)_3',
 'Интеграция в воскресный выпуск новостей "Чё Происходит" (2 слот)_2',
 'Интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот_4',
 'Интеграция в обзор_5',
 'Интеграция в подкаст «Сладкая жизнь» (1 слот)_2',
 'Интеграция в подкаст «Сладкая жизнь» (2 слот)_6',
 'Интеграция в разборы_4',
 'Интеграция в реакцию на трейлер_2',
 'Интеграция в рецепт_4',
 'Интеграция в ролик на тему по выбору клиента_2',
 'Интеграция в тревел-ролики_1',
 'Интеграция в формат "Криминальный ТикТок"_2',
 'Интеграция в формат «Книга жалоб» (1 слот)_4',
 'Интеграция в формат «Книга жалоб» (2 слот)_1',
 'Интеграция в формат «Свой/чужой» (1 слот)_4',
 'Интеграция в формат «Свой/чужой» (2 слот)_1',
 'Интеграция в формат «Спешл»_5',
 'Интеграция в форма

In [490]:
def get_column_values_packet(df: pd.DataFrame,
                             key: int,
                             column_with_position: str,
                             column_with_price: str,
                             column_with_coverage: str,
                             column_list: list) -> pd.DataFrame:
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
    df (pd.DataFrame) - DataFrame, в котором делаем преобразования;
    key (int) - Ключ в виде int, который нужно добавить к названию, чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
    column_with_position (str) - Столбец, в котором указаны позиции;
    column_with_price (str) - Столбец, в котором указаны цены на позиции;
    column_with_coverage (str) - Столбец, в котором указан охват позиции;
    column_list (list) - 
    
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
    Создаём список с уникальными значениями позиций, приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
    spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                               .str.lower()\
                                               .str.replace('/\n', ', ')\
                                               .str.replace('\n', ', ')\
                                               .str.replace('*', '')\
                                               .unique()
    
    #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
    #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
    spisok = spisok[~pd.isna(spisok)]
        
        
    # Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
    
    # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
        df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
        where(df[column_with_position] == unique_position[i], other='')
    
    
    Удаляем исходные столбцы, чтобы не было повтора:
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)
    
    
    Возвращаем обновлённый DataFrame
    return df    
                
    """

    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]

# Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
        
        new_column_name = f'{unique_position[i]}_{key}'
        
# Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:    
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')
            
        df[f'охват {new_column_name}'] = \
                df[column_with_coverage].where(df[column_with_position] == unique_position[i], other='')
        
        
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [491]:
column_list_2 = []

for column_number in range(8, 13):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_youtube = get_column_values_packet(avtormedia_youtube,
                                                  column_number,
                                                  position_column,
                                                  price_column,
                                                  coverage_column,
                                                  column_list_2)

In [492]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,материалы и статистика,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,...,Интеграция в рецепт + фото в Инст_9,охват Интеграция в рецепт + фото в Инст_9,Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9,охват Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9,Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10,охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10,Интеграция в видео на YouTube + пост в сообществе YouTube_11,охват Интеграция в видео на YouTube + пост в сообществе YouTube_11,Интеграция на YouTube + Сторис в Instagram_12,охват Интеграция на YouTube + Сторис в Instagram_12
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,,NaN,,NaN,,NaN,,NaN,
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,,NaN,,NaN,,NaN,,NaN,
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,,NaN,,NaN,,NaN,,NaN,
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,,NaN,,NaN,,NaN,,NaN,
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,,NaN,,NaN,,NaN,,NaN,


In [493]:
sorted_column_names_2 = sorted(column_list_2)
sorted_column_names_2

['3 интеграции_8',
 'Cпецпроект + фото-пост в Инст_8',
 'Интеграция + фото-пост в Инст_9',
 'Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10',
 'Интеграция в видео на YouTube + пост в сообществе YouTube_11',
 'Интеграция в видео на YouTube + фото-пост в Инст_8',
 'Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8',
 'Интеграция в обзор + фото в Инст_8',
 'Интеграция в рецепт + фото в Инст_9',
 'Интеграция на YouTube + Сторис в Instagram_12',
 'Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8',
 'Интеграция на Основном канале + Романов Говорит_8',
 'Пакет соц. сети (сторис ИГ + пост ВК + пост ТГ) (пост в формате «текст прислан», пост и фото от клиента)_8',
 'Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9',
 'Спецпроект + Анонс конкурса_8',
 'Спецпроект на YouTube + пост в Telegram_9',
 'интеграция YouTube + Дзен_8']

#### Обработаем новые столбцы

##### Начнём со списка sorted_column_names_1

In [494]:
dict_new_columns = {'стоимость усложненная интеграция':['Усложненная интеграция_2',
                                                        'Усложненная интеграция_3',
                                                        'Расширенная интеграция_1'],
                    
                    'стоимость пост в сообществе': ['Пост в сообществе_2',
                                                    'Пост в сообществе_3',
                                                    'Пост в сообществе_5'],
                    
                    'стоимость выездная интеграция':['Выездная интеграция_2',
                                                    'Выездная интеграция_3'],
                    
                   'стоимость интеграция с выбором темы видео':['Расширенная интеграция в ролик на тему по выбору клиента_1',
                                                                'Возможность выбрать тему ролика_6',
                                                                'Интеграция в ролик на тему по выбору клиента_2'],  
                    
                    'стоимость мидролл до 30 сек':['стоимость мидролл до 30 сек',
                                                   'Мидролл_1']
                    
                   }

In [495]:
def agg_columns(df: pd.DataFrame, dictation: dict) -> pd.DataFrame:
    
    """
    
    
    ПАРАМЕТРЫ:
    df: pd.DataFrame - 
    dictation: dict
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    """
    
    for new_column, old_column in dictation.items():

        for col in old_column:
            # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
            # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
            df[col] = df[col].fillna('').astype(str)

        df[new_column] = df[old_column].agg(''.join, axis=1)  

        #Вернём обратно NaN
        df[new_column] = df[new_column].replace('', np.nan)
        
#         # Удаляем столбцы из словаря
#         df.drop(columns=old_column, inplace=True)
        
    return df

In [496]:
# avtormedia_youtube = agg_columns(avtormedia_youtube, dict_new_columns)

In [497]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        avtormedia_youtube[col] = avtormedia_youtube[col].fillna('').astype(str)
    
    avtormedia_youtube[new_column] = avtormedia_youtube[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    avtormedia_youtube[new_column] = avtormedia_youtube[new_column].replace('', np.nan)
    avtormedia_youtube[col] = avtormedia_youtube[col].astype(str)

In [498]:
avtormedia_youtube.drop(columns=['Усложненная интеграция_2',
                                 'Усложненная интеграция_3',
                                 'Расширенная интеграция_1',
                                 'Пост в сообществе_2',
                                 'Пост в сообществе_3',
                                 'Пост в сообществе_5',
                                 'Выездная интеграция_2',
                                 'Выездная интеграция_3',
                                 'Расширенная интеграция в ролик на тему по выбору клиента_1',
                                 'Возможность выбрать тему ролика_6',
                                 'Интеграция в ролик на тему по выбору клиента_2',
                                 'Мидролл_1'], axis=1, inplace=True)

In [499]:
avtormedia_youtube

,блогер,тематика,количество подписчиков,материалы и статистика,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,...,Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10,охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10,Интеграция в видео на YouTube + пост в сообществе YouTube_11,охват Интеграция в видео на YouTube + пост в сообществе YouTube_11,Интеграция на YouTube + Сторис в Instagram_12,охват Интеграция на YouTube + Сторис в Instagram_12,стоимость усложненная интеграция,стоимость пост в сообществе,стоимость выездная интеграция,стоимость интеграция с выбором темы видео
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
6,дюшес,челленджи\nэксперименты,5 840 000,Ссылка,NaN,1 500 000,970 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
7,пушер,челленджи\nэксперименты,4 780 000,Ссылка,NaN,281 000,478 723 ₽,425 531 ₽,420 000 ₽,1 063 829 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
8,илья варламов,урбанистика\nобщество\nполитика,4 710 000,Ссылка,NaN,1 200 000,1 200 000 ₽,600 000 ₽,1 000 000 ₽,5 881 250 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,1 400 000 ₽,834 000 ₽
9,gerasev,челленджи,4 000 000,Ссылка,NaN,1 000 000,490 000 ₽,345 744 ₽,450 000 ₽,1 418 085 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
10,it's экспериментаторы,эксперименты,3 680 000,Ссылка,NaN,800 000,750 000 ₽,400 000 ₽,NaN,1 200 000 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN


In [500]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость интеграция для профильных клиентов 1 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 1 слот',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость интеграция для непрофильных клиентов 2 слот',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'YouTube город',
 'YouTube название канала

In [501]:
avtormedia_youtube.rename(columns = {'Бегущая строка_1':'стоимость бегущая строка',
                                     'Мидролл_1':'стоимость мидролл до 30 сек',
                                     'Выездная интеграция в воскресный выпуск новостей "Чё Происходит"_5':'стоимость выездная интеграция в воскресный выпуск новостей "Чё Происходит"',
                                     'Выездной спецпроект_1':'стоимость выездной спецпроект',
                                     'Интеграция (3 слот)_2':'стоимость интеграция 3 слот',
                                     'Интеграция (4 слот)_3':'стоимость интеграция 4 слот',
                                     'Интеграция в воскресный выпуск новостей "Чё Происходит" (2 слот)_2':'стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 2 слот',
                                     'Интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот_4':'стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот',
                                     'Интеграция в обзор_5':'стоимость интеграция в обзор',
                                     'Интеграция в подкаст «Сладкая жизнь» (1 слот)_3':'стоимость интеграция в подкаст «Сладкая жизнь» 1 слот',
                                     'Интеграция в подкаст «Сладкая жизнь» (2 слот)_6':'стоимость интеграция в подкаст «Сладкая жизнь» 2 слот',
                                     'Интеграция в разборы_4':'стоимость интеграция в разборы',
                                     'Интеграция в реакцию на трейлер_2':'стоимость интеграция в реакцию на трейлер',
                                     'Интеграция в рецепт_4':'стоимость интеграция в рецепт',
                                     'Интеграция в ролик с Линой Диановой (1 слот)_1':'стоимость интеграция в ролик с Линой Диановой 1 слот',
                                     'Интеграция в ролик с Линой Диановой (2 слот)_2':'стоимость интеграция в ролик с Линой Диановой 2 слот',
                                     'Интеграция в тревел-ролики_1':'стоимость интеграция в тревел-ролики',
                                     'Интеграция в формат "Криминальный ТикТок"_2':'стоимость интеграция в формат "Криминальный ТикТок"',
                                     'Интеграция в формат «Книга жалоб» (1 слот)_4':'стоимость интеграция в формат «Книга жалоб» 1 слот',
                                     'Интеграция в формат «Книга жалоб» (2 слот)_1':'стоимость интеграция в формат «Книга жалоб» 2 слот',
                                     'Интеграция в формат «Спешл»_5':'стоимость интеграция в формат «Спешл»',
                                     'Интеграция в формат Стиратель на YouTube_1':'стоимость интеграция в формат Стиратель на YouTube',
                                     'Интеграция в формат новости_3':'стоимость интеграция в формат новости',
                                     'Интеграция с участием Максима Каца_1':'стоимость интеграция с участием Максима Каца',
                                     'Повторное размещение реализованной интеграции_1':'стоимость повторное размещение реализованной интеграции',
                                     'Пре-ролл  в формат "Линия выгорания"_3':'стоимость преролл в формат "Линия выгорания"',
                                     'Пре-ролл в подкаст «Сладкая жизнь»_7':'стоимость преролл в подкаст «Сладкая жизнь»',
                                     'Пре-ролл в ролик с Линой Диановой_4':'стоимость преролл в ролик с Линой Диановой',
                                     'Пре-ролл в формат «Книга жалоб»_2':'стоимость преролл в формат «Книга жалоб»',
                                     'Преролл в обзор_3':'стоимость преролл в обзор',
                                     'Преролл в реакцию на трейлер или разборы_5':'стоимость преролл в реакцию на трейлер или разборы',
                                     'Преролл в рецепт_2':'стоимость преролл в рецепт',
                                     'Преролл в формат новости_1':'стоимость преролл в формат новости',
                                     'Продакт-плейсмент_4':'стоимость продакт плейсмент',
                                     'Ссылка в закрепленном комментарии и ссылка в описании до ката_4':'стоимость псылка в закрепленном комментарии и ссылка в описании до ката',
                                    }, inplace = True)

##### Обработаем список sorted_column_names_2

In [502]:
sorted_column_names_2

['3 интеграции_8',
 'Cпецпроект + фото-пост в Инст_8',
 'Интеграция + фото-пост в Инст_9',
 'Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10',
 'Интеграция в видео на YouTube + пост в сообществе YouTube_11',
 'Интеграция в видео на YouTube + фото-пост в Инст_8',
 'Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8',
 'Интеграция в обзор + фото в Инст_8',
 'Интеграция в рецепт + фото в Инст_9',
 'Интеграция на YouTube + Сторис в Instagram_12',
 'Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8',
 'Интеграция на Основном канале + Романов Говорит_8',
 'Пакет соц. сети (сторис ИГ + пост ВК + пост ТГ) (пост в формате «текст прислан», пост и фото от клиента)_8',
 'Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9',
 'Спецпроект + Анонс конкурса_8',
 'Спецпроект на YouTube + пост в Telegram_9',
 'интеграция YouTube + Дзен_8']

In [503]:
avtormedia_youtube.rename(columns = {'3 интеграции_8':'стоимость 3 интеграции',
                                     'Cпецпроект + фото-пост в Инст_8':'стоимость Cпецпроект и фотопост в Инстаграм',
                                     'Интеграция + фото-пост в Инст_9':'стоимость интеграция и фотопост в Инстаграм',
                                     'Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10':'стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
                                     'Интеграция в видео на YouTube + пост в сообществе YouTube_11':'стоимость интеграция в видео на YouTube и пост в сообществе YouTube',
                                     'Интеграция в видео на YouTube + фото-пост в Инст_8':'стоимость интеграция в видео на YouTube и фотопост в Инстаграм',
                                     'Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8':'стоимость интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
                                     'Интеграция в обзор + фото в Инст_8':'стоимость интеграция в обзор и фотопост в Инстаграм',
                                     'Интеграция в рецепт + фото в Инст_9':'стоимость интеграция в рецепт и фотопост в Инстаграм',
                                     'Интеграция на YouTube + Сторис в Instagram_12':'стоимость интеграция на YouTube и сторис в Инстаграм',
                                     'Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8':'стоимость интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
                                     'Интеграция на Основном канале + Романов Говорит_8':'стоимость интеграция на Основном канале + Романов Говорит',
                                     'Спецпроект + Анонс конкурса_8':'стоимость спецпроект и анонс конкурса',
                                     'Спецпроект на YouTube + пост в Telegram_9':'стоимость спецпроект на YouTube и пост в Телеграм',
                                     'интеграция YouTube + Дзен_8':'стоимость интеграция YouTube и Zen'}, inplace = True)

In [504]:
[col for col in avtormedia_youtube.columns if 'охват' in col]

['охват 8',
 'охват 9',
 'охват 10',
 'охват 11',
 'охват 12',
 'охват 3 интеграции_8',
 'охват Cпецпроект + фото-пост в Инст_8',
 'охват Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8',
 'охват Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8',
 'охват Интеграция в видео на YouTube + фото-пост в Инст_8',
 'охват Спецпроект + Анонс конкурса_8',
 'охват Интеграция в обзор + фото в Инст_8',
 'охват интеграция YouTube + Дзен_8',
 'охват Интеграция на Основном канале + Романов Говорит_8',
 'охват Пакет соц. сети (сторис ИГ + пост ВК + пост ТГ) (пост в формате «текст прислан», пост и фото от клиента)_8',
 'охват Интеграция + фото-пост в Инст_9',
 'охват Спецпроект на YouTube + пост в Telegram_9',
 'охват Интеграция в рецепт + фото в Инст_9',
 'охват Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9',
 'охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном коммен

> Отредактируем также название столбов с охватом пакетных предложений

In [505]:
avtormedia_youtube.rename(columns = {'охват 3 интеграции_8':'охват позиции 3 интеграции',
                                     'охват Cпецпроект + фото-пост в Инст_8':'охват позиции Cпецпроект и фотопост в Инстаграм',
                                     'охват Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8':'охват позиции интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
                                     'охват Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8':'охват позиции интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
                                     'охват Интеграция в видео на YouTube + фото-пост в Инст_8':'охват позиции интеграция в видео на YouTube и фотопост в Инстаграм',
                                     'охват Спецпроект + Анонс конкурса_8':'охват позиции спецпроект и анонс конкурса',
                                     'охват Интеграция в обзор + фото в Инст_8':'охват позиции интеграция в обзор и фотопост в Инстаграм',
                                     'охват интеграция YouTube + Дзен_8':'охват позиции интеграция YouTube и Zen',
                                     'охват Интеграция на Основном канале + Романов Говорит_8':'охват позиции интеграция на Основном канале + Романов Говорит',
                                     'охват Интеграция + фото-пост в Инст_9':'охват позиции интеграция и фотопост в Инстаграм',
                                     'охват Спецпроект на YouTube + пост в Telegram_9':'охват позиции спецпроект на YouTube и пост в Телеграм',
                                     'охват Интеграция в рецепт + фото в Инст_9':'охват позиции интеграция в рецепт и фотопост в Инстаграм',
                                     'охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10':'охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
                                     'охват Интеграция в видео на YouTube + пост в сообществе YouTube_11':'охват позиции интеграция в видео на YouTube и пост в сообществе YouTube',
                                     'охват Интеграция на YouTube + Сторис в Instagram_12':'охват позиции интеграция на YouTube и сторис в Инстаграм'
                                    }, inplace=True)

In [506]:
avtormedia_youtube.drop(columns=['материалы и статистика',
                                 'охват 8',
                                 'охват 9',
                                 'охват 10',
                                 'охват 11',
                                 'охват 12'], inplace=True)

In [507]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,среднее кол-во просмотров shorts,...,"стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии","охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии",стоимость интеграция в видео на YouTube и пост в сообществе YouTube,охват позиции интеграция в видео на YouTube и пост в сообществе YouTube,стоимость интеграция на YouTube и сторис в Инстаграм,охват позиции интеграция на YouTube и сторис в Инстаграм,стоимость усложненная интеграция,стоимость пост в сообществе,стоимость выездная интеграция,стоимость интеграция с выбором темы видео
1,himan,челленджи\nэксперименты,19 100 000,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,2 200 000,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
2,it's mamix,эксперименты,15 300 000,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,NaN,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
3,вильям бруно,челленджи\nюмор,9 970 000,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,3 400 000,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
4,луана,анимация,6 470 000,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,1 500 000,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
5,onetwo,юмор\nскетчи,6 000 000,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,NaN,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN


> Отчистим пустые строки и сделаем их как NaN

In [508]:
# # Получаем список столбцов, содержащих слово "охват"
# ohvat_columns = [col for col in avtormedia_youtube.columns if 'охват' in col]

# # Заменяем пустые значения в этих столбцах на NaN
# avtormedia_youtube[ohvat_columns] = avtormedia_youtube[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [509]:
show_duplicates(avtormedia_youtube)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,среднее кол-во просмотров shorts,...,"стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии","охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии",стоимость интеграция в видео на YouTube и пост в сообществе YouTube,охват позиции интеграция в видео на YouTube и пост в сообществе YouTube,стоимость интеграция на YouTube и сторис в Инстаграм,охват позиции интеграция на YouTube и сторис в Инстаграм,стоимость усложненная интеграция,стоимость пост в сообществе,стоимость выездная интеграция,стоимость интеграция с выбором темы видео


### Двадцать восьмое объединение

In [510]:
# # Преобразуйте все столбцы в строки (str)
# avtormedia_youtube = avtormedia_youtube.astype(str)
# bufer = bufer.astype(str)

In [511]:
bufer = merge_bufer_df(bufer, avtormedia_youtube, 'YouTube')

Количество дубликатов: 0


In [512]:
# Посмотрим на размер аудитории

bufer.shape

(828, 264)

### Подготовка таблицы avtormedia_instagram

In [513]:
avtormedia_instagram.head()

,блогер,тематика,подписчики,"материалы, и статистика",охват сториз,сториз,охват постов,фотопост,карусель,охват reels,...,цена,охват,пакетное предложение,цена,охват,пакетное предложение,цена,охват,ссылка,статистика
1,даша козловская,творчество\nбьюти\nлайфстайл,2 600 000,Ссылка,155 000,690 000 ₽,281 000,1 063 829 ₽,NaN,370 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/dasha__kozlovskaya/,https://drive.google.com/drive/u/1/folders/1cW...
2,илья варламов,общество\nполитика\nпутешествия,1 300 000,Ссылка,42 000,300 000 ₽,160 000,337 000 ₽,435 000 ₽,362 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/varlamov/?hl=ru,https://drive.google.com/drive/u/1/folders/165...
3,sonya neks,бьюти\nлайфстайл\nспорт,1 100 000,Ссылка,25 000,60 000 ₽,NaN,NaN,NaN,169 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://instagram.com/sonya_neks_,https://drive.google.com/drive/folders/1e7vH1L...
4,makeupkaty,бьюти\nлайфстайл,888 000,Ссылка,137 000,147 872 ₽,228 000,260 638 ₽,337 234 ₽,178 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/makeupkaty/,https://drive.google.com/drive/u/1/folders/1oc...
5,туся,творчество\nбьюти,776 000,Ссылка,53 000,140 000 ₽,51 000,473 404 ₽,614 893 ₽,117 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/tysya/?igshid=1loov0...,https://drive.google.com/drive/u/1/folders/1Xf...


In [514]:
[col for col in avtormedia_instagram.columns]

['блогер',
 'тематика',
 'подписчики',
 'материалы, и статистика',
 'охват сториз',
 'сториз',
 'охват постов',
 'фотопост',
 'карусель',
 'охват reels',
 'reels',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'ссылка',
 'статистика']

In [515]:
[col for col in bufer.columns if 'Instagram' in col]

['Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика',
 'Instagram город',
 'Instagram среднее кол-во просмотров серии сторис',
 'Instagram стоимость серия сторис до 60 сек',
 'Instagram ца по городам',
 'Instagram ца по возрасту',
 'Instagram ца по полу']

In [516]:
avtormedia_instagram

,блогер,тематика,подписчики,"материалы, и статистика",охват сториз,сториз,охват постов,фотопост,карусель,охват reels,...,цена,охват,пакетное предложение,цена,охват,пакетное предложение,цена,охват,ссылка,статистика
1,даша козловская,творчество\nбьюти\nлайфстайл,2 600 000,Ссылка,155 000,690 000 ₽,281 000,1 063 829 ₽,NaN,370 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/dasha__kozlovskaya/,https://drive.google.com/drive/u/1/folders/1cW...
2,илья варламов,общество\nполитика\nпутешествия,1 300 000,Ссылка,42 000,300 000 ₽,160 000,337 000 ₽,435 000 ₽,362 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/varlamov/?hl=ru,https://drive.google.com/drive/u/1/folders/165...
3,sonya neks,бьюти\nлайфстайл\nспорт,1 100 000,Ссылка,25 000,60 000 ₽,NaN,NaN,NaN,169 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://instagram.com/sonya_neks_,https://drive.google.com/drive/folders/1e7vH1L...
4,makeupkaty,бьюти\nлайфстайл,888 000,Ссылка,137 000,147 872 ₽,228 000,260 638 ₽,337 234 ₽,178 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/makeupkaty/,https://drive.google.com/drive/u/1/folders/1oc...
5,туся,творчество\nбьюти,776 000,Ссылка,53 000,140 000 ₽,51 000,473 404 ₽,614 893 ₽,117 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/tysya/?igshid=1loov0...,https://drive.google.com/drive/u/1/folders/1Xf...
6,эмиль зиганша,лайфстайл\nстиль,340 000,Ссылка,40 000,130 023 ₽,55 000,236 406 ₽,513 002 ₽,85 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/zigansha/,https://drive.google.com/drive/u/1/folders/1AP...
7,дмитрий романов,юмор\nлайфстайл\nобщество,307 000,Ссылка,NaN,NaN,NaN,NaN,NaN,43 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://instagram.com/romahadima,https://drive.google.com/drive/u/1/folders/1sm...
8,диана anaidmi,лайфстайл\nмотивация\nспорт,265 000,Ссылка,45 000,100 000 ₽,88 000,140 000 ₽,172 340 ₽,100 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/anaidmi/,https://drive.google.com/drive/u/1/folders/1RF...
9,manilounalina,лайфстайл\nобщество,263 000,Ссылка,26 000,30 000 ₽,NaN,NaN,NaN,70 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://instagram.com/maniloun,https://drive.google.com/drive/folders/1O0ySrt...
10,максим кац,урбанистика\nэкономика\nобщество,189 000,Ссылка,43 000,180 000 ₽,79 000,120 000 ₽,NaN,67 000,...,686 400 ₽,943 000,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/maxim_katz/,https://drive.google.com/drive/folders/1qnIdoG...


In [518]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров 1 сторис',
                 'стоимость 1 сторис',
                 'среднее кол-во просмотров фотопоста',
                 'стоимость фотопост',
                 'стоимость карусель',
                 'среднее кол-во просмотров reels / видеопоста',
                 'стоимость reels / видеопост',
                 'пакетное предложение 1',
                 'цена 1',
                 'охват 1',
                 'пакетное предложение 2',
                 'цена 2',
                 'охват 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'пакетное предложение 4',
                 'цена 4',
                 'охват 4',
                 'ссылка',
                 'статистика']

avtormedia_instagram.columns = bufer_columns

In [ ]:
# column_list_3 = []

# for column_number in range(1, 3):
#     position_column = f'спешл {column_number}'
#     price_column = f'цена {column_number}'
#     avtormedia_instagram = get_column_values(avtormedia_instagram,
#                                              column_number,
#                                              position_column,
#                                              price_column,
#                                              column_list_3)

In [519]:
column_list_4 = []

for column_number in range(1, 5):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_instagram = get_column_values_packet(avtormedia_instagram,
                                                    column_number,
                                                    position_column,
                                                    price_column,
                                                    coverage_column,
                                                    column_list_4)

##### Обработаем список sorted_column_names_3

In [ ]:
# sorted_column_names_3 = sorted(column_list_3)
# sorted_column_names_3

In [ ]:
# avtormedia_instagram.rename(columns = {'Рекламный день (сториз 7-10 экранов)_2':'стоимость рекламный день (сториз 7-10 экранов)',
#                                        'Сториз (5 экранов)_1':'стоимость сториз (5 экранов)'}, inplace = True)

##### Обработаем список sorted_column_names_4

In [521]:
sorted_column_names_4 = sorted(column_list_4)
sorted_column_names_4

['Cторис (серия из трех) + Пост ТГ_4',
 'Интеграция в видео на YouTube + фото-пост в Инст_1',
 'Интеграция в обзор + фото в Инст_2',
 'Интеграция в рецепт + фото в Инст_1',
 'Интеграция на YouTube + Сторис в Instagram_2',
 'Пост в Telegram (удаление через сутки) + Пост в Instagram_1',
 'Пост в ТГ + сторис в ИГ_1',
 'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2',
 'Фото в Инст + сторис в Инст (серия из трех)_1',
 'Фото-пост в Инст + пост в Telegram_1',
 'Фото-пост в Инст + пост в Telegram_3']

In [522]:
dict_new_columns = {'стоимость фотопост в Инстаграм + пост в Телеграм': [ 'Фото-пост в Инст + пост в Telegram_1',
                                                                         'Фото-пост в Инст + пост в Telegram_3'],
                   
                   'охват позиции стоимость фотопост в Инстаграм + пост в Телеграм':['охват Фото-пост в Инст + пост в Telegram_1',
                                                                              'охват Фото-пост в Инст + пост в Telegram_3']}


# avtormedia_instagram = agg_columns(avtormedia_instagram, dict_new_columns)

In [523]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        avtormedia_instagram[col] = avtormedia_instagram[col].fillna('').astype(str)
    
    avtormedia_instagram[new_column] = avtormedia_instagram[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    avtormedia_instagram[new_column] = avtormedia_instagram[new_column].replace('', np.nan)
    avtormedia_instagram[col] = avtormedia_instagram[col].astype(str)

In [525]:
avtormedia_instagram.rename(columns = {'Cторис (серия из трех) + Пост ТГ_4':'стоимость серия из 3х сторис и пост в Телеграм',
                                       'Интеграция в видео на YouTube + фото-пост в Инст_1':'стоимость интеграция в видео на Ютуб и фотопост в Инстаграм',
                                       'Интеграция в обзор + фото в Инст_2':'стоимость интеграция в обзор и фотопост в Инстаграм',
                                       'Интеграция в рецепт + фото в Инст_1':'стоимость интеграция в рецепт и фотопост в Инстаграм',
                                       'Интеграция на YouTube + Сторис в Instagram_2':'стоимость интеграция на Ютуб и сторис в Инстаграм',
                                       'Пост в Telegram (удаление через сутки) + Пост в Instagram_1':'стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                       'Пост в ТГ + сторис в ИГ_1':'стоимость пост в Телеграм и сторис в Инстаграм',
                                       'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2':'стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
                                       'Фото в Инст + сторис в Инст (серия из трех)_1':'стоимость фотопост в Инстаграм + серию из 3х сторис в Инстаграм'
                                      }, inplace = True)

In [527]:
avtormedia_instagram.columns

Index(['блогер', 'тематика', 'количество подписчиков',
       'материалы и статистика', 'среднее кол-во просмотров 1 сторис',
       'стоимость 1 сторис', 'среднее кол-во просмотров фотопоста',
       'стоимость фотопост', 'стоимость карусель',
       'среднее кол-во просмотров reels / видеопоста',
       'стоимость reels / видеопост', 'охват 1', 'охват 2', 'охват 3',
       'охват 4', 'ссылка', 'статистика',
       'Фото-пост в Инст + пост в Telegram_1',
       'охват Фото-пост в Инст + пост в Telegram_1',
       'стоимость пост в Телеграм и сторис в Инстаграм',
       'охват Пост в ТГ + сторис в ИГ_1',
       'стоимость интеграция в видео на Ютуб и фотопост в Инстаграм',
       'охват Интеграция в видео на YouTube + фото-пост в Инст_1',
       'стоимость фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
       'охват Фото в Инст + сторис в Инст (серия из трех)_1',
       'стоимость интеграция в рецепт и фотопост в Инстаграм',
       'охват Интеграция в рецепт + фото в Инст_1',


In [528]:
avtormedia_instagram.drop(columns=['Фото-пост в Инст + пост в Telegram_1',
                                   'Фото-пост в Инст + пост в Telegram_3',
                                   'охват Фото-пост в Инст + пост в Telegram_1',
                                   'охват Фото-пост в Инст + пост в Telegram_3',
                                      
                                   'охват 1',
                                   'охват 2',
                                   'охват 3',
                                   'охват 4'], inplace=True)

In [529]:
[col for col in avtormedia_instagram.columns if 'охват' in col]

['охват Пост в ТГ + сторис в ИГ_1',
 'охват Интеграция в видео на YouTube + фото-пост в Инст_1',
 'охват Фото в Инст + сторис в Инст (серия из трех)_1',
 'охват Интеграция в рецепт + фото в Инст_1',
 'охват Пост в Telegram (удаление через сутки) + Пост в Instagram_1',
 'охват Интеграция на YouTube + Сторис в Instagram_2',
 'охват Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2',
 'охват Интеграция в обзор + фото в Инст_2',
 'охват Cторис (серия из трех) + Пост ТГ_4',
 'охват позиции стоимость фотопост в Инстаграм + пост в Телеграм']

In [530]:
avtormedia_instagram.rename(columns = {'охват Пост в ТГ + сторис в ИГ_1':'охват позиции стоимость за пост в Телеграм и сторис в Инстаграм',
                                       'охват Интеграция в видео на YouTube + фото-пост в Инст_1':'охват позиции стоимость за интеграцию в видео на Ютуб и фотопост в Инстаграм',
                                       'охват Фото в Инст + сторис в Инст (серия из трех)_1':'охват позиции стоимость за фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
                                       'охват Интеграция в рецепт + фото в Инст_1':'охват позиции стоимость за интеграцию в рецепт и фотопост в Инстаграм',
                                       'охват Пост в Telegram (удаление через сутки) + Пост в Instagram_1':'охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                       'охват Интеграция на YouTube + Сторис в Instagram_2':'охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм',
                                       'охват Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2':'охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
                                       'охват Интеграция в обзор + фото в Инст_2':'охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм',
                                       'охват Cторис (серия из трех) + Пост ТГ_4':'охват позиции стоимость за серию из 3х сторис и пост в Телеграм'
                                      }, inplace = True)
                                   

> Отчистим пустые строки и сделаем их как NaN

In [531]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_instagram.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_instagram[ohvat_columns] = avtormedia_instagram[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [532]:
show_duplicates(avtormedia_instagram)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,материалы и статистика,среднее кол-во просмотров 1 сторис,стоимость 1 сторис,среднее кол-во просмотров фотопоста,стоимость фотопост,стоимость карусель,среднее кол-во просмотров reels / видеопоста,...,стоимость интеграция на Ютуб и сторис в Инстаграм,охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм,"стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК","охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК",стоимость интеграция в обзор и фотопост в Инстаграм,охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм,стоимость серия из 3х сторис и пост в Телеграм,охват позиции стоимость за серию из 3х сторис и пост в Телеграм,стоимость фотопост в Инстаграм + пост в Телеграм,охват позиции стоимость фотопост в Инстаграм + пост в Телеграм


### Двадцать девятое объединение

In [533]:
bufer = merge_bufer_df(bufer, avtormedia_instagram, 'Instagram')

Количество дубликатов: 16


In [534]:
show_duplicates(bufer).head(6)

Количество дубликатов: 16


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram стоимость интеграция на Ютуб и сторис в Инстаграм,Instagram охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм,"Instagram стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК","Instagram охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК",Instagram стоимость интеграция в обзор и фотопост в Инстаграм,Instagram охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм,Instagram стоимость серия из 3х сторис и пост в Телеграм,Instagram охват позиции стоимость за серию из 3х сторис и пост в Телеграм,Instagram стоимость фотопост в Инстаграм + пост в Телеграм,Instagram охват позиции стоимость фотопост в Инстаграм + пост в Телеграм
799,makeupkaty,бьюти\nлайфстайл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
831,makeupkaty,бьюти\nлайфстайл,makeupkaty,https://www.instagram.com/makeupkaty/,бьюти\nлайфстайл,888 000,228 000,137 000,178 000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
809,manilounalina,общество,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
836,manilounalina,лайфстайл\nобщество,manilounalina,https://instagram.com/maniloun,лайфстайл\nобщество,263 000,NaN,26 000,70 000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
791,qewbite,кино,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
845,qewbite,кино\nлайфстайл,qewbite,https://instagram.com/qewbite,кино\nлайфстайл,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [535]:
# До

bufer.shape

(846, 287)

In [536]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [537]:
# После

bufer.shape

(838, 287)

In [538]:
show_duplicates(bufer)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram стоимость интеграция на Ютуб и сторис в Инстаграм,Instagram охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм,"Instagram стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК","Instagram охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК",Instagram стоимость интеграция в обзор и фотопост в Инстаграм,Instagram охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм,Instagram стоимость серия из 3х сторис и пост в Телеграм,Instagram охват позиции стоимость за серию из 3х сторис и пост в Телеграм,Instagram стоимость фотопост в Инстаграм + пост в Телеграм,Instagram охват позиции стоимость фотопост в Инстаграм + пост в Телеграм


### Подготовка таблицы avtormedia_telegram

In [541]:
avtormedia_telegram.head(1)

,блогер,тематика,подписчики,"материалы, и статистика",средние просмотры,короткое видео,пост,спешл,цена,спешл,цена,пакетное предложение,цена,охват,пакетное предложение,цена,охват,ссылка,статистика ссылка
1,илья варламов @varlamov_news,новости,1 200 000,Ссылка,470 000,833 333 ₽,450 000 ₽,Сторис,500 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov_news,https://drive.google.com/drive/u/1/folders/165...


In [542]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров поста',
                 'стоимость видеопост',
                 'стоимость фотопост',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'пакетное предложение 4',
                 'цена 4',
                 'охват 4',
                 'ссылка',
                 'статистика']

avtormedia_telegram.columns = bufer_columns

In [543]:
column_list_5 = []

for column_number in range(1, 3):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_telegram = get_column_values(avtormedia_telegram,
                                            column_number,
                                            position_column,
                                            price_column,
                                            column_list_5)

In [544]:
column_list_6 = []

for column_number in range(3, 5):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_telegram = get_column_values_packet(avtormedia_telegram,
                                                   column_number,
                                                   position_column,
                                                   price_column,
                                                   coverage_column,
                                                   column_list_6)

##### Обработаем список sorted_column_names_5

In [545]:
sorted_column_names_5 = sorted(column_list_5)
sorted_column_names_5

['Авторский пост_1',
 'Кружок_1',
 'Кружок_2',
 'Пост 2/48_2',
 'Пост 3/72_1',
 'Пост без удаления_1',
 'Сторис_1']

In [547]:
dict_new_columns = {'стоимость кружочек до 60 сек': ['Кружок_1',
                                                     'Кружок_2']
                   }

avtormedia_telegram = agg_columns(avtormedia_telegram, dict_new_columns)

In [548]:
avtormedia_telegram.drop(columns=['Кружок_1',
                                   'Кружок_2'], inplace=True)

In [549]:
avtormedia_telegram.rename(columns = {'Авторский пост_1':'стоимость за авторский пост',
                                      'Пост без удаления_1':'стоимость за пост из удаления',
                                      'Пост 2/48_2':'стоимость за пост (2/48)',
                                      'Пост 3/72_1':'стоимость за пост (3/72)',
                                      'Сторис_1':'стоимость 1 сторис до 15 сек'}, inplace = True)

##### Обработаем список sorted_column_names_6

In [550]:
sorted_column_names_6 = sorted(column_list_6)
sorted_column_names_6

['3 поста в TG_3',
 '5 постов в Telegram (удаление через сутки)_3',
 'Cторис (серия из трех) + Пост ТГ_4',
 'Пост в Telegram (удаление через сутки) + Пост в Instagram_4',
 'Пост в ТГ + сторис в ИГ_3',
 'Рекламное сообщение во все чаты "Вместе"_3',
 'Спецпроект на YouTube + пост в Telegram_3',
 'Фото-пост в Инст + пост в Telegram_3']

In [551]:
avtormedia_telegram.rename(columns = {'3 поста в TG_3':'стоимость 3 поста в Телеграм',
                                      '5 постов в Telegram (удаление через сутки)_3':'стоимость 5 постов в Телеграм (удаление через сутки)',
                                      'Cторис (серия из трех) + Пост ТГ_4':'стоимость серия из 3х сторис в Инстаграм и пост в Телеграм',
                                      'Пост в Telegram (удаление через сутки) + Пост в Instagram_4':'стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                      'Пост в ТГ + сторис в ИГ_3':'стоимость пост в Телеграм и сториз в Инстаграм',
                                      'Рекламное сообщение во все чаты "Вместе"_3':'стоимость рекламное сообщение во все чаты "Вместе"',
                                      'Спецпроект на YouTube + пост в Telegram_3':'стоимость спецпроект на Ютуб и пост в Телеграм',
                                      'Фото-пост в Инст + пост в Telegram_3':'стоимость фотопост в Инстаграм и пост в Телеграм'
                                     }, inplace = True)

In [552]:
avtormedia_telegram.rename(columns = {'охват Фото-пост в Инст + пост в Telegram_3':'охват позиции стоимость за фотопост в Инстаграм и пост в Телеграм',
                                      'охват Спецпроект на YouTube + пост в Telegram_3':'охват позиции стоимость за спецпроект на Ютуб и пост в Телеграм',
                                      'охват Пост в ТГ + сторис в ИГ_3':'охват позиции стоимость за пост в Телеграм и сториз в Инстаграм',
                                      'охват 5 постов в Telegram (удаление через сутки)_3':'охват позиции стоимость за 5 постов в Телеграм (удаление через сутки)',
                                      'охват 3 поста в TG_3':'охват позиции стоимость за 3 поста в Телеграм',
                                      'охват Рекламное сообщение во все чаты "Вместе"_3':'охват позиции стоимость за рекламное сообщение во все чаты "Вместе"',
                                      'охват Пост в Telegram (удаление через сутки) + Пост в Instagram_4':'охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                      'охват Cторис (серия из трех) + Пост ТГ_4':'охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм'
                                     }, inplace = True)

In [554]:
avtormedia_telegram.drop(columns=['материалы и статистика',
                                  'охват 3',
                                  'охват 4'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [555]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_telegram.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_telegram[ohvat_columns] = avtormedia_telegram[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [556]:
show_duplicates(avtormedia_telegram)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,среднее кол-во просмотров поста,стоимость видеопост,стоимость фотопост,ссылка,статистика,стоимость 1 сторис до 15 сек,стоимость за авторский пост,...,охват позиции стоимость за 5 постов в Телеграм (удаление через сутки),стоимость 3 поста в Телеграм,охват позиции стоимость за 3 поста в Телеграм,"стоимость рекламное сообщение во все чаты ""Вместе""","охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм,стоимость кружочек до 60 сек


### Тридцатое объединение

In [557]:
bufer = merge_bufer_df(bufer, avtormedia_telegram, 'Telegram')

Количество дубликатов: 40


In [558]:
# duplicates = bufer[bufer.duplicated('блогер', keep=False)]
# duplicates.sort_values(by='блогер').head(6)

show_duplicates(bufer).head(6)

Количество дубликатов: 40


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Telegram стоимость 5 постов в Телеграм (удаление через сутки),Telegram охват позиции стоимость за 5 постов в Телеграм (удаление через сутки),Telegram стоимость 3 поста в Телеграм,Telegram охват позиции стоимость за 3 поста в Телеграм,"Telegram стоимость рекламное сообщение во все чаты ""Вместе""","Telegram охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм
861,demin's lounge,техно-блог,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,50 000 ₽,30 000,NaN,NaN,NaN,NaN,NaN,NaN
798,demin's lounge,техно-блог,None,None,техно-блог,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854,droider,техно-блог,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
797,droider,техно-блог,None,None,техно-блог,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844,himan,челленджи\nэксперименты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
781,himan,"челленджи, эксперименты",None,None,"челленджи, эксперименты",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [559]:
# До

bufer.shape

(865, 308)

In [560]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [561]:
# После

bufer.shape

(845, 308)

In [562]:
show_duplicates(bufer)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Telegram стоимость 5 постов в Телеграм (удаление через сутки),Telegram охват позиции стоимость за 5 постов в Телеграм (удаление через сутки),Telegram стоимость 3 поста в Телеграм,Telegram охват позиции стоимость за 3 поста в Телеграм,"Telegram стоимость рекламное сообщение во все чаты ""Вместе""","Telegram охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм


### Подготовка таблицы avtormedia_vk

In [563]:
avtormedia_vk.head()

,блогер,тематика,подписчики,"материалы, и статистика",средние просмотры,пост,vk-клип,сторис,пакетное предложение,цена,охват,ссылка,статистика
1,илья варламов,общество\nурбанистика\nполитика,301 000,Ссылка,8 000,87 777 ₽,125 000 ₽,37 500 ₽,NaN,NaN,NaN,https://vk.com/varlamov,https://drive.google.com/drive/u/1/folders/165...
2,qewbite,кино,241 000,Ссылка,20 000,30 000 ₽,NaN,NaN,NaN,NaN,NaN,https://vk.com/qewbite,https://drive.google.com/drive/u/1/folders/1wo...
3,himan,челленджи\nэксперименты,227 000,Ссылка,40 000,60 000 ₽,NaN,NaN,NaN,NaN,NaN,https://vk.com/hibestman,https://drive.google.com/drive/folders/1AG9et9...
4,onetwo,скетчи\nюмор,221 000,Ссылка,15 000,38 297 ₽,53 191 ₽,27 659 ₽,NaN,NaN,NaN,https://vk.com/onetwo_pub,https://drive.google.com/drive/u/1/folders/1HC...
5,тим ильясов,лайфстайл\nмода,105 000,Ссылка,35 000,35 000 ₽,105 000 ₽,NaN,Фото в Инст + сторис в Инст (серия из трех) + ...,105 000 ₽,57 000,https://vk.com/tim_iliasov,https://drive.google.com/drive/u/1/folders/1Uz...


In [564]:
[col for col in bufer.columns if 'VK группа' in col]

['VK группа блогер',
 'VK группа ссылка',
 'VK группа тематика',
 'VK группа количество подписчиков',
 'VK группа среднее кол-во просмотров поста',
 'VK группа среднее кол-во просмотров вк клипа',
 'VK группа среднее кол-во лайков',
 'VK группа среднее кол-во комментариев',
 'VK группа стоимость фотопост',
 'VK группа стоимость видеопост',
 'VK группа стоимость вк клип',
 'VK группа статистика',
 'VK группа город',
 'VK группа стоимость передачи прав на 1 месяц',
 'VK группа ца по городам',
 'VK группа ца по возрасту',
 'VK группа ца по полу']

In [565]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров поста',
                 'стоимость вк клип',
                 'стоимость сторис',
                 'стоимость фотопост',
                 'пакетное предложение 1',
                 'цена 1',
                 'охват 1',
                 'ссылка',
                 'статистика']

avtormedia_vk.columns = bufer_columns

In [566]:
column_list_7 = []

for column_number in range(1, 2):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_vk = get_column_values_packet(avtormedia_vk,
                                             column_number,
                                             position_column,
                                             price_column,
                                             coverage_column,
                                             column_list_7)

##### Обработаем список sorted_column_names_7

In [567]:
sorted_column_names_7 = sorted(column_list_7)
sorted_column_names_7

['Сайт + соцсети (публикации на Droider.ru, FB, Twitter, Вконтакте)_1',
 'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_1']

In [568]:
avtormedia_vk.rename(columns = {'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_1':'стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
                                'Сайт + соцсети (публикации на Droider.ru, FB, Twitter, Вконтакте)_1':'стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
                                'охват Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_1':'охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
                                'охват Сайт + соцсети (публикации на Droider.ru, FB, Twitter, Вконтакте)_1':'охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
                               }, inplace = True)

In [570]:
[col for col in avtormedia_vk.columns]

avtormedia_vk.drop(columns=['материалы и статистика',
                            'охват 1'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [571]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_vk.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_vk[ohvat_columns] = avtormedia_vk[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [572]:
show_duplicates(avtormedia_vk)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,среднее кол-во просмотров поста,стоимость вк клип,стоимость сторис,стоимость фотопост,ссылка,статистика,"стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)","охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)"


### Тридцать первое объединение

In [573]:
bufer = merge_bufer_df(bufer, avtormedia_vk, 'VK группа')

Количество дубликатов: 18


In [574]:
show_duplicates(bufer)

Количество дубликатов: 18


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,"Telegram охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм,VK группа стоимость сторис,"VK группа стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","VK группа охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","VK группа стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)","VK группа охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)"
850,droider,техно-блог,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,142 553 ₽,NaN,NaN,193 262 ₽,157 500
798,droider,техно-блог,None,None,техно-блог,None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
781,himan,"челленджи, эксперименты",None,None,"челленджи, эксперименты",None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
847,himan,челленджи\nэксперименты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
791,onetwo,"скетчи, юмор",None,None,"скетчи, юмор",None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
848,onetwo,скетчи\nюмор,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,53 191 ₽,NaN,NaN,NaN,NaN
795,qewbite,"кино, лайфстайл",qewbite,https://instagram.com/qewbite,"кино, лайфстайл",,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
846,qewbite,кино,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854,дюшес,челленджи\nэксперименты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,70 000 ₽,NaN,NaN,NaN,NaN
784,дюшес,"челленджи, эксперименты, юмор",None,None,"челленджи, эксперименты, юмор",None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN


In [575]:
# До

bufer.shape

(855, 313)

In [576]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [577]:
# После

bufer.shape

(846, 313)

In [578]:
show_duplicates(bufer)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,"Telegram охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм,VK группа стоимость сторис,"VK группа стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","VK группа охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","VK группа стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)","VK группа охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)"


### Подготовка таблицы avtormedia_zen

In [579]:
avtormedia_zen.head()

,блогер,тематика,подписчики,"материалы, и статистика",средние просмотры,пост,интеграция в видео,статья,видео,спешл,цена,спешл,цена,пакетное предложение,цена,охват,ссылка,статистика
1,популярная наука,научпоп,432 000,Ссылка,35 000,90 425 ₽,NaN,90 425 ₽,101 063 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://dzen.ru/nauka,https://drive.google.com/drive/u/1/folders/13B...
2,андрей бунбич,кулинария,246 000,Ссылка,25 000,71 276 ₽,36 170 ₽,71 276 ₽,82 978 ₽,Модерация комментариев,29 787 ₽,Выездная интеграция в видео,47 872 ₽,Статья в Яндекс.Дзен + фото-пост в Инст,78 368 ₽,25 000,https://dzen.ru/bunbich,https://drive.google.com/drive/folders/1-ZzMVf...


In [581]:
[col for col in bufer.columns if 'Дзен' in col]

['Дзен блогер',
 'Дзен количество подписчиков',
 'Дзен среднее кол-во просмотров',
 'Дзен стоимость пост',
 'Дзен ссылка',
 'Дзен город',
 'Дзен название канала',
 'Дзен тематика',
 'Дзен стоимость видеопост']

In [580]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров',
                 'стоимость интеграция до 1 мин',
                 'стоимость пост',
                 'стоимость статья',
                 'стоимость за видеопост',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'ссылка',
                 'статистика']

avtormedia_zen.columns = bufer_columns

In [582]:
column_list_8 = []

for column_number in range(1, 3):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_zen = get_column_values(avtormedia_zen,
                                       column_number,
                                       position_column,
                                       price_column,
                                       column_list_8)

In [583]:
column_list_9 = []

for column_number in range(3, 4):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_zen = get_column_values_packet(avtormedia_zen,
                                              column_number,
                                              position_column,
                                              price_column,
                                              coverage_column,
                                              column_list_9)

##### Обработаем список sorted_column_names_8

In [584]:
sorted_column_names_8 = sorted(column_list_8)
sorted_column_names_8

['Выездная интеграция в видео_2', 'Модерация комментариев_1']

In [585]:
avtormedia_zen.rename(columns = {'Выездная интеграция в видео_2':'стоимость за выездную интеграцию в видео',
                                 'Модерация комментариев_1':'стоимость за модерацию комментариев'}, inplace = True)

##### Обработаем список sorted_column_names_9

In [586]:
sorted_column_names_9 = sorted(column_list_9)
sorted_column_names_9

['Статья в Яндекс.Дзен + фото-пост в Инст_3']

In [587]:
avtormedia_zen.rename(columns = {'Статья в Яндекс.Дзен + фото-пост в Инст_3':'стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм',
                                 'охват Статья в Яндекс.Дзен + фото-пост в Инст_3':'охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм'}, inplace = True)

In [588]:
avtormedia_zen.drop(columns=['материалы и статистика',
                             'охват 3'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [589]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_zen.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_zen[ohvat_columns] = avtormedia_zen[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [590]:
show_duplicates(avtormedia_zen)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,среднее кол-во просмотров,стоимость интеграция до 1 мин,стоимость пост,стоимость статья,стоимость за видеопост,ссылка,статистика,стоимость за модерацию комментариев,стоимость за выездную интеграцию в видео,стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм,охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм


### Тридцать второе объединение

In [591]:
bufer = merge_bufer_df(bufer, avtormedia_zen, 'Дзен')

Количество дубликатов: 4


In [592]:
show_duplicates(bufer)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,"VK группа стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)","VK группа охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)",Дзен стоимость интеграция до 1 мин,Дзен стоимость статья,Дзен стоимость за видеопост,Дзен статистика,Дзен стоимость за модерацию комментариев,Дзен стоимость за выездную интеграцию в видео,Дзен стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм,Дзен охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм
841,андрей бунбич,"кулинария, лайфстайл",андрей бунбич,https://www.instagram.com/bunbich/,"кулинария, лайфстайл",94 500,None,None,6 000,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
847,андрей бунбич,кулинария,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,71 276 ₽,71 276 ₽,82 978 ₽,https://drive.google.com/drive/folders/1-ZzMVf...,29 787 ₽,47 872 ₽,78 368 ₽,25 000
808,популярная наука,научпоп,None,None,научпоп,None,None,None,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
846,популярная наука,научпоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,90 425 ₽,90 425 ₽,101 063 ₽,https://drive.google.com/drive/u/1/folders/13B...,NaN,NaN,NaN,NaN


In [593]:
# До

bufer.shape

(848, 321)

In [594]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [595]:
# После

bufer.shape

(846, 321)

### Подготовка таблицы avtormedia_podcast

In [596]:
avtormedia_podcast.head()

,блогер,тематика,подписчики,"материалы, и статистика",средние просмотры,рекламная вставка,партнерская рубрика,ссылка,статистика
1,droidercast,техно-блог,,Ссылка,NaN,66 193 ₽,106 382 ₽,https://droidercast.podster.fm/,https://drive.google.com/drive/u/1/folders/1dl...


In [597]:
[col for col in avtormedia_podcast.columns]

['блогер',
 'тематика',
 'подписчики',
 'материалы, и статистика',
 'средние просмотры',
 'рекламная вставка',
 'партнерская рубрика',
 'ссылка',
 'статистика']

In [598]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во прослушиваний',
                 'стоимость рекламная вставка',
                 'стоимость партнерская рубрика',
                 'ссылка',
                 'статистика']

avtormedia_podcast.columns = bufer_columns

In [599]:
avtormedia_podcast.drop(columns='материалы и статистика', inplace=True)

### Тридцать третье объединение

In [600]:
bufer = merge_bufer_df(bufer, avtormedia_podcast, 'Подкаст')

Количество дубликатов: 0


In [601]:
# После

bufer.shape

(847, 329)

In [602]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('reklama@avtormedia.ru', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('Стоимость указана с налогами', inplace=True)

## Подготовка данных по блогерам от PLATFORMA.COMMUNITY

In [603]:
key_word = ['Вид контракта', 'CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ', до учета налогов'  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Канал']

platformacommunity_youtube = pretraining_df(platformacommunity_youtube, key_word, drop_word, upd_columns)

### Подготовка таблицы platformacommunity_youtube

In [604]:
platformacommunity_youtube.head(10)

,канал,"проект, ведущий, формат",тематика,подписчики,средние просмотры для рекламных интеграций,статистика,"pre-roll / olv, от, руб.","mid-roll / olv, от, руб.","интеграция 1-ый слот, от, руб.","интеграция 2-ой слот, от, руб.","продакт плейсмент, от, руб.","размещение в сообществе, от, руб.",пример,ссылка,статистика ссылка
0,уральские пельмени,Официальный канал шоу «Уральские Пельмени»,Специальные сборки номеров под рекламодателя,"2 550 000,00","2 000 000,00",Статистика,"1 200 000,00",NaN,"2 500 000,00",NaN,NaN,NaN,Тинькофф,https://www.youtube.com/c/ProPelmeny/featured,https://drive.google.com/drive/folders/1vQIyBd...
1,NaN,NaN,Эксклюзивные номера,NaN,"1 000 000,00",NaN,"700 000,00",NaN,NaN,NaN,NaN,NaN,Пятерочка,NaN,NaN
2,NaN,NaN,Регулярные выкладки,NaN,"700 000,00",NaN,"550 000,00",NaN,NaN,NaN,NaN,NaN,HAVAL,NaN,NaN
3,NaN,NaN,"""ЕДЕМ С ПЕЛЬМЕНЯМИ"" travel шоу про гастроли и ...",NaN,"300 000,00",NaN,NaN,NaN,NaN,NaN,"800 000,00",NaN,Орматек,NaN,NaN
4,вписка,Коля Редькин,Интервью,"1 830 000,00","2 500 000,00",Статистика,"1 500 000,00",по запросу,по запросу,NaN,по запросу,NaN,NaN,https://www.youtube.com/@VPISKA/featured,https://drive.google.com/drive/folders/1KxM-Mc...
5,истории русской попсы,Олег Кармунин,"Музыкальный критик и автор телеграм-канала ""Ру...","149 000,00","150 000,00",Статистика,"100 000,00","80 000,00","130 000,00","100 000,00",NaN,NaN,NaN,https://www.youtube.com/@ruspopshow/videos,https://drive.google.com/drive/folders/1N2NTA5...
6,кликклак,"Руслан Усачев, Эльдар Джарахов, Юрий Музыченко...","Канал друзей, где не прекращаются трэш, угар и...","6 940 000,00","2 000 000,00",Статистика,"1 650 000,00",NaN,"3 300 000,00",NaN,NaN,NaN,NaN,https://www.youtube.com/@klikklak,https://drive.google.com/drive/folders/1II3xtx...
7,джарахов,Эльдар Джарахов,"Популярный блогер, комик, музыкант","3 680 000,00","500 000,00",Статистика,"440 000,00",NaN,"660 000,00",NaN,NaN,NaN,NaN,https://www.youtube.com/@dzharakhov,https://drive.google.com/drive/folders/14Initg...
8,брокен дэнс,Николай Редькин – российский музыкальный журна...,Видео-эссе о современной зарубежной и российск...,"200 000,00","200 000,00",Статистика,NaN,NaN,"300 000,00","150 000,00",NaN,NaN,NaN,https://www.youtube.com/@broken_dance,https://drive.google.com/drive/folders/1lXd1YR...
9,prnrp,"Даня Порнорэп - музыкальный критик, журналист ...",Анализ современной музыки на стыке искусствове...,"20 000,00","50 000,00",Статистика,NaN,NaN,"50 000,00","30 000,00",NaN,NaN,NaN,https://www.youtube.com/@trap_critic,https://drive.google.com/drive/folders/1iUO5CB...


> Заполним имеющиеся пропуски "сверху вниз"

In [605]:
platformacommunity_youtube[['канал', 'проект, ведущий, формат', 'подписчики', 'ссылка', 'статистика ссылка']] = \
            platformacommunity_youtube[['канал', 'проект, ведущий, формат', 'подписчики', 'ссылка', 'статистика ссылка']].fillna(method='ffill')

> Удалим не нужные нам столбцы

In [606]:
platformacommunity_youtube.drop(columns=['статистика', 'пример'], inplace=True)

In [607]:
platformacommunity_youtube.columns

Index(['канал', 'проект, ведущий, формат', 'тематика', 'подписчики',
       'средние просмотры для рекламных интеграций',
       'pre-roll / olv, от, руб.', 'mid-roll / olv, от, руб.',
       'интеграция 1-ый слот, от, руб.', 'интеграция 2-ой слот, от, руб.',
       'продакт плейсмент, от, руб.', 'размещение в сообществе, от, руб.',
       'ссылка', 'статистика ссылка'],
      dtype='object')

> На данный момент мы не будем объединять столбцы 'канал' и 'проект, ведущий, формат', так как тогда мы не сможем правильно объединить

In [608]:
# platformacommunity_youtube['блогер'] = platformacommunity_youtube.apply(
#     lambda row: row['канал'] + ' - ' + row['проект, ведущий, формат'] if len(row['проект, ведущий, формат']) < 40 else '',
#     axis=1
# )

In [609]:
# platformacommunity_youtube['блогер'] = \
#                 platformacommunity_youtube['канал'] + ' - ' + platformacommunity_youtube['проект, ведущий, формат']

# platformacommunity_youtube.drop(columns=['статистика аудитории',
#                                          'проект, ведущий, формат'], axis=1, inplace= True)

In [610]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость интеграция для профильных клиентов 1 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 1 слот',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость интеграция для непрофильных клиентов 2 слот',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'YouTube город',
 'YouTube название канала

In [611]:
[col for col in platformacommunity_youtube.columns]

['канал',
 'проект, ведущий, формат',
 'тематика',
 'подписчики',
 'средние просмотры для рекламных интеграций',
 'pre-roll / olv, от, руб.',
 'mid-roll / olv, от, руб.',
 'интеграция 1-ый слот, от, руб.',
 'интеграция 2-ой слот, от, руб.',
 'продакт плейсмент, от, руб.',
 'размещение в сообществе, от, руб.',
 'ссылка',
 'статистика ссылка']

In [612]:
bufer_columns = ['блогер',
                 'проект, ведущий, формат',
                 'шоу',
                 'количество подписчиков',
                 'среднее кол-во просмотров',
                 'стоимость преролл до 30 сек',
                 'стоимость мидролл до 30 сек',
                 'стоимость интеграция 1 слот',
                 'стоимость интеграция 2 слот',
                 'стоимость продакт плейсмент',
                 'стоимость пост в сообществе',
                 'ссылка',
                 'статистика']

In [613]:
platformacommunity_youtube.columns = bufer_columns

### Тридцать четвёртое объединение

In [614]:
platformacommunity_youtube.loc\
    [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

platformacommunity_youtube.loc\
    [platformacommunity_youtube['блогер'] == 'стс', 'блогер']  = 'телеканал стс'

# platformacommunity_youtube.loc\
#     [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

# platformacommunity_youtube.loc\
#     [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

In [615]:
bufer = merge_bufer_df(bufer, platformacommunity_youtube, 'YouTube')

Количество дубликатов: 48


In [616]:
show_duplicates(bufer)

Количество дубликатов: 48


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Подкаст блогер,Подкаст тематика,Подкаст количество подписчиков,Подкаст среднее кол-во прослушиваний,Подкаст стоимость рекламная вставка,Подкаст стоимость партнерская рубрика,Подкаст ссылка,Подкаст статистика,"YouTube проект, ведущий, формат",YouTube шоу
923,hardcore media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Epic Fighting,"Лицом к лицу сойдутся звезды интернета, популя..."
922,hardcore media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hardcore MMA,Профессиональный Российский промоушен смешанны...
921,hardcore media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hardcore Fighting Championship,Первый в России и крупнейший в мире спортивный...
906,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Разгоны\r,NaN
910,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НеТурист,NaN
909,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Офис,NaN
908,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Что у вас случилось?,NaN
907,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Книжный клуб,NaN
905,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Самый Умный Комик\r,Юмор
895,александр шебанов,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ежедневный выпуск лучших мемов,NaN


### Подготовка таблицы platformacommunity_instagram

In [617]:
platformacommunity_instagram

,Персоналии,Страница,Подписчики,Тематика,Статистика и инфо,Цены,Unnamed: 6,Unnamed: 7,"ATL активации, предоставление артиста в рекламные кампании"
0,NaN,NaN,NaN,NaN,NaN,Фотопост,Сторис,Видеопост,NaN
1,NaN,NaN,NaN,NaN,NaN,От,От,От,NaN
2,LOVV66,https://instagram.com/lovv66,"634 000,00","Рэпер. Один из главных фрешменов, по мнению ТН...",NaN,"300 000,00","120 000,00","300 000,00",по запросу
3,SCALLY MILANO,https://instagram.com/scallymilano,"456 000,00",Рэпер. Дмитриев. Считается одним из первых в Р...,NaN,"400 000,00","140 000,00","400 000,00",по запросу
4,TENDERLYBAE,https://instagram.com/tenderlybae,"3 900 000,00",Стримерша и певица. В 2021 году Амина победила...,NaN,"300 000,00","150 000,00","400 000,00",по запросу
5,PINQ,https://instagram.com/pinq,"153 000,00",Рэп-исполнитель и дизайнер,NaN,"75 000,00","40 000,00","75 000,00",по запросу
6,ШАЙНИ,https://instagram.com/shynegswagga,"143 000,00",Рэп-исполнитель,NaN,"45 000,00","25 000,00","90 000,00",по запросу
7,ЯНА ЛОГВИНА,https://www.instagram.com/logvina.ya/,"686 000,00",Трансформационный тренер,NaN,"150 000,00","120 000,00","200 000,00",по запросу
8,ЗОЯ ЗИНОВЬЕВА,https://instagram.com/zzoya_diary?igshid=NTc4M...,"65 000,00","Актриса, артист-вокалист",NaN,"60 000,00","50 000,00","100 000,00",по запросу
9,Анастасия Исаева,https://instagram.com/nnassie?igshid=MzRlODBiN...,29 000,Медийный коуч,NaN,"15 000,00","15 000,00","20 000,00",по запросу


In [618]:
platformacommunity_instagram.drop(columns=['Статистика и инфо',
                                           'ATL активации, предоставление артиста в рекламные кампании'], inplace=True)

In [619]:
platformacommunity_instagram['Персоналии'] = platformacommunity_instagram['Персоналии'].str.lower()

In [620]:
platformacommunity_instagram.columns

Index(['Персоналии', 'Страница', 'Подписчики', 'Тематика', 'Цены ',
       'Unnamed: 6', 'Unnamed: 7'],
      dtype='object')

In [621]:
[col for col in bufer.columns if 'Instagram' in col]

['Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика',
 'Instagram город',
 'Instagram среднее кол-во просмотров серии сторис',
 'Instagram стоимость серия сторис до 60 сек',
 'Instagram ца по городам',
 'Instagram ца по возрасту',
 'Instagram ца по полу',
 'Instagram материалы и статистика',
 'Instagram стоимость 1 сторис',
 'Instagram стоимость карусель',
 'Instagram стоимость пост в Телеграм и сторис в Инстаграм',
 'Instagram охват позиции стоимость за пост в Телеграм и сторис в Инстаграм',
 'Instagram стоимость 

In [622]:
bufer_columns = ['блогер',
                 'ссылка',
                 'количество подписчиков',
                 'тематика',
                 'стоимость фотопост',
                 'стоимость 1 сторис',
                 'стоимость reels / видеопост']

platformacommunity_instagram.columns = bufer_columns

In [623]:
platformacommunity_instagram

,блогер,ссылка,количество подписчиков,тематика,стоимость фотопост,стоимость 1 сторис,стоимость reels / видеопост
0,NaN,NaN,NaN,NaN,Фотопост,Сторис,Видеопост
1,NaN,NaN,NaN,NaN,От,От,От
2,lovv66,https://instagram.com/lovv66,"634 000,00","Рэпер. Один из главных фрешменов, по мнению ТН...","300 000,00","120 000,00","300 000,00"
3,scally milano,https://instagram.com/scallymilano,"456 000,00",Рэпер. Дмитриев. Считается одним из первых в Р...,"400 000,00","140 000,00","400 000,00"
4,tenderlybae,https://instagram.com/tenderlybae,"3 900 000,00",Стримерша и певица. В 2021 году Амина победила...,"300 000,00","150 000,00","400 000,00"
5,pinq,https://instagram.com/pinq,"153 000,00",Рэп-исполнитель и дизайнер,"75 000,00","40 000,00","75 000,00"
6,шайни,https://instagram.com/shynegswagga,"143 000,00",Рэп-исполнитель,"45 000,00","25 000,00","90 000,00"
7,яна логвина,https://www.instagram.com/logvina.ya/,"686 000,00",Трансформационный тренер,"150 000,00","120 000,00","200 000,00"
8,зоя зиновьева,https://instagram.com/zzoya_diary?igshid=NTc4M...,"65 000,00","Актриса, артист-вокалист","60 000,00","50 000,00","100 000,00"
9,анастасия исаева,https://instagram.com/nnassie?igshid=MzRlODBiN...,29 000,Медийный коуч,"15 000,00","15 000,00","20 000,00"


In [624]:
platformacommunity_instagram[platformacommunity_instagram['блогер'] == 'полина маришова']

,блогер,ссылка,количество подписчиков,тематика,стоимость фотопост,стоимость 1 сторис,стоимость reels / видеопост
27,полина маришова,instagram,223 000,Тренер по речи,Только комплексный вид рекламы 6-7 сторис + по...,NaN,NaN


In [625]:
platformacommunity_instagram = platformacommunity_instagram.dropna(subset=['блогер'])

In [626]:
platformacommunity_instagram.loc[platformacommunity_instagram['блогер'] == 'рита дакота', 'стоимость видеопост и серия сторис'] = 750000
platformacommunity_instagram.loc[platformacommunity_instagram['блогер'] == 'полина маришова', 'стоимость фотопост и 6-7 сторис'] = 600000

In [627]:
platformacommunity_instagram['стоимость фотопост'] = \
platformacommunity_instagram['стоимость фотопост'].replace('Видео пост + серия сторис 750 000', np.nan)

platformacommunity_instagram['стоимость фотопост'] = \
            platformacommunity_instagram['стоимость фотопост'].replace('Только комплексный вид рекламы 6-7 сторис + пост 600 000', np.nan)

In [628]:
platformacommunity_instagram

,блогер,ссылка,количество подписчиков,тематика,стоимость фотопост,стоимость 1 сторис,стоимость reels / видеопост,стоимость видеопост и серия сторис,стоимость фотопост и 6-7 сторис
2,lovv66,https://instagram.com/lovv66,"634 000,00","Рэпер. Один из главных фрешменов, по мнению ТН...","300 000,00","120 000,00","300 000,00",NaN,NaN
3,scally milano,https://instagram.com/scallymilano,"456 000,00",Рэпер. Дмитриев. Считается одним из первых в Р...,"400 000,00","140 000,00","400 000,00",NaN,NaN
4,tenderlybae,https://instagram.com/tenderlybae,"3 900 000,00",Стримерша и певица. В 2021 году Амина победила...,"300 000,00","150 000,00","400 000,00",NaN,NaN
5,pinq,https://instagram.com/pinq,"153 000,00",Рэп-исполнитель и дизайнер,"75 000,00","40 000,00","75 000,00",NaN,NaN
6,шайни,https://instagram.com/shynegswagga,"143 000,00",Рэп-исполнитель,"45 000,00","25 000,00","90 000,00",NaN,NaN
7,яна логвина,https://www.instagram.com/logvina.ya/,"686 000,00",Трансформационный тренер,"150 000,00","120 000,00","200 000,00",NaN,NaN
8,зоя зиновьева,https://instagram.com/zzoya_diary?igshid=NTc4M...,"65 000,00","Актриса, артист-вокалист","60 000,00","50 000,00","100 000,00",NaN,NaN
9,анастасия исаева,https://instagram.com/nnassie?igshid=MzRlODBiN...,29 000,Медийный коуч,"15 000,00","15 000,00","20 000,00",NaN,NaN
10,ольяная екатерина,https://www.instagram.com/ekaterina.olyanaya,60 000,"Таролог, мотивационный спикер","20 000,00","15 000,00","30 000,00",NaN,NaN
11,света солар,https://instagram.com/svetasollar?igshid=1aj82...,1 300 000,"Блогер лайфстайл, юмор",70 000,50 000,80 000,NaN,NaN


In [629]:
show_duplicates(platformacommunity_instagram)

Количество дубликатов: 0


,блогер,ссылка,количество подписчиков,тематика,стоимость фотопост,стоимость 1 сторис,стоимость reels / видеопост,стоимость видеопост и серия сторис,стоимость фотопост и 6-7 сторис


### Тридцать пятое объединение

In [630]:
bufer = merge_bufer_df(bufer, platformacommunity_instagram, 'Instagram')

Количество дубликатов: 48


### Подготовка таблицы avtormedia_telegram

In [631]:
platformacommunity_telegram

,Канал,Проект / ведущий / идеолог,Тематика,Подписчики,"Пост (текст) с закреплением в ТОП, до учета налогов","Пост (текст) без закрепления в ТОП, до учета налогов","Пост (текст+фото) , до учета налогов","Пост (видео) , до учета налогов","Пост (кружочек) , до учета налогов",Unnamed: 9
0,Лина Дианова,Лина Дианова,Психология,35 000,"90 000,00","60 000,00",NaN,NaN,NaN,https://t.me/lina_dianova
1,KADI,Диана Кади,Интервью,6 500,"25 000,00","20 000,00",NaN,NaN,NaN,https://t.me/kadinew
2,Серотонин,"Профессиональные журналисты глянца (GQ, Vogue,...",Lifestyle,13 000,NaN,NaN,"15 000,00","15 000,00","15 000,00",https://t.me/+i5_0knOc8SE3M2Y0
3,Лиза Лазерсон,"Феминистка, блогер, ведушая ""Эхо Москвы"" Лиза ...",Социальная и общественная публицистика,34 000,"55 000,00","35 000,00",NaN,NaN,NaN,https://t.me/znachit_net
4,Plarforma. Producer hub,Platforma Agency,Инфо о рынке digital видео спонсорства,370,NaN,по теме - бесплатно,NaN,NaN,NaN,https://t.me/digitalproducerhub
5,good news everyone,новости,новости,2 800,"15 000,00","10 000,00",NaN,NaN,NaN,https://t.me/good_news_every1
6,Позже всех. Ну почти.,новости,Новости и исторические факты каждый день,18 000,"25 000,00","15 000,00",NaN,NaN,NaN,https://t.me/lllating
7,Putin.Life,новости,"Не только актуальные новости, но и эксклюзив о...",45 000,"15 000,00","10 000,00",NaN,NaN,NaN,https://t.me/putin_life
8,От Первого Лица | ОПЛ,Новости,"Новости о том, что происходит в России: провер...",130 000,"25 000,00","15 000,00",NaN,NaN,NaN,https://t.me/otpervogolitsa
9,Mr. Putin,"СКАНДАЛЫ, ФЕЙКИ, МЕМЫ, АНАЛИТИКА",Что иностранцы на самом деле думают о нашей ст...,130 000,"25 000,00","15 000,00",NaN,NaN,NaN,https://t.me/goryachiye_novosti


In [632]:
platformacommunity_telegram['Канал'] = platformacommunity_telegram['Канал'].str.lower()

In [633]:
[col for col in bufer.columns if 'Telegram' in col]

['Telegram блогер',
 'Telegram ссылка',
 'Telegram тематика',
 'Telegram количество подписчиков',
 'Telegram среднее кол-во просмотров поста',
 'Telegram средняя вовлечённость',
 'Telegram стоимость фотопост',
 'Telegram стоимость видеопост',
 'Telegram стоимость кружочек до 60 сек',
 'Telegram стоимость текстовое сообщение без фото',
 'Telegram статистика',
 'Telegram город',
 'Telegram стоимость 1 сторис до 15 сек',
 'Telegram стоимость за авторский пост',
 'Telegram стоимость за пост из удаления',
 'Telegram стоимость за пост (3/72)',
 'Telegram стоимость за пост (2/48)',
 'Telegram стоимость фотопост в Инстаграм и пост в Телеграм',
 'Telegram охват позиции стоимость за фотопост в Инстаграм и пост в Телеграм',
 'Telegram стоимость спецпроект на Ютуб и пост в Телеграм',
 'Telegram охват позиции стоимость за спецпроект на Ютуб и пост в Телеграм',
 'Telegram стоимость пост в Телеграм и сториз в Инстаграм',
 'Telegram охват позиции стоимость за пост в Телеграм и сториз в Инстаграм',
 'T

In [634]:
[col for col in platformacommunity_telegram.columns]

['Канал',
 'Проект / ведущий / идеолог',
 'Тематика',
 'Подписчики',
 'Пост (текст) с закреплением в ТОП, до учета налогов',
 'Пост (текст) без закрепления в ТОП, до учета налогов',
 'Пост (текст+фото) , до учета налогов',
 'Пост (видео) , до учета налогов',
 'Пост (кружочек) , до учета налогов',
 'Unnamed: 9']

In [635]:
bufer_columns = ['блогер',
                 'шоу',
                 'тематика',
                 'количество подписчиков',
                 'стоимость текстовый пост с закрепом',
                 'стоимость текстовый пост без закрепа',
                 'стоимость фотопост',
                 'стоимость видеопост',
                 'стоимость кружочек до 60 сек',
                 'ссылка']

platformacommunity_telegram.columns = bufer_columns

In [636]:
platformacommunity_telegram

,блогер,шоу,тематика,количество подписчиков,стоимость текстовый пост с закрепом,стоимость текстовый пост без закрепа,стоимость фотопост,стоимость видеопост,стоимость кружочек до 60 сек,ссылка
0,лина дианова,Лина Дианова,Психология,35 000,"90 000,00","60 000,00",NaN,NaN,NaN,https://t.me/lina_dianova
1,kadi,Диана Кади,Интервью,6 500,"25 000,00","20 000,00",NaN,NaN,NaN,https://t.me/kadinew
2,серотонин,"Профессиональные журналисты глянца (GQ, Vogue,...",Lifestyle,13 000,NaN,NaN,"15 000,00","15 000,00","15 000,00",https://t.me/+i5_0knOc8SE3M2Y0
3,лиза лазерсон,"Феминистка, блогер, ведушая ""Эхо Москвы"" Лиза ...",Социальная и общественная публицистика,34 000,"55 000,00","35 000,00",NaN,NaN,NaN,https://t.me/znachit_net
4,plarforma. producer hub,Platforma Agency,Инфо о рынке digital видео спонсорства,370,NaN,по теме - бесплатно,NaN,NaN,NaN,https://t.me/digitalproducerhub
5,good news everyone,новости,новости,2 800,"15 000,00","10 000,00",NaN,NaN,NaN,https://t.me/good_news_every1
6,позже всех. ну почти.,новости,Новости и исторические факты каждый день,18 000,"25 000,00","15 000,00",NaN,NaN,NaN,https://t.me/lllating
7,putin.life,новости,"Не только актуальные новости, но и эксклюзив о...",45 000,"15 000,00","10 000,00",NaN,NaN,NaN,https://t.me/putin_life
8,от первого лица | опл,Новости,"Новости о том, что происходит в России: провер...",130 000,"25 000,00","15 000,00",NaN,NaN,NaN,https://t.me/otpervogolitsa
9,mr. putin,"СКАНДАЛЫ, ФЕЙКИ, МЕМЫ, АНАЛИТИКА",Что иностранцы на самом деле думают о нашей ст...,130 000,"25 000,00","15 000,00",NaN,NaN,NaN,https://t.me/goryachiye_novosti


### Тридцать шестое объединение

In [637]:
bufer = merge_bufer_df(bufer, platformacommunity_telegram, 'Telegram')

Количество дубликатов: 66


In [638]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('Нина Оуэн @ohnonina +7 901 761 4650', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

In [639]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Подкаст стоимость партнерская рубрика,Подкаст ссылка,Подкаст статистика,"YouTube проект, ведущий, формат",YouTube шоу,Instagram стоимость видеопост и серия сторис,Instagram стоимость фотопост и 6-7 сторис,Telegram шоу,Telegram стоимость текстовый пост с закрепом,Telegram стоимость текстовый пост без закрепа
0,катя клэп,"бьюти, лайфстайл",катя клэп,https://www.instagram.com/kateclapp/,"бьюти, лайфстайл",10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,катя мотовилова,"бьюти, лайфстайл",катя мотовилова,https://www.instagram.com/kate_motovilova/,"бьюти, лайфстайл",276 000,30 000,15 000,70 000,5 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,маша нарциссизм,"бьюти, лайфстайл",None,None,"бьюти, лайфстайл",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,строкова татьяна,"бьюти, лайфстайл",None,None,"бьюти, лайфстайл",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,рязанова татьяна,"бьюти, лайфстайл",None,None,"бьюти, лайфстайл",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,не тереби дао,Lifestyle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,видеоблогер и диджей Ксюша Дукалис,NaN,"325 000,00"
1029,тина канделаки,"Публицистика, LifeStyle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"российская телеведущая и журналистка, продюсер...",NaN,NaN
1030,super.ru,Новости шоу-бизнеса,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"российская журналистка, видеоблогер, репортер ...",NaN,NaN
1031,the flow,Лайфстайл о мире медиа и хип-хопа,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Медиа о рэпе и всём, что популярно у современ...",NaN,"100 000,00"


## Подготовка данных единого DataFrame

### Очистка данных

In [640]:
itog = bufer.replace({None: np.nan})
itog = bufer.replace('None', np.nan)

In [641]:
itog

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Подкаст стоимость партнерская рубрика,Подкаст ссылка,Подкаст статистика,"YouTube проект, ведущий, формат",YouTube шоу,Instagram стоимость видеопост и серия сторис,Instagram стоимость фотопост и 6-7 сторис,Telegram шоу,Telegram стоимость текстовый пост с закрепом,Telegram стоимость текстовый пост без закрепа
0,катя клэп,"бьюти, лайфстайл",катя клэп,https://www.instagram.com/kateclapp/,"бьюти, лайфстайл",10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,катя мотовилова,"бьюти, лайфстайл",катя мотовилова,https://www.instagram.com/kate_motovilova/,"бьюти, лайфстайл",276 000,30 000,15 000,70 000,5 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,маша нарциссизм,"бьюти, лайфстайл",NaN,NaN,"бьюти, лайфстайл",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,строкова татьяна,"бьюти, лайфстайл",NaN,NaN,"бьюти, лайфстайл",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,рязанова татьяна,"бьюти, лайфстайл",NaN,NaN,"бьюти, лайфстайл",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,не тереби дао,Lifestyle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,видеоблогер и диджей Ксюша Дукалис,NaN,"325 000,00"
1029,тина канделаки,"Публицистика, LifeStyle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"российская телеведущая и журналистка, продюсер...",NaN,NaN
1030,super.ru,Новости шоу-бизнеса,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"российская журналистка, видеоблогер, репортер ...",NaN,NaN
1031,the flow,Лайфстайл о мире медиа и хип-хопа,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Медиа о рэпе и всём, что популярно у современ...",NaN,"100 000,00"


In [646]:
[col for col in itog.columns]

['блогер',
 'тематика',
 'Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика',
 'YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouT

In [647]:
column_names_list = itog.columns.tolist()

In [649]:
spisok_columns=['Instagram количество подписчиков',
                'Instagram среднее кол-во просмотров фотопоста',
                'Instagram среднее кол-во просмотров 1 сторис',
                'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',

 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',

 'YouTube стоимость интеграция 2 слот',

 'Telegram количество подписчиков',
 'Telegram среднее кол-во просмотров поста',
 'Telegram средняя вовлечённость',
 'Telegram стоимость фотопост',
 'Telegram стоимость видеопост',
 'Telegram стоимость кружочек до 60 сек',
 'Telegram стоимость текстовое сообщение без фото',

 'Дзен Шоу количество подписчиков',
 'Дзен Шоу среднее кол-во просмотров ролика',
 'Дзен Шоу стоимость интеграция до 1 мин',
 'Дзен Шоу стоимость продакт плейсмент',
 'Дзен Шоу стоимость преролл до 30 сек',
 'Дзен Шоу стоимость мидролл до 30 сек',

 'Дзен Шоу стоимость интеграция 2 слот',

 'VK Видео количество подписчиков',
 'VK Видео среднее кол-во просмотров ролика',
 'VK Видео стоимость интеграция до 1 мин',
 'VK Видео стоимость продакт плейсмент',
 'VK Видео стоимость преролл до 30 сек',
 'VK Видео стоимость мидролл до 30 сек',

 'VK Видео стоимость интеграция 2 слот',
 'VK Видео стоимость за интеграцию до 1 мин 2 слот',

 'VK группа количество подписчиков',
 'VK группа среднее кол-во просмотров поста',
 'VK группа среднее кол-во просмотров вк клипа',
 'VK группа среднее кол-во лайков',
 'VK группа среднее кол-во комментариев',
 'VK группа стоимость фотопост',
 'VK группа стоимость видеопост',
 'VK группа стоимость вк клип',

 'Twitch количество подписчиков',
 'Twitch среднее кол-во зрителей',
 'Twitch уникальные зрители (30 дней)',
 'Twitch стоимость за баннер + ссылка в описании 1 мес',
 'Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции',
 'Twitch стоимость нативное голосовое упоминание товара в трансляции',
 'Twitch стоимость ссылка в чат-боте на 1 стрим',
 'Twitch стоимость демонстрация/обзор товара на трансляции',

 'VK Видео среднее кол-во просмотров',
 'VK Видео стоимость интеграция 1 слот',
 'VK Видео стоимость преролл до 20 сек',
 'VK Видео стоимость мидролл до 20 сек',
 'VK Видео стоимость интеграция с участием актеров 1 слот',
 'VK Видео стоимость интеграция с участием комиков из выпуска 1 слот',
 'VK Видео стоимость интеграция с участием Пушного, Крастера, Старого 1 слот',
 'VK Видео стоимость интеграция до 60 секунд 1 слот',
 'VK Видео стоимость интеграция с участием актеров 2 слот',
 'VK Видео стоимость интеграция с участием комиков из выпуска 2 слот',
 'VK Видео стоимость интеграция с участием Пушного, Крастера, Старого 2 слот',
 'VK Видео стоимость интеграция с участием блогера 1 слот',
 'VK Видео стоимость интеграция с участием Сергея Орлова 1 слот',
 'VK Видео стоимость интеграция с участием ведущих 1 слот',
 'VK Видео стоимость интеграция с участием Мамикса 1 слот',
 'VK Видео стоимость устник до 30 сек 1 слот',
 'VK Видео стоимость интеграция с участием блогера 2 слот',
 'VK Видео стоимость интеграция с участием Сергея Орлова 2 слот',
 'VK Видео стоимость интеграция с участием ведущих 2 слот',
 'OK Шоу среднее кол-во просмотров',
 'OK Шоу стоимость интеграция 1 слот',
 'OK Шоу стоимость интеграция 2 слот',
 'OK Шоу стоимость продакт плейсмент',
 'OK Шоу стоимость преролл до 20 сек',
 'OK Шоу стоимость мидролл до 20 сек',

 'YouTube среднее кол-во просмотров',
 'YouTube стоимость интеграция для профильных клиентов 1 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 1 слот',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость интеграция 4 слот',
 'YouTube стоимость интеграция 5 слот',
 'YouTube стоимость интеграция для профильных клиентов 2 слот',
 'YouTube стоимость интеграция для непрофильных клиентов 2 слот',
 'YouTube стоимость спонсорство',
 'YouTube стоимость интеграция 3 слот',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'TikTok количество подписчиков',
 'TikTok среднее кол-во просмотров',
 'TikTok стоимость видеопост',

 'Дзен количество подписчиков',
 'Дзен среднее кол-во просмотров',
 'Дзен стоимость пост',

 'Instagram среднее кол-во просмотров серии сторис',
 'Instagram стоимость серия сторис до 60 сек',

 'Threads количество подписчиков',
 'Threads стоимость фотопост',
 'Threads стоимость видеопост',

 'VK личная страница количество подписчиков',
 'VK личная страница среднее кол-во просмотров поста',
 'VK личная страница среднее кол-во просмотров вк клипа',
 'VK личная страница стоимость вк клип',
 'VK личная страница стоимость фотопост',
 'VK личная страница стоимость видеопост',
 'VK личная страница стоимость передачи прав на 1 месяц',
 'VK группа стоимость передачи прав на 1 месяц',

 'YouTube стоимость за shorts до 1 мин',

 'Дзен стоимость видеопост',

 'Likee количество подписчиков',
 'Likee среднее кол-во просмотров',
 'Likee стоимость видеопост',

 'Yappy количество подписчиков',
 'Yappy среднее кол-во просмотров',
 'Yappy стоимость видеопост',

 'RuTube среднее кол-во просмотров видео',
 'RuTube стоимость преролл до 30 сек',
 'RuTube стоимость интеграция 1 слот',

 'Twitch уникальные зрители (14 дней)',
 'Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим',
 'Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа',
 'Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)',

 'YouTube стоимость спецпроект',
 'YouTube стоимость бегущая строка',
 'YouTube стоимость интеграция в тревел-ролики',
 'YouTube стоимость преролл в формат новости',
 'YouTube стоимость повторное размещение реализованной интеграции',
 'YouTube стоимость интеграция в ролик с Линой Диановой 1 слот',
 'YouTube стоимость выездной спецпроект',
 'YouTube стоимость интеграция с участием Максима Каца',
 'YouTube стоимость интеграция в формат Стиратель на YouTube',
 'YouTube стоимость интеграция в формат «Книга жалоб» 2 слот',
 'YouTube стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 2 слот',
 'YouTube стоимость интеграция в реакцию на трейлер',
 'YouTube стоимость интеграция в ролик с Линой Диановой 2 слот',
 'YouTube стоимость преролл в рецепт',
 'YouTube стоимость интеграция в формат "Криминальный ТикТок"',
 'YouTube стоимость преролл в формат «Книга жалоб»',
 'YouTube стоимость интеграция в формат новости',
 'YouTube стоимость преролл в формат "Линия выгорания"',
 'YouTube стоимость преролл в обзор',
 'YouTube стоимость интеграция в подкаст «Сладкая жизнь» 1 слот',
 'YouTube стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот',
 'YouTube стоимость интеграция в разборы',
 'YouTube стоимость псылка в закрепленном комментарии и ссылка в описании до ката',
 'YouTube стоимость преролл в ролик с Линой Диановой',
 'YouTube стоимость интеграция в рецепт',
 'YouTube стоимость интеграция в формат «Книга жалоб» 1 слот',
 'YouTube стоимость выездная интеграция в воскресный выпуск новостей "Чё Происходит"',
 'YouTube стоимость преролл в реакцию на трейлер или разборы',
 'YouTube стоимость интеграция в обзор',
 'YouTube стоимость интеграция в формат «Спешл»',
 'YouTube стоимость интеграция в подкаст «Сладкая жизнь» 2 слот',
 'YouTube стоимость преролл в подкаст «Сладкая жизнь»',
 'YouTube стоимость 3 интеграции',
 'YouTube охват позиции 3 интеграции',
 'YouTube стоимость Cпецпроект и фотопост в Инстаграм',
 'YouTube охват позиции Cпецпроект и фотопост в Инстаграм',
 'YouTube стоимость интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
 'YouTube охват позиции интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
 'YouTube стоимость интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
 'YouTube охват позиции интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
 'YouTube стоимость интеграция в видео на YouTube и фотопост в Инстаграм',
 'YouTube охват позиции интеграция в видео на YouTube и фотопост в Инстаграм',
 'YouTube стоимость спецпроект и анонс конкурса',
 'YouTube охват позиции спецпроект и анонс конкурса',
 'YouTube стоимость интеграция в обзор и фотопост в Инстаграм',
 'YouTube охват позиции интеграция в обзор и фотопост в Инстаграм',
 'YouTube стоимость интеграция YouTube и Zen',
 'YouTube охват позиции интеграция YouTube и Zen',
 'YouTube стоимость интеграция на Основном канале + Романов Говорит',
 'YouTube охват позиции интеграция на Основном канале + Романов Говорит',
 'YouTube стоимость интеграция и фотопост в Инстаграм',
 'YouTube охват позиции интеграция и фотопост в Инстаграм',
 'YouTube стоимость спецпроект на YouTube и пост в Телеграм',
 'YouTube охват позиции спецпроект на YouTube и пост в Телеграм',
 'YouTube стоимость интеграция в рецепт и фотопост в Инстаграм',
 'YouTube охват позиции интеграция в рецепт и фотопост в Инстаграм',
 'YouTube стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
 'YouTube охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
 'YouTube стоимость интеграция в видео на YouTube и пост в сообществе YouTube',
 'YouTube охват позиции интеграция в видео на YouTube и пост в сообществе YouTube',
 'YouTube стоимость интеграция на YouTube и сторис в Инстаграм',
 'YouTube охват позиции интеграция на YouTube и сторис в Инстаграм',
 'YouTube стоимость усложненная интеграция',
 'YouTube стоимость пост в сообществе',
 'YouTube стоимость выездная интеграция',
 'YouTube стоимость интеграция с выбором темы видео',
 'Instagram стоимость 1 сторис',
 'Instagram стоимость карусель',
 'Instagram стоимость сториз (5 экранов)',
 'Instagram стоимость рекламный день (сториз 7-10 экранов)',
 'Instagram стоимость пост в Телеграм и сторис в Инстаграм',
 'Instagram охват позиции стоимость за пост в Телеграм и сторис в Инстаграм',
 'Instagram стоимость интеграция в видео на Ютуб и фотопост в Инстаграм',
 'Instagram охват позиции стоимость за интеграцию в видео на Ютуб и фотопост в Инстаграм',
 'Instagram стоимость фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
 'Instagram охват позиции стоимость за фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
 'Instagram стоимость интеграция в рецепт и фотопост в Инстаграм',
 'Instagram охват позиции стоимость за интеграцию в рецепт и фотопост в Инстаграм',
 'Instagram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
 'Instagram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
 'Instagram стоимость интеграция на Ютуб и сторис в Инстаграм',
 'Instagram охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм',
 'Instagram стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
 'Instagram охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
 'Instagram стоимость интеграция в обзор и фотопост в Инстаграм',
 'Instagram охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм',
 'Instagram стоимость серия из 3х сторис и пост в Телеграм',
 'Instagram охват позиции стоимость за серию из 3х сторис и пост в Телеграм',
 'Instagram стоимость фотопост в Инстаграм + пост в Телеграм',
 'Instagram охват позиции стоимость фотопост в Инстаграм + пост в Телеграм',
 'Telegram стоимость за авторский пост',
 'Telegram стоимость за пост из удаления',
 'Telegram стоимость фотопост в Инстаграм и пост в Телеграм',
 'Telegram охват позиции стоимость за фотопост в Инстаграм и пост в Телеграм',
 'Telegram стоимость спецпроект на Ютуб и пост в Телеграм',
 'Telegram охват позиции стоимость за спецпроект на Ютуб и пост в Телеграм',
 'Telegram стоимость пост в Телеграм и сториз в Инстаграм',
 'Telegram охват позиции стоимость за пост в Телеграм и сториз в Инстаграм',
 'Telegram стоимость 5 постов в Телеграм (удаление через сутки)',
 'Telegram охват позиции стоимость за 5 постов в Телеграм (удаление через сутки)',
 'Telegram стоимость 3 поста в Телеграм',
 'Telegram охват позиции стоимость за 3 поста в Телеграм',
 'Telegram стоимость рекламное сообщение во все чаты "Вместе"',
 'Telegram охват позиции стоимость за рекламное сообщение во все чаты "Вместе"',
 'Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
 'Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
 'Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм',
 'Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм',
 'VK группа стоимость сторис',
 'VK группа стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
 'VK группа охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
 'VK группа стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
 'VK группа охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
 'Дзен стоимость интеграция до 1 мин',
 'Дзен стоимость статья',
 'Дзен стоимость за видеопост',

 'Дзен стоимость за модерацию комментариев',
 'Дзен стоимость за выездную интеграцию в видео',
 'Дзен стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм',
 'Дзен охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм',

 'Подкаст количество подписчиков',
 'Подкаст среднее кол-во прослушиваний',
 'Подкаст стоимость рекламная вставка',
 'Подкаст стоимость партнерская рубрика',

 'Instagram стоимость видеопост и серия сторис',
 'Instagram стоимость фотопост и 6-7 сторис',

 'Telegram стоимость текстовый пост с закрепом',
 'Telegram стоимость текстовый пост без закрепа']

In [650]:
difference1 = list(set(column_names_list).difference(spisok_columns))
difference1

['TikTok ссылка',
 'YouTube Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9',
 'VK Видео статистика',
 'Telegram стоимость за пост (2/48)',
 'Telegram стоимость 1 сторис до 15 сек',
 'RuTube город',
 'Yappy блогер',
 'Likee тематика',
 'Telegram ссылка',
 'YouTube проект, ведущий, формат',
 'OK Шоу блогер',
 'Instagram ца по возрасту',
 'Instagram статистика',
 'Дзен ссылка',
 'Threads тематика',
 'Telegram стоимость за пост (3/72)',
 'YouTube статистика',
 'YouTube ссылка',
 'Дзен статистика',
 'Likee город',
 'Подкаст ссылка',
 'Twitch ссылка',
 'тематика',
 'Instagram материалы и статистика',
 'Threads блогер',
 'VK Видео блогер',
 'Instagram тематика',
 'VK группа ца по полу',
 'Подкаст блогер',
 'Подкаст статистика',
 'Threads ссылка',
 'контакты менеджера',
 'YouTube Преролл в формат «Свой/чужой»_3',
 'Instagram город',
 'Дзен город',
 'VK группа блогер',
 'YouTube Пакет соц. сети (сторис ИГ + пост ВК + пост ТГ) (пост в формате «текст пр

In [655]:
exclude_words = ['город',
                 'ссылка',
                 'статистика',
                 'блогер',
                 'тематика',
                 'ца по возрасту',
                 'проект, ведущий, формат',
                 'материалы и статистика',
                 'ца по полу',
                 'контакты менеджера',
                 'название канала',
                 'ца по городам',
                 'шоу',
                 'ца по странам',
                 'налог']

# Создаем список из названий столбцов, исключая те, которые содержат определённые слова
spisok_columns = [col for col in itog.columns if not any(word in col for word in exclude_words)]

In [657]:
for columns in spisok_columns:
    itog[columns] = itog[columns].astype(str)
    itog[columns] = itog[columns].str.replace('р.', '', regex=True)
    itog[columns] = itog[columns].str.replace(r'(\d+),(\d+)', r'\1.\2', regex=True)
    itog[columns] = itog[columns].str.replace('[^\d.]', '', regex=True)
    itog[columns] = itog[columns].str.replace(' ', '')
    itog[columns] = itog[columns].replace('', np.nan)
    itog[columns] = itog[columns].astype(float)